# 主要代码

## 功能性函数

### utils_pps: VFI+数值模拟

In [ ]:
classdef utils_pps
    methods (Static)


        %% =====================================================================
        %                       1. 参数设置函数
        %  =====================================================================
        function cS = setup_parameters(varargin)
            fprintf('===== 步骤 1: 设置模型参数 =====\n');

            cS = struct();

            % --- A. 生命周期与人口结构 ---
            cS.tb = 22;       % 初始工作年龄
            cS.tr = 61;       % 退休年龄
            cS.td = 100;      % 最高寿命
            cS.tn = cS.td - cS.tb + 1; % 总期数

            % --- B. 经济主体偏好 (Epstein-Zin) ---
            cS.beta = 0.95;     % 折现因子
            cS.gamma = 3.84;    % 相对风险规避系数
            cS.psi = 0.15;      % 跨期替代弹性

            % --- C. 收入过程 ---
            % 确定性部分 g(t) = exp(aa + b1*t + b2*t^2 + b3*t^3)
            cS.aa = -2.170042 + 2.700381;
            cS.b1 = 0.16818;
            cS.b2 = -0.0323371 / 10;
            cS.b3 = 0.0019704 / 100;
            % 随机部分
            cS.smay = sqrt(0.169993); % 暂时冲击标准差 (u_t)
            cS.smav = sqrt(0.112572); % 持久冲击标准差 (z_t)
            cS.corr_z_epsilon = 0; % 持久冲击与风险收益的相关性 (未使用)
            cS.corr_u_epsilon = 0; % 暂时冲击与风险收益的相关性 (未使用)

            % --- D. 资产与回报率 ---
            cS.rf = 1.02;     % 无风险总回报率
            cS.mu = 0.04;     % 风险资产超额回报率
            cS.sigr = 0.27;   % 风险资产回报率标准差
            cS.rp = 1.04;     % 个人养老金账户(PPS)回报率
            cS.sigrrp = 0.27; % 个人养老金账户(PPS)风险资产回报率标准差

            % --- E. 养老金与税收 ---
            cS.ret_fac = 0.6827;      % 退休后基础养老金 (替代率)
            cS.tau_y = 0.06;          % 工资税率
            cS.tau_q = 0.03;          % 个人养老金提取税率
            % cS.Q_max 将在下方通过新函数计算


            % --- F. 数值求解参数 ---
            cS.n_shocks = 5;      % 离散化的随机冲击节点数
            cS.ncash = 51;        % 现金持有量网格数
            cS.nfund = 51;        % 养老金账户网格数
            cS.maxcash = 200;     % 现金网格上限 (归一化)
            cS.mincash = 0.25;    % 现金网格下限 (归一化)
            cS.maxfund = 200;     % 养老金网格上限 (归一化)
            cS.minfund = 0.01;    % 养老金网格下限 (归一化)
            cS.save_path = 'result/'; % 结果保存路径

                        % 生存概率
            cS.survprob = zeros(cS.tn - 1, 1);
            cS.survprob_data = [0.99975, 0.99974, 0.99973, 0.99972, 0.99971, 0.99969, 0.99968, 0.99966, 0.99963, 0.99961, ...
                0.99958, 0.99954, 0.99951, 0.99947, 0.99943, 0.99938, 0.99934, 0.99928, 0.99922, 0.99916, ...
                0.99908, 0.999, 0.99891, 0.99881, 0.9987, 0.99857, 0.99843, 0.99828, 0.99812, 0.99794, ...
                0.99776, 0.99756, 0.99735, 0.99713, 0.9969, 0.99666, 0.9964, 0.99613, 0.99583, 0.99551, ...
                0.99515, 0.99476, 0.99432, 0.99383, 0.9933, 0.9927, 0.99205, 0.99133, 0.99053, 0.98961, ...
                0.98852, 0.98718, 0.98553, 0.98346, 0.98089, 0.97772, 0.97391, 0.96943, 0.96429, 0.95854, ...
                0.95221, 0.94537, 0.93805, 0.93027, 0.92202, 0.91327, 0.90393, 0.89389, 0.88304, 0.87126, ...
                0.85846, 0.84452, 0.82935, 0.81282, 0.79485, 0.77543, 0.75458, 0.7324, 0.70893, 0.68424, ...
                0.68424, 0.68424];

                        % *** 新增: 调用函数计算归一化的 Q_max ***
            % cS.Q_max = utils_pps.calculate_q_max(absolute_q_max_yuan, anchor_income_yuan, cS.f_y);
            cS.Q_max =99999;

            if isempty(varargin) % 便于敏感性分析时改动参数而不进一步计算派生参数
                cS = utils_pps.setup_process(cS);
            end
           
        end

        function cS = setup_process(cS)
            % --- G. 派生参数与预计算 ---
            cS.pension_rate = 1 / (cS.td - cS.tr + 1); % 退休后年金化给付比率
            % 效用函数参数
            cS.psi_1 = 1.0 - 1.0 / cS.psi;
            cS.psi_2 = 1.0 / cS.psi_1;
            cS.theta = (1.0 - cS.gamma) / (1.0 - cS.psi_1);


            for i = 1:min(length(cS.survprob_data), length(cS.survprob))
                cS.survprob(i) = cS.survprob_data(i);
            end

            % 离散化正态分布 (Tauchen方法)
            tauchenoptions.parallel=0;
            [grid, weig_matrix] = utils_pps.discretizeAR1_Tauchen(0, 0, 1, cS.n_shocks, 2, tauchenoptions);
            cS.shock_grid = grid;
            cS.shock_weig = diag(weig_matrix);

            % 风险资产回报率网格
            cS.gret = cS.rf + cS.mu + cS.shock_grid * cS.sigr;
            cS.gret_rp = cS.rp + cS.shock_grid * cS.sigrrp;

            % 状态转移概率 (三维冲击)
            cS.nweig1 = zeros(cS.n_shocks, cS.n_shocks, cS.n_shocks);
            for i6 = 1:cS.n_shocks
                for i7 = 1:cS.n_shocks
                    for i8 = 1:cS.n_shocks
                        cS.nweig1(i6, i7, i8) = cS.shock_weig(i6) * cS.shock_weig(i7) * cS.shock_weig(i8);
                    end
                end
            end

            % 状态变量网格
            lgcash = linspace(log(cS.mincash), log(cS.maxcash), cS.ncash)';
            cS.gcash = exp(lgcash);
            lgfund = linspace(log(cS.minfund), log(cS.maxfund), cS.nfund)';
            cS.gfund = exp(lgfund);
            cS.gfund(1) = 0; % 确保可以从0开始

            % --- 收入过程计算 ---
            % 确定性收入剖面
            cS.f_y = zeros(cS.tr - cS.tb + 1, 1);
            for i1 = cS.tb:cS.tr
                age = i1;
                cS.f_y(age - cS.tb + 1) = exp(cS.aa + cS.b1*age + cS.b2*age^2 + cS.b3*age^3);
            end


            % 收入过程网格
            cS.yh = zeros(cS.n_shocks, cS.n_shocks); % 暂时冲击 exp(u_t)
            cS.yp = zeros(cS.n_shocks, cS.n_shocks); % 持久冲击增长 exp(z_t)
            cS.gyp = zeros(cS.n_shocks, cS.n_shocks, cS.tn - 1); % 总收入增长

            for i1 = 1:cS.n_shocks
                grid2_u = cS.shock_grid(i1) .* cS.corr_u_epsilon + cS.shock_grid .* sqrt(1 - cS.corr_u_epsilon^2);
                cS.yh(:, i1) = exp(grid2_u * cS.smay);

                grid2_z = cS.shock_grid(i1) .* cS.corr_z_epsilon + cS.shock_grid .* sqrt(1 - cS.corr_z_epsilon^2);
                cS.yp(:, i1) = exp(grid2_z * cS.smav);
            end

            % 工作期收入增长
            work_periods = cS.tr - cS.tb;
            for t = 1:work_periods
                G_t = cS.f_y(t+1) / cS.f_y(t); % 确定性增长
                cS.gyp(:,:,t) = repmat(G_t, cS.n_shocks, cS.n_shocks) .* cS.yp;
            end

            % 退休期收入增长 (无增长)
            cS.gyp(:,:,(work_periods+1):(cS.tn-1)) = 1.0;

            fprintf('参数设置完成。\n');
        end

        %
        %% =====================================================================
        %                       2. VFI 求解器函数 (修正版)
        %  =====================================================================
        function results = solve_model_vfi(cS)
            % --- 初始化 ---
            % 启动并行池
            if isempty(gcp('nocreate'))
                parpool('local');
            end

            % 初始化策略和价值函数数组
            V = zeros(cS.ncash, cS.nfund, cS.tn);
            C_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            A_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            Q_policy = zeros(cS.ncash, cS.nfund, cS.tn);

            % --- 终端期 (t=T) ---
            % Epstein-Zin效用下的终端价值 V_T = ((1-beta)*C_T^(1-1/psi))^(1/(1-1/psi))
            % 其中 C_T = W_T (即 cS.gcash), V_T = (1-beta)^psi_2 * W_T
            for i_fund = 1:cS.nfund
                V(:, i_fund, cS.tn) = cS.gcash * (1-cS.beta)^cS.psi_2;
                C_policy(:, i_fund, cS.tn) = cS.gcash;
            end
            A_policy(:, :, cS.tn) = 0.0;
            Q_policy(:, :, cS.tn) = 0.0;

            % --- 逆向归纳求解 ---
            fprintf('开始逆向归纳求解...\n');
            options = optimoptions('fmincon', 'Display', 'off', 'Algorithm', 'interior-point', ...
                'MaxFunctionEvaluations', 1000, 'MaxIterations', 400, ...
                'OptimalityTolerance', 1e-10, 'StepTolerance', 1e-10, ...
                'ConstraintTolerance', 1e-10);

            % -- 退休期 (t = T-1, ..., K) --
            fprintf('求解退休期...\n');
            retire_start_time = tic;
            for t = (cS.tn-1):-1:(cS.tr-cS.tb+1)
                fprintf('  t = %d (年龄 %d)\n', t, t + cS.tb - 1);
                V_next = squeeze(V(:,:,t+1)); % 下一期值函数

                % *** 修改: 创建 griddedInterpolant 对象 ***
                % V_next 维度 (ncash, nfund)。行对应gcash, 列对应gfund
                % griddedInterpolant({Y,X}, V)


                temp_C = zeros(cS.ncash, cS.nfund);
                temp_A = zeros(cS.ncash, cS.nfund);
                temp_V = zeros(cS.ncash, cS.nfund);

                parfor i_fund = 1:cS.nfund

                    local_C = zeros(cS.ncash, 1);
                    local_A = zeros(cS.ncash, 1);
                    local_V = zeros(cS.ncash, 1);

                    gfund_at_retirement = cS.gfund(i_fund);
                    constant_pension_payout = (1 - cS.tau_q) * gfund_at_retirement * cS.pension_rate;

                    V_next_interp_obj = griddedInterpolant(cS.gcash, V_next(:,i_fund),  'spline', 'linear');

                    for i_cash = 1:cS.ncash
                        gcash_val = cS.gcash(i_cash);

                        lb = [1e-6, 0];
                        x0 = [0.5 * gcash_val, 1-1e-06];
                        if i_cash > 1 && local_C(i_cash-1) > 0
                            x0 = [local_C(i_cash-1), local_A(i_cash-1)];
                            % lb = [local_C(i_cash-1)-1e-06, 0];
                        end

                        ub = [gcash_val, 1];

                        % *** 修改: 将插值对象传递给目标函数 ***
                        obj_fun = @(x) utils_pps.fun_valuefunc_retired(x, gcash_val, constant_pension_payout, i_fund, V_next_interp_obj, cS, t);

                        [policy, fval] = fmincon(obj_fun, x0, [], [], [], [], lb, ub, [], options);

                        local_C(i_cash) = policy(1);
                        % if abs(policy(1)-gcash_val)<1e-05
                        %     local_A(i_cash) = nan; % C = Cash
                        % else
                        local_A(i_cash) = policy(2);
                        % end
                        local_V(i_cash) = -fval;
                    end
                    temp_C(:, i_fund) = local_C;
                    temp_A(:, i_fund) = local_A;
                    temp_V(:, i_fund) = local_V;
                end
                C_policy(:, :, t) = temp_C;
                A_policy(:, :, t) = temp_A;
                V(:, :, t) = temp_V;
            end
            fprintf('退休期求解完成，耗时 %.2f 分钟\n', toc(retire_start_time)/60);

            % -- 工作期 (t = K-1, ..., 1) --
            fprintf('求解工作期...\n');
            work_start_time = tic;
            for t = (cS.tr - cS.tb):-1:1
                fprintf('  t = %d (年龄 %d)\n', t, t + cS.tb - 1);
                V_next = squeeze(V(:,:,t+1));

                % *** 修改: 创建 griddedInterpolant 对象 ***
                [X,Y] = ndgrid(cS.gcash,cS.gfund);
                V_next_interp_obj = griddedInterpolant(X, Y, V_next,  'spline','linear');

                temp_C = zeros(cS.ncash, cS.nfund);
                temp_A = zeros(cS.ncash, cS.nfund);
                temp_Q = zeros(cS.ncash, cS.nfund);
                temp_V = zeros(cS.ncash, cS.nfund);

                parfor i_fund = 1:cS.nfund
                    local_C = zeros(cS.ncash, 1);
                    local_A = zeros(cS.ncash, 1);
                    local_Q = zeros(cS.ncash, 1);
                    local_V = zeros(cS.ncash, 1);

                    gfund_val = cS.gfund(i_fund);
                    for i_cash = 1:cS.ncash
                        gcash_val = cS.gcash(i_cash);

                        % 2. 放弃在 lb 中施加单调性硬约束，Q 的下界永远是 0
                        lb = [1e-3, 0, 0];
                        % *** 核心修改 ***
                        if i_cash > 1
                            x0 = [local_C(i_cash-1), local_A(i_cash-1), local_Q(i_cash-1)];
                            % lb = [local_C(i_cash-1)-1e-6, 0, local_Q(i_cash-1)-1e-6];
                        else
                            x0 = [gcash_val * 0.5, 1-1e-06, 1e-6]; % C, alpha, Q
                        end

                        % 3. ub 只包含物理和政策约束
                        ub = [gcash_val, 1, min(cS.Q_max,gcash_val/(1 - cS.tau_y))];


                        % *** 修改: 将插值对象传递给目标函数 ***
                        obj_fun = @(x) utils_pps.fun_valuefunc_work(x, gcash_val, gfund_val, V_next_interp_obj, cS, t);

                        A_ineq = [1, 0, 1 - cS.tau_y];
                        b_ineq = gcash_val;

                        [policy, fval] = fmincon(obj_fun, x0, A_ineq, b_ineq, [], [], lb, ub, [], options);

                        local_C(i_cash) = policy(1);
                        % if abs(A_ineq*policy'-b_ineq)<1e-05
                        %     local_A(i_cash) = nan; % C+（1-tau_y）Q = Cash
                        % else
                        local_A(i_cash) = policy(2);
                        % end
                        local_Q(i_cash) = policy(3);
                        % if abs(policy(3))<0.0001
                        %     policy(3)
                        % end
                        local_V(i_cash) = -fval;
                    end
                    temp_C(:, i_fund) = local_C;
                    temp_A(:, i_fund) = local_A;
                    temp_Q(:, i_fund) = local_Q;
                    temp_V(:, i_fund) = local_V;
                end
                C_policy(:, :, t) = temp_C;
                A_policy(:, :, t) = temp_A;
                Q_policy(:, :, t) = temp_Q;
                V(:, :, t) = temp_V;
            end
            fprintf('工作期求解完成，耗时 %.2f 分钟\n', toc(work_start_time)/60);

            % --- 封装结果 ---
            results.V = V;
            results.C_policy = C_policy;
            results.A_policy = A_policy;
            results.Q_policy = Q_policy;
            % 平滑处理
            % results.A_policy(find(abs(cS.gcash-results.C_policy-(1-cS.tau_y)*results.Q_policy)<1e-05))=nan;
        end
        
        
        function results = solve_model_vfi_riskyrp(cS)
            % --- 初始化 ---
            % 启动并行池
            if isempty(gcp('nocreate'))
                parpool('local');
            end

            % 初始化策略和价值函数数组
            V = zeros(cS.ncash, cS.nfund, cS.tn);
            C_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            A_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            Q_policy = zeros(cS.ncash, cS.nfund, cS.tn);

            % --- 终端期 (t=T) ---
            % Epstein-Zin效用下的终端价值 V_T = ((1-beta)*C_T^(1-1/psi))^(1/(1-1/psi))
            % 其中 C_T = W_T (即 cS.gcash), V_T = (1-beta)^psi_2 * W_T
            for i_fund = 1:cS.nfund
                V(:, i_fund, cS.tn) = cS.gcash * (1-cS.beta)^cS.psi_2;
                C_policy(:, i_fund, cS.tn) = cS.gcash;
            end
            A_policy(:, :, cS.tn) = 0.0;
            Q_policy(:, :, cS.tn) = 0.0;

            % --- 逆向归纳求解 ---
            fprintf('开始逆向归纳求解...\n');
            options = optimoptions('fmincon', 'Display', 'off', 'Algorithm', 'interior-point', ...
                'MaxFunctionEvaluations', 1000, 'MaxIterations', 400, ...
                'OptimalityTolerance', 1e-10, 'StepTolerance', 1e-10, ...
                'ConstraintTolerance', 1e-10);

            % -- 退休期 (t = T-1, ..., K) --
            fprintf('求解退休期...\n');
            retire_start_time = tic;
            for t = (cS.tn-1):-1:(cS.tr-cS.tb+1)
                fprintf('  t = %d (年龄 %d)\n', t, t + cS.tb - 1);
                V_next = squeeze(V(:,:,t+1)); % 下一期值函数

                % *** 修改: 创建 griddedInterpolant 对象 ***
                % V_next 维度 (ncash, nfund)。行对应gcash, 列对应gfund
                % griddedInterpolant({Y,X}, V)


                temp_C = zeros(cS.ncash, cS.nfund);
                temp_A = zeros(cS.ncash, cS.nfund);
                temp_V = zeros(cS.ncash, cS.nfund);

                parfor i_fund = 1:cS.nfund

                    local_C = zeros(cS.ncash, 1);
                    local_A = zeros(cS.ncash, 1);
                    local_V = zeros(cS.ncash, 1);

                    gfund_at_retirement = cS.gfund(i_fund);
                    constant_pension_payout = (1 - cS.tau_q) * gfund_at_retirement * cS.pension_rate;

                    V_next_interp_obj = griddedInterpolant(cS.gcash, V_next(:,i_fund),  'spline', 'linear');

                    for i_cash = 1:cS.ncash
                        gcash_val = cS.gcash(i_cash);

                        lb = [1e-6, 0];
                        x0 = [0.5 * gcash_val, 1-1e-06];
                        if i_cash > 1 && local_C(i_cash-1) > 0
                            x0 = [local_C(i_cash-1), local_A(i_cash-1)];
                            % lb = [local_C(i_cash-1)-1e-06, 0];
                        end

                        ub = [gcash_val, 1];

                        % *** 修改: 将插值对象传递给目标函数 ***
                        obj_fun = @(x) utils_pps.fun_valuefunc_retired(x, gcash_val, constant_pension_payout, i_fund, V_next_interp_obj, cS, t);

                        [policy, fval] = fmincon(obj_fun, x0, [], [], [], [], lb, ub, [], options);

                        local_C(i_cash) = policy(1);
                        % if abs(policy(1)-gcash_val)<1e-05
                        %     local_A(i_cash) = nan; % C = Cash
                        % else
                        local_A(i_cash) = policy(2);
                        % end
                        local_V(i_cash) = -fval;
                    end
                    temp_C(:, i_fund) = local_C;
                    temp_A(:, i_fund) = local_A;
                    temp_V(:, i_fund) = local_V;
                end
                C_policy(:, :, t) = temp_C;
                A_policy(:, :, t) = temp_A;
                V(:, :, t) = temp_V;
            end
            fprintf('退休期求解完成，耗时 %.2f 分钟\n', toc(retire_start_time)/60);

            % -- 工作期 (t = K-1, ..., 1) --
            fprintf('求解工作期...\n');
            work_start_time = tic;
            for t = (cS.tr - cS.tb):-1:1
                fprintf('  t = %d (年龄 %d)\n', t, t + cS.tb - 1);
                V_next = squeeze(V(:,:,t+1));

                % *** 修改: 创建 griddedInterpolant 对象 ***
                [X,Y] = ndgrid(cS.gcash,cS.gfund);
                V_next_interp_obj = griddedInterpolant(X, Y, V_next,  'spline','linear');

                temp_C = zeros(cS.ncash, cS.nfund);
                temp_A = zeros(cS.ncash, cS.nfund);
                temp_Q = zeros(cS.ncash, cS.nfund);
                temp_V = zeros(cS.ncash, cS.nfund);

                parfor i_fund = 1:cS.nfund
                    local_C = zeros(cS.ncash, 1);
                    local_A = zeros(cS.ncash, 1);
                    local_Q = zeros(cS.ncash, 1);
                    local_V = zeros(cS.ncash, 1);

                    gfund_val = cS.gfund(i_fund);
                    for i_cash = 1:cS.ncash
                        gcash_val = cS.gcash(i_cash);

                        % 2. 放弃在 lb 中施加单调性硬约束，Q 的下界永远是 0
                        lb = [1e-3, 0, 0];
                        % *** 核心修改 ***
                        if i_cash > 1
                            x0 = [local_C(i_cash-1), local_A(i_cash-1), local_Q(i_cash-1)];
                            % lb = [local_C(i_cash-1)-1e-6, 0, local_Q(i_cash-1)-1e-6];
                        else
                            x0 = [gcash_val * 0.5, 1-1e-06, 1e-6]; % C, alpha, Q
                        end

                        % 3. ub 只包含物理和政策约束
                        ub = [gcash_val, 1, min(cS.Q_max,gcash_val/(1 - cS.tau_y))];


                        % *** 修改: 将插值对象传递给目标函数 ***
                        obj_fun = @(x) utils_pps.fun_valuefunc_work_riskrp(x, gcash_val, gfund_val, V_next_interp_obj, cS, t);

                        A_ineq = [1, 0, 1 - cS.tau_y];
                        b_ineq = gcash_val;

                        [policy, fval] = fmincon(obj_fun, x0, A_ineq, b_ineq, [], [], lb, ub, [], options);

                        local_C(i_cash) = policy(1);
                        % if abs(A_ineq*policy'-b_ineq)<1e-05
                        %     local_A(i_cash) = nan; % C+（1-tau_y）Q = Cash
                        % else
                        local_A(i_cash) = policy(2);
                        % end
                        local_Q(i_cash) = policy(3);
                        % if abs(policy(3))<0.0001
                        %     policy(3)
                        % end
                        local_V(i_cash) = -fval;
                    end
                    temp_C(:, i_fund) = local_C;
                    temp_A(:, i_fund) = local_A;
                    temp_Q(:, i_fund) = local_Q;
                    temp_V(:, i_fund) = local_V;
                end
                C_policy(:, :, t) = temp_C;
                A_policy(:, :, t) = temp_A;
                Q_policy(:, :, t) = temp_Q;
                V(:, :, t) = temp_V;
            end
            fprintf('工作期求解完成，耗时 %.2f 分钟\n', toc(work_start_time)/60);

            % --- 封装结果 ---
            results.V = V;
            results.C_policy = C_policy;
            results.A_policy = A_policy;
            results.Q_policy = Q_policy;
            % 平滑处理
            % results.A_policy(find(abs(cS.gcash-results.C_policy-(1-cS.tau_y)*results.Q_policy)<1e-05))=nan;
        end

        %% =====================================================================
        %                VFI - 退休期值函数
        %  =====================================================================
        function v = fun_valuefunc_retired(x, cash, pension_payout, i_fund_K, V_next_interp_obj, cS, t)
            % x = [消费比例 c_rate, 风险资产比例 alpha]
            % pension_payout: 这是一个根据F_K计算出的固定值
            % i_fund_K: 这是F_K在网格上的索引，用于插值V_next，因为它在退休期间不变

            alpha = x(2);

            consumption = x(1);
            if consumption <= 1e-8, v = 1e10; return; end

            sav = cash - consumption;
            if sav < 0, v = 1e10; return; end

            % Epstein-Zin 瞬时效用项
            u_term = (1 - cS.beta) * (consumption^cS.psi_1);

            % 下一期现金持有量 (归一化)
            ret_fac_norm = cS.ret_fac;
            cash_1 = (cS.rf * (1 - alpha) + cS.gret * alpha) * sav + ret_fac_norm + pension_payout;
            % cash_1 = max(min(cash_1, cS.gcash(end)), cS.gcash(1));

            % 下一期养老金状态 F_{t+1} 就是 F_K
            fund_1 = repmat(cS.gfund(i_fund_K), cS.n_shocks, 1);

            % *** 修改: 使用 griddedInterpolant 对象进行插值 ***
            % 调用方式 F({Yq, Xq})
            int_V = V_next_interp_obj(cash_1);
            int_V(int_V <= 0) = 1e-9;

            % 计算期望
            expected_term = cS.shock_weig' * (int_V.^(1 - cS.gamma));

            % 完整的 Epstein-Zin 值函数 (退休后 gyp=1)
            value = (u_term + cS.beta * cS.survprob(t) * (expected_term)^(cS.psi_1 / (1 - cS.gamma)))^cS.psi_2;

            v = -value; % fmincon 最小化
        end

        %% =====================================================================
        %                VFI - 工作期值函数
        %  =====================================================================

        function v = fun_valuefunc_work(x, cash, fund, V_next_interp_obj, cS, t)
            % x = [绝对消费 C, 风险资产比例 alpha, 绝对养老金缴费 Q]
            consumption = x(1);
            alpha       = x(2);
            pension_contrib = x(3);

            if consumption <= 1e-8
                v = 1e10;
                return;
            end

            liquid_sav = cash - consumption - (1 - cS.tau_y) * pension_contrib;

            if liquid_sav < -1e-6
                v = 1e10;
                return;
            end

            % Epstein-Zin 瞬时效用项
            u_term = (1 - cS.beta) * (consumption^cS.psi_1);

            % 构造下一期状态的网格 (三维冲击)
            n = cS.n_shocks;
            gyp_3d = repmat(reshape(cS.gyp(:,:,t), [n, 1, n]), [1, n, 1]);
            gret_3d = repmat(reshape(cS.gret, [1, 1, n]), [n, n, 1]);
            yh_3d = repmat(reshape(cS.yh, [1, n, n]), [n, 1, 1]);

            % 下一期归一化状态变量
            portfolio_return = cS.rf * (1 - alpha) + gret_3d * alpha;
            cash_1 = portfolio_return .* (liquid_sav ./ gyp_3d) + (1-cS.tau_y) .* yh_3d;
            fund_1 = (fund + pension_contrib) * cS.rp ./ gyp_3d;

            % cash_1 = max(min(cash_1, cS.gcash(end)), cS.gcash(1));
            % fund_1 = max(min(fund_1, cS.gfund(end)), cS.gfund(1));

            % *** 修改: 使用 griddedInterpolant 对象进行插值 ***
            % 调用方式 F({Yq, Xq})
            int_V = V_next_interp_obj(cash_1, fund_1);
            int_V(int_V <= 0) = 1e-9;

            % 构造期望项 (包含归一化因子的增长)
            term_inside_exp = (gyp_3d .* int_V).^(1 - cS.gamma);
            expected_term = sum(cS.nweig1 .* term_inside_exp, 'all');

            % 完整的 Epstein-Zin 值函数
            value = (u_term + cS.beta * cS.survprob(t) * (expected_term)^(cS.psi_1 / (1 - cS.gamma)))^cS.psi_2;

            v = -value; % fmincon 最小化
        end


        function v = fun_valuefunc_work_riskrp(x, cash, fund, V_next_interp_obj, cS, t)
            % x = [绝对消费 C, 风险资产比例 alpha, 绝对养老金缴费 Q]
            consumption = x(1);
            alpha       = x(2);
            pension_contrib = x(3);

            if consumption <= 1e-8
                v = 1e10;
                return;
            end

            liquid_sav = cash - consumption - (1 - cS.tau_y) * pension_contrib;

            if liquid_sav < -1e-6
                v = 1e10;
                return;
            end

            % Epstein-Zin 瞬时效用项
            u_term = (1 - cS.beta) * (consumption^cS.psi_1);

            % 构造下一期状态的网格 (三维冲击)
            n = cS.n_shocks;
            gyp_3d = repmat(reshape(cS.gyp(:,:,t), [n, 1, n]), [1, n, 1]);
            gret_3d = repmat(reshape(cS.gret, [1, 1, n]), [n, n, 1]);
            gret_rp_3d = repmat(reshape(cS.gret_rp, [1, 1, n]), [n, n, 1]);
            yh_3d = repmat(reshape(cS.yh, [1, n, n]), [n, 1, 1]);

            % 下一期归一化状态变量
            portfolio_return = cS.rf * (1 - alpha) + gret_3d * alpha;
            cash_1 = portfolio_return .* (liquid_sav ./ gyp_3d) + (1-cS.tau_y) .* yh_3d;
            fund_1 = (fund + pension_contrib) * gret_rp_3d ./ gyp_3d;

            % cash_1 = max(min(cash_1, cS.gcash(end)), cS.gcash(1));
            % fund_1 = max(min(fund_1, cS.gfund(end)), cS.gfund(1));

            % *** 修改: 使用 griddedInterpolant 对象进行插值 ***
            % 调用方式 F({Yq, Xq})
            int_V = V_next_interp_obj(cash_1, fund_1);
            int_V(int_V <= 0) = 1e-9;

            % 构造期望项 (包含归一化因子的增长)
            term_inside_exp = (gyp_3d .* int_V).^(1 - cS.gamma);
            expected_term = sum(cS.nweig1 .* term_inside_exp, 'all');

            % 完整的 Epstein-Zin 值函数
            value = (u_term + cS.beta * cS.survprob(t) * (expected_term)^(cS.psi_1 / (1 - cS.gamma)))^cS.psi_2;

            v = -value; % fmincon 最小化
        end

        %% =====================================================================
        %                       3. 模拟器函数
        %  =====================================================================
        function simulate_model(results, cS)
            % --- 初始化 ---
            fprintf('模拟开始...\n');
            rng(42); % 可复现性
            nsim = 10000; % 模拟个体数量

            % 解包策略函数
            C_policy = results.C_policy;
            A_policy = results.A_policy;
            Q_policy = results.Q_policy;

            % 初始化模拟数组 (所有变量首先在归一化单位下计算)
            % 冲击和外生过程
            simGPY = ones(cS.tn, nsim);       % 持久收入增长因子 G_t * exp(z_t)
            simY_norm = zeros(cS.tn, nsim);    % 归一化暂时/基础收入 exp(u_t) 或 ret_fac
            simR = zeros(cS.tn, nsim);         % 风险回报率

            % 归一化单位下的决策和状态变量
            simW_norm = zeros(cS.tn, nsim);    % 归一化流动性财富 w_t = W_t/P_t
            simF_norm = zeros(cS.tn, nsim);    % 归一化养老金 f_t = F_t/P_t
            simC_norm = zeros(cS.tn, nsim);    % 归一化消费 c_t = C_t/P_t
            simQ_norm = zeros(cS.tn, nsim);    % 归一化缴费 q_t = Q_t/P_t
            simA = zeros(cS.tn, nsim);         % 风险资产配置 alpha_t
            simS_norm = zeros(cS.tn, nsim);    % 归一化风险资产 s_t
            simB_norm = zeros(cS.tn, nsim);    % 归一化无风险资产 b_t
            simQ_Cash_ratio = zeros(cS.tn, nsim); % 缴费与现金流比率 q/cash

            % --- 1. 模拟外生冲击和收入过程 ---
            fprintf('  生成收入和回报率路径...\n');
            work_periods = cS.tr - cS.tb;

            % 使用 Antithetic Variates 方法减少模拟误差
            for i1 = 1:floor(nsim/2)
                z_shocks = randn(work_periods, 1) * cS.smav;
                u_shocks_log = randn(work_periods, 1) * cS.smay;
                r_shocks = randn(cS.tn, 1) * cS.sigr;

                % 正向冲击路径
                simGPY(2:work_periods+1, i1) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(z_shocks);
                simY_norm(1:work_periods, i1) = exp(u_shocks_log);
                simR(:, i1) = cS.rf + cS.mu + r_shocks;

                % 反向冲击路径 (镜像)
                i2 = nsim/2 + i1;
                simGPY(2:work_periods+1, i2) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(-z_shocks);
                simY_norm(1:work_periods, i2) = exp(-u_shocks_log);
                simR(:, i2) = cS.rf + cS.mu - r_shocks;
            end

            % 退休后基础养老金 (替代率)
            norm_base_pension = cS.ret_fac ;
            simY_norm(work_periods+1:cS.tn, :) = norm_base_pension;

            % --- 2. 迭代模拟生命周期决策 (在归一化单位下) ---
            fprintf('  模拟生命周期决策 (归一化单位)...\n');
            for t = 1:cS.tn
                if t <= work_periods % 工作期
                    norm_cash = simW_norm(t, :) + (1-cS.tau_y) * simY_norm(t, :);
                else % 退休期
                    norm_pension_payout = (1-cS.tau_q) * simF_norm(t, :) * cS.pension_rate;
                    norm_cash = simW_norm(t, :) + simY_norm(t, :) + norm_pension_payout;
                end

                % 为当前期 t 的所有策略函数创建插值器
                Fc_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, C_policy(:,:,t), 'spline', 'linear');
                Fa_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, A_policy(:,:,t), 'spline', 'linear');
                Fq_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, Q_policy(:,:,t), 'spline', 'linear');

                % 对所有 nsim 个体进行插值，得到决策
                simC_norm(t, :) = Fc_interpolant(norm_cash, simF_norm(t, :));
                simA(t, :) = Fa_interpolant(norm_cash, simF_norm(t, :));
                simQ_norm(t, :) = Fq_interpolant(norm_cash, simF_norm(t, :));

                % 应用约束和边界条件
                simA(t, :) = max(min(simA(t, :), 1), 0);
                simC_norm(t, :) = max(min(simC_norm(t, :), norm_cash), 0);
                simQ_norm(t, :) = max(min(simQ_norm(t, :), cS.Q_max), 0);

                if t <= work_periods % 工作期
                    total_outflow = simC_norm(t, :) + (1-cS.tau_y)*simQ_norm(t, :);
                    idx_exceed = total_outflow > norm_cash;
                    if any(idx_exceed)
                        scale_factor = norm_cash(idx_exceed) ./ total_outflow(idx_exceed) * 0.9999;
                        simC_norm(t, idx_exceed) = simC_norm(t, idx_exceed) .* scale_factor;
                        simQ_norm(t, idx_exceed) = simQ_norm(t, idx_exceed) .* scale_factor;
                    end
                    simQ_Cash_ratio(t, :) = simQ_norm(t, :) ./ norm_cash;
                    simQ_Cash_ratio(t, isnan(simQ_Cash_ratio(t,:)) | isinf(simQ_Cash_ratio(t,:))) = 0;
                    liquid_sav_norm = norm_cash - simC_norm(t, :) - (1-cS.tau_y)*simQ_norm(t, :);
                else % 退休期
                    simQ_norm(t,:) = 0;
                    simC_norm(t,:) = min(simC_norm(t,:), norm_cash * 0.9999);
                    liquid_sav_norm = norm_cash - simC_norm(t, :);
                end

                simS_norm(t, :) = simA(t, :) .* liquid_sav_norm;
                simB_norm(t, :) = liquid_sav_norm - simS_norm(t, :);

                % 更新下一期状态 (归一化单位)
                if t < cS.tn
                    portfolio_return_next = simB_norm(t, :) * cS.rf + simS_norm(t, :) .* simR(t, :);
                    simW_norm(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);

                    if t < work_periods
                        simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
                    elseif t == work_periods % 退休前最后一年
                        simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
                    else % 退休后
                        simF_norm(t+1, :) = simF_norm(t, :);
                    end
                end
            end
            fprintf('  归一化单位模拟完成。\n');

            % --- 3. 将结果转换为以人民币计价的真实数值 ---
            fprintf('  将模拟结果转换为真实人民币单位...\n');

            % **锚定参数**
            initial_income_yuan = 60000; % 假设22岁时平均年持久性收入为 60,000 元

            % 初始化真实单位数组
            simP_real = zeros(cS.tn, nsim); % 真实持久性收入 (CNY)
            simY_real = zeros(cS.tn, nsim); % 真实总收入 (CNY)
            simW_real = zeros(cS.tn, nsim); % 真实流动性财富 (CNY)
            simF_real = zeros(cS.tn, nsim); % 真实养老金财富 (CNY)
            simC_real = zeros(cS.tn, nsim); % 真实消费 (CNY)
            simQ_real = zeros(cS.tn, nsim); % 真实养老金缴费 (CNY)

            % 模拟真实持久性收入路径 P_t
            simP_real(1, :) = initial_income_yuan; % 锚定初始值
            for t = 1:(cS.tn - 1)
                simP_real(t+1, :) = simP_real(t, :) .* simGPY(t+1, :);
            end

            % "去归一化" 所有相关变量
            for t = 1:cS.tn
                simW_real(t, :) = simW_norm(t, :) .* simP_real(t, :);
                simF_real(t, :) = simF_norm(t, :) .* simP_real(t, :);
                simC_real(t, :) = simC_norm(t, :) .* simP_real(t, :);
                simQ_real(t, :) = simQ_norm(t, :) .* simP_real(t, :);
                % 计算真实总收入
                if t <= work_periods
                    simY_real(t, :) = simP_real(t, :) .* simY_norm(t, :);
                else
                    simY_real(t, :) = simP_real(t, :);
                end
            end

            % --- 4. 汇总真实结果并绘图 (单位: 万元) ---
            fprintf('  汇总真实结果并绘图 (单位: 万元)...\n');
            yuan_to_wanyuan = 1 / 10000;
            ages = (cS.tb:cS.td)';
            x_fill = [ages; flipud(ages)]; % 用于填充置信区间的x坐标

            % --- 数据处理 ---
            % 均值
            meanC_plot = mean(simC_real, 2) * yuan_to_wanyuan;
            meanY_plot = mean(simY_real, 2) * yuan_to_wanyuan;
            meanTotalW_plot = mean(simW_real + simF_real, 2) * yuan_to_wanyuan;
            meanLiquidW_plot = mean(simW_real, 2) * yuan_to_wanyuan;
            meanF_plot = mean(simF_real, 2) * yuan_to_wanyuan;
            meanA_plot = nanmean(simA, 2);
            meanQ_plot = mean(simQ_real, 2) * yuan_to_wanyuan;

            % 标准差
            stdC_plot = std(simC_real, 0, 2) * yuan_to_wanyuan;
            stdY_plot = std(simY_real, 0, 2) * yuan_to_wanyuan;
            stdTotalW_plot = std(simW_real + simF_real, 0, 2) * yuan_to_wanyuan;
            stdLiquidW_plot = std(simW_real, 0, 2) * yuan_to_wanyuan;
            stdF_plot = std(simF_real, 0, 2) * yuan_to_wanyuan;
            stdA_plot = nanstd(simA, 0, 2);
            stdQ_plot = std(simQ_real, 0, 2) * yuan_to_wanyuan;

            % --- 绘图 ---
            figure('Position', [100, 100, 1400, 900]);

            % 子图 1: 生命周期剖面均值
            subplot(2,2,1);
            hold on;
            % 颜色定义
            color_total_w = [0, 0.4470, 0.7410]; % Blue
            color_liquid_w = [0.3010, 0.7450, 0.9330]; % Cyan
            color_c = [0.8500, 0.3250, 0.0980]; % Red
            color_y = [0, 0, 0]; % Black

            % 绘制置信区间 (fill)
            fill(x_fill, [meanTotalW_plot - stdTotalW_plot; flipud(meanTotalW_plot + stdTotalW_plot)], color_total_w, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanLiquidW_plot - stdLiquidW_plot; flipud(meanLiquidW_plot + stdLiquidW_plot)], color_liquid_w, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanC_plot - stdC_plot; flipud(meanC_plot + stdC_plot)], color_c, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanY_plot - stdY_plot; flipud(meanY_plot + stdY_plot)], color_y, 'FaceAlpha', 0.1, 'EdgeColor', 'none');

            % 绘制均值线
            plot(ages, meanTotalW_plot, 'Color', color_total_w, 'LineWidth', 2, 'DisplayName', '总财富 (流动+养老金)');
            plot(ages, meanLiquidW_plot, 'Color', color_liquid_w, 'LineStyle', '--', 'LineWidth', 1.5, 'DisplayName', '流动性财富');
            plot(ages, meanC_plot, 'Color', color_c, 'LineWidth', 2, 'DisplayName', '消费');
            plot(ages, meanY_plot, 'Color', color_y, 'LineStyle', ':', 'LineWidth', 2, 'DisplayName', '收入');
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('生命周期剖面 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('人民币 (万元)');
            legend('show', 'Location', 'northwest'); grid on; xlim([cS.tb, cS.td]);

            % 子图 2: 投资与养老金缴费决策
            subplot(2,2,2);
            hold on;
            % 颜色定义
            color_a = [0.4940, 0.1840, 0.5560]; % Purple
            color_q = [0.4660, 0.6740, 0.1880]; % Green

            % 左轴
            yyaxis left;
            % 置信区间
            fill(x_fill, [meanA_plot - stdA_plot; flipud(meanA_plot + stdA_plot)], color_a, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanQ_plot - stdQ_plot; flipud(meanQ_plot + stdQ_plot)], color_q, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanA_plot, 'Color', color_a, 'LineWidth', 2, 'DisplayName', '风险资产配置比例 (\alpha)');
            plot(ages(1:work_periods), meanQ_plot(1:work_periods), 'Color', color_q, 'LineWidth', 2, 'DisplayName', '个人养老金年缴费 (万元)');
            ylabel('比例 / 人民币 (万元)');
            ylim([-0.05, 1.05]);

            % 右轴
            yyaxis right;
            simQY_ratio_real = simQ_real ./ simY_real;
            simQY_ratio_real(isinf(simQY_ratio_real) | isnan(simQY_ratio_real)) = NaN;
            meanQY_ratio_plot = nanmean(simQY_ratio_real, 2);
            stdQY_ratio_plot = nanstd(simQY_ratio_real, 0, 2);
            % 置信区间
            fill(x_fill(1:2*work_periods), [meanQY_ratio_plot(1:work_periods) - stdQY_ratio_plot(1:work_periods); flipud(meanQY_ratio_plot(1:work_periods) + stdQY_ratio_plot(1:work_periods))], color_q, 'FaceAlpha', 0.1, 'EdgeColor', 'none');
            % 均值线
            plot(ages(1:work_periods), meanQY_ratio_plot(1:work_periods), 'Color', color_q, 'LineStyle', '--', 'LineWidth', 1.5, 'DisplayName', '缴费/收入比');
            ylabel('缴费收入比');

            hold off;
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            title('投资与养老金缴费决策 (均值 \pm 1\sigma)'); xlabel('年龄');
            grid on; xlim([cS.tb, cS.td]);
            legend('show', 'Location', 'best');

            % 子图 3: 个人养老金账户财富动态
            subplot(2,2,3);
            hold on;
            color_f = [0.9290, 0.6940, 0.1250]; % Orange
            % 置信区间
            fill(x_fill, [meanF_plot - stdF_plot; flipud(meanF_plot + stdF_plot)], color_f, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanF_plot, 'Color', color_f, 'LineWidth', 2, 'DisplayName', '养老金账户资产');
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('个人养老金账户财富动态 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('人民币 (万元)');
            legend('show', 'Location', 'northwest'); grid on; xlim([cS.tb, cS.td]);

            % 子图 4: 总财富-收入比均值
            subplot(2,2,4);
            hold on;
            color_ratio = [0.6350, 0.0780, 0.1840]; % Maroon
            simWY_ratio_real = (simW_real + simF_real) ./ simY_real;
            simWY_ratio_real(isinf(simWY_ratio_real) | isnan(simWY_ratio_real)) = NaN;
            meanWY_ratio_plot = nanmean(simWY_ratio_real, 2);
            stdWY_ratio_plot = nanstd(simWY_ratio_real, 0, 2);
            % 置信区间
            fill(x_fill, [meanWY_ratio_plot - stdWY_ratio_plot; flipud(meanWY_ratio_plot + stdWY_ratio_plot)], color_ratio, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanWY_ratio_plot, 'Color', color_ratio, 'LineWidth', 2);
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('总财富-收入比 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('比率');
            grid on; xlim([cS.tb, cS.td]);

            sgtitle('生命周期模拟结果 (以2024年人民币计价)', 'FontSize', 16, 'FontWeight', 'bold');

            output_filename = fullfile(cS.save_path, 'vfi_simulation_results_real_cny_with_ci.png');
            print(gcf, output_filename, '-dpng', '-r300');
            fprintf('真实单位模拟图形已保存到 %s\n', output_filename);
        end
        % =====================================================================
        %                辅助函数: 计算归一化的养老金缴费上限
        % =====================================================================
        function q_max_normalized = calculate_q_max(absolute_q_max_yuan, anchor_income_yuan, f_y_profile)
            % 描述:
            % 本函数根据现实世界的绝对缴费上限(如12000元)和代表性收入水平，
            % 计算出适用于模型归一化单位的缴费上限比率 cS.Q_max。
            %
            % 输入:
            %   absolute_q_max_yuan - 政策规定的年度个人养老金缴费绝对上限 (人民币)
            %   anchor_income_yuan  - 用于锚定模型收入水平的初始工作年收入 (人民币)
            %   f_y_profile         - 模型计算出的整个工作生涯的确定性收入剖面 (无单位)
            %
            % 输出:
            %   q_max_normalized    - 归一化后的缴费上限 (相对于平均持久性收入的比率)

            fprintf('  正在计算归一化的养老金缴费上限 (Q_max)...\n');

            % 1. 检查输入是否有效
            if isempty(f_y_profile) || f_y_profile(1) <= 0
                error('输入的收入剖面 f_y_profile 无效。');
            end

            % 2. 计算转换因子，将模型的抽象收入单位与人民币挂钩
            % 因子 = 人民币 / 模型单位
            yuan_per_model_unit = anchor_income_yuan / f_y_profile(1);
            fprintf('    锚定收入: 初始年收入 %.0f 元 对应模型单位 %.4f\n', anchor_income_yuan, f_y_profile(1));

            % 3. 计算整个工作生涯的确定性收入路径 (人民币计价)
            work_income_yuan = f_y_profile * yuan_per_model_unit;

            % 4. 计算工作生涯的平均年收入 (代表性的持久性收入水平)
            average_work_income_yuan = mean(work_income_yuan);
            fprintf('    模型隐含的工作生涯平均年收入为: %.0f 元\n', average_work_income_yuan);

            % 5. 计算归一化的 Q_max
            q_max_normalized = absolute_q_max_yuan / average_work_income_yuan;
            fprintf('    政策上限 %.0f 元 对应的归一化 Q_max 为: %.4f\n', absolute_q_max_yuan, q_max_normalized);

        end
        %  =====================================================================
        %  =====================================================================
        function [z_grid, P] = discretizeAR1_Tauchen(mew, rho, sigma, znum, Tauchen_q, ~)
            % Tauchen方法离散化AR(1)过程
            if znum == 1
                z_grid = mew / (1 - rho);
                P = 1;
                return;
            end

            zstar = mew / (1 - rho);
            sigmaz = sigma / sqrt(1 - rho^2);

            z_grid = zstar + linspace(-Tauchen_q * sigmaz, Tauchen_q * sigmaz, znum)';
            omega = z_grid(2) - z_grid(1);

            P = zeros(znum, znum);
            for i = 1:znum
                for j = 1:znum
                    if j == 1
                        P(i, j) = normcdf((z_grid(j) + omega/2 - rho * z_grid(i) - mew) / sigma);
                    elseif j == znum
                        P(i, j) = 1 - normcdf((z_grid(j) - omega/2 - rho * z_grid(i) - mew) / sigma);
                    else
                        P(i, j) = normcdf((z_grid(j) + omega/2 - rho * z_grid(i) - mew) / sigma) - ...
                            normcdf((z_grid(j) - omega/2 - rho * z_grid(i) - mew) / sigma);
                    end
                end
            end
        end


    end

end

### utils_pps_fulllliquidity: 完全流动的个人养老金Fund

In [ ]:
classdef utils_pps_fullliquidity
    methods (Static)


        %% =====================================================================
        %                       1. 参数设置函数
        %  =====================================================================
        function cS = setup_parameters(varargin)
            fprintf('===== 步骤 1: 设置模型参数 =====\n');

            cS = struct();

            % --- A. 生命周期与人口结构 ---
            cS.tb = 22;       % 初始工作年龄
            cS.tr = 61;       % 退休年龄
            cS.td = 100;      % 最高寿命
            cS.tn = cS.td - cS.tb + 1; % 总期数

            % --- B. 经济主体偏好 (Epstein-Zin) ---
            cS.beta = 0.95;     % 折现因子
            cS.gamma = 3.84;    % 相对风险规避系数
            cS.psi = 0.15;      % 跨期替代弹性

            % --- C. 收入过程 ---
            % 确定性部分 g(t) = exp(aa + b1*t + b2*t^2 + b3*t^3)
            cS.aa = -2.170042 + 2.700381;
            cS.b1 = 0.16818;
            cS.b2 = -0.0323371 / 10;
            cS.b3 = 0.0019704 / 100;
            % 随机部分
            cS.smay = sqrt(0.169993); % 暂时冲击标准差 (u_t)
            cS.smav = sqrt(0.112572); % 持久冲击标准差 (z_t)
            cS.corr_z_epsilon = 0; % 持久冲击与风险收益的相关性 (未使用)
            cS.corr_u_epsilon = 0; % 暂时冲击与风险收益的相关性 (未使用)

            % --- D. 资产与回报率 ---
            cS.rf = 1.02;     % 无风险总回报率
            cS.mu = 0.04;     % 风险资产超额回报率
            cS.sigr = 0.27;   % 风险资产回报率标准差
            cS.rp = 1.04;     % 个人养老金账户(PPS)回报率
            cS.sigrrp = 0.27; % 个人养老金账户(PPS)风险资产回报率标准差

            % --- E. 养老金与税收 ---
            cS.ret_fac = 0.6827;      % 退休后基础养老金 (替代率)
            cS.tau_y = 0.06;          % 工资税率
            cS.tau_q = 0.03;          % 个人养老金提取税率
            % cS.Q_max 将在下方通过新函数计算


            % --- F. 数值求解参数 ---
            cS.n_shocks = 5;      % 离散化的随机冲击节点数
            cS.ncash = 51;        % 现金持有量网格数
            cS.nfund = 51;        % 养老金账户网格数
            cS.maxcash = 200;     % 现金网格上限 (归一化)
            cS.mincash = 0.25;    % 现金网格下限 (归一化)
            cS.maxfund = 200;     % 养老金网格上限 (归一化)
            cS.minfund = 0.01;    % 养老金网格下限 (归一化)
            cS.save_path = 'result/'; % 结果保存路径

                        % 生存概率
            cS.survprob = zeros(cS.tn - 1, 1);
            cS.survprob_data = [0.99975, 0.99974, 0.99973, 0.99972, 0.99971, 0.99969, 0.99968, 0.99966, 0.99963, 0.99961, ...
                0.99958, 0.99954, 0.99951, 0.99947, 0.99943, 0.99938, 0.99934, 0.99928, 0.99922, 0.99916, ...
                0.99908, 0.999, 0.99891, 0.99881, 0.9987, 0.99857, 0.99843, 0.99828, 0.99812, 0.99794, ...
                0.99776, 0.99756, 0.99735, 0.99713, 0.9969, 0.99666, 0.9964, 0.99613, 0.99583, 0.99551, ...
                0.99515, 0.99476, 0.99432, 0.99383, 0.9933, 0.9927, 0.99205, 0.99133, 0.99053, 0.98961, ...
                0.98852, 0.98718, 0.98553, 0.98346, 0.98089, 0.97772, 0.97391, 0.96943, 0.96429, 0.95854, ...
                0.95221, 0.94537, 0.93805, 0.93027, 0.92202, 0.91327, 0.90393, 0.89389, 0.88304, 0.87126, ...
                0.85846, 0.84452, 0.82935, 0.81282, 0.79485, 0.77543, 0.75458, 0.7324, 0.70893, 0.68424, ...
                0.68424, 0.68424];

                        % *** 新增: 调用函数计算归一化的 Q_max ***
            % cS.Q_max = utils_pps_fullliquidity.calculate_q_max(absolute_q_max_yuan, anchor_income_yuan, cS.f_y);
            cS.Q_max =99999;

            if isempty(varargin) % 便于敏感性分析时改动参数而不进一步计算派生参数
                cS = utils_pps_fullliquidity.setup_process(cS);
            end
           
        end

        function cS = setup_process(cS)
            % --- G. 派生参数与预计算 ---
            cS.pension_rate = 1 / (cS.td - cS.tr + 1); % 退休后年金化给付比率
            % 效用函数参数
            cS.psi_1 = 1.0 - 1.0 / cS.psi;
            cS.psi_2 = 1.0 / cS.psi_1;
            cS.theta = (1.0 - cS.gamma) / (1.0 - cS.psi_1);


            for i = 1:min(length(cS.survprob_data), length(cS.survprob))
                cS.survprob(i) = cS.survprob_data(i);
            end

            % 离散化正态分布 (Tauchen方法)
            tauchenoptions.parallel=0;
            [grid, weig_matrix] = utils_pps_fullliquidity.discretizeAR1_Tauchen(0, 0, 1, cS.n_shocks, 2, tauchenoptions);
            cS.shock_grid = grid;
            cS.shock_weig = diag(weig_matrix);

            % 风险资产回报率网格
            cS.gret = cS.rf + cS.mu + cS.shock_grid * cS.sigr;
            cS.gret_rp = cS.rp + cS.shock_grid * cS.sigrrp;

            % 状态转移概率 (三维冲击)
            cS.nweig1 = zeros(cS.n_shocks, cS.n_shocks, cS.n_shocks);
            for i6 = 1:cS.n_shocks
                for i7 = 1:cS.n_shocks
                    for i8 = 1:cS.n_shocks
                        cS.nweig1(i6, i7, i8) = cS.shock_weig(i6) * cS.shock_weig(i7) * cS.shock_weig(i8);
                    end
                end
            end

            % 状态变量网格
            lgcash = linspace(log(cS.mincash), log(cS.maxcash), cS.ncash)';
            cS.gcash = exp(lgcash);
            lgfund = linspace(log(cS.minfund), log(cS.maxfund), cS.nfund)';
            cS.gfund = exp(lgfund);
            cS.gfund(1) = 0; % 确保可以从0开始

            % --- 收入过程计算 ---
            % 确定性收入剖面
            cS.f_y = zeros(cS.tr - cS.tb + 1, 1);
            for i1 = cS.tb:cS.tr
                age = i1;
                cS.f_y(age - cS.tb + 1) = exp(cS.aa + cS.b1*age + cS.b2*age^2 + cS.b3*age^3);
            end


            % 收入过程网格
            cS.yh = zeros(cS.n_shocks, cS.n_shocks); % 暂时冲击 exp(u_t)
            cS.yp = zeros(cS.n_shocks, cS.n_shocks); % 持久冲击增长 exp(z_t)
            cS.gyp = zeros(cS.n_shocks, cS.n_shocks, cS.tn - 1); % 总收入增长

            for i1 = 1:cS.n_shocks
                grid2_u = cS.shock_grid(i1) .* cS.corr_u_epsilon + cS.shock_grid .* sqrt(1 - cS.corr_u_epsilon^2);
                cS.yh(:, i1) = exp(grid2_u * cS.smay);

                grid2_z = cS.shock_grid(i1) .* cS.corr_z_epsilon + cS.shock_grid .* sqrt(1 - cS.corr_z_epsilon^2);
                cS.yp(:, i1) = exp(grid2_z * cS.smav);
            end

            % 工作期收入增长
            work_periods = cS.tr - cS.tb;
            for t = 1:work_periods
                G_t = cS.f_y(t+1) / cS.f_y(t); % 确定性增长
                cS.gyp(:,:,t) = repmat(G_t, cS.n_shocks, cS.n_shocks) .* cS.yp;
            end

            % 退休期收入增长 (无增长)
            cS.gyp(:,:,(work_periods+1):(cS.tn-1)) = 1.0;

            fprintf('参数设置完成。\n');
        end

        %
        %% =====================================================================
        %                       2. VFI 求解器函数 (修正版)
        %  =====================================================================
        function results = solve_model_vfi(cS)
            % --- 初始化 ---
            % 启动并行池
            if isempty(gcp('nocreate'))
                parpool('local');
            end

            % 初始化策略和价值函数数组
            V = zeros(cS.ncash, cS.nfund, cS.tn);
            C_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            A_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            Q_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            Q_policy_withd = zeros(cS.ncash, cS.nfund, cS.tn);

            % --- 终端期 (t=T) ---
            % Epstein-Zin效用下的终端价值 V_T = ((1-beta)*C_T^(1-1/psi))^(1/(1-1/psi))
            % 其中 C_T = W_T (即 cS.gcash), V_T = (1-beta)^psi_2 * W_T
            for i_fund = 1:cS.nfund
                V(:, i_fund, cS.tn) = cS.gcash * (1-cS.beta)^cS.psi_2;
                C_policy(:, i_fund, cS.tn) = cS.gcash;
            end
            A_policy(:, :, cS.tn) = 0.0;
            Q_policy(:, :, cS.tn) = 0.0;

            % --- 逆向归纳求解 ---
            fprintf('开始逆向归纳求解...\n');
            options = optimoptions('fmincon', 'Display', 'off', 'Algorithm', 'interior-point', ...
                'MaxFunctionEvaluations', 1000, 'MaxIterations', 400, ...
                'OptimalityTolerance', 1e-10, 'StepTolerance', 1e-10, ...
                'ConstraintTolerance', 1e-10);

            % -- 退休期 (t = T-1, ..., K) --
            fprintf('求解退休期...\n');
            retire_start_time = tic;
            for t = (cS.tn-1):-1:(cS.tr-cS.tb+1)
                fprintf('  t = %d (年龄 %d)\n', t, t + cS.tb - 1);
                V_next = squeeze(V(:,:,t+1)); % 下一期值函数

                % *** 修改: 创建 griddedInterpolant 对象 ***
                % V_next 维度 (ncash, nfund)。行对应gcash, 列对应gfund
                % griddedInterpolant({Y,X}, V)


                temp_C = zeros(cS.ncash, cS.nfund);
                temp_A = zeros(cS.ncash, cS.nfund);
                temp_V = zeros(cS.ncash, cS.nfund);

                parfor i_fund = 1:cS.nfund

                    local_C = zeros(cS.ncash, 1);
                    local_A = zeros(cS.ncash, 1);
                    local_V = zeros(cS.ncash, 1);

                    gfund_at_retirement = cS.gfund(i_fund);
                    constant_pension_payout = (1 - cS.tau_q) * gfund_at_retirement * cS.pension_rate;

                    V_next_interp_obj = griddedInterpolant(cS.gcash, V_next(:,i_fund),  'spline', 'linear');

                    for i_cash = 1:cS.ncash
                        gcash_val = cS.gcash(i_cash);

                        lb = [1e-6, 0];
                        x0 = [0.5 * gcash_val, 1-1e-06];
                        if i_cash > 1 && local_C(i_cash-1) > 0
                            x0 = [local_C(i_cash-1), local_A(i_cash-1)];
                            % lb = [local_C(i_cash-1)-1e-06, 0];
                        end

                        ub = [gcash_val, 1];

                        % *** 修改: 将插值对象传递给目标函数 ***
                        obj_fun = @(x) utils_pps_fullliquidity.fun_valuefunc_retired(x, gcash_val, constant_pension_payout, i_fund, V_next_interp_obj, cS, t);

                        [policy, fval] = fmincon(obj_fun, x0, [], [], [], [], lb, ub, [], options);

                        local_C(i_cash) = policy(1);
                        % if abs(policy(1)-gcash_val)<1e-05
                        %     local_A(i_cash) = nan; % C = Cash
                        % else
                        local_A(i_cash) = policy(2);
                        % end
                        local_V(i_cash) = -fval;
                    end
                    temp_C(:, i_fund) = local_C;
                    temp_A(:, i_fund) = local_A;
                    temp_V(:, i_fund) = local_V;
                end
                C_policy(:, :, t) = temp_C;
                A_policy(:, :, t) = temp_A;
                V(:, :, t) = temp_V;
            end
            fprintf('退休期求解完成，耗时 %.2f 分钟\n', toc(retire_start_time)/60);

            % -- 工作期 (t = K-1, ..., 1) --
            fprintf('求解工作期...\n');
            work_start_time = tic;
            for t = (cS.tr - cS.tb):-1:1
                fprintf('  t = %d (年龄 %d)\n', t, t + cS.tb - 1);
                V_next = squeeze(V(:,:,t+1));

                % *** 修改: 创建 griddedInterpolant 对象 ***
                [X,Y] = ndgrid(cS.gcash,cS.gfund);
                V_next_interp_obj = griddedInterpolant(X, Y, V_next,  'spline','linear');

                temp_C = zeros(cS.ncash, cS.nfund);
                temp_A = zeros(cS.ncash, cS.nfund);
                temp_Q = zeros(cS.ncash, cS.nfund);
                temp_Q_withd = zeros(cS.ncash, cS.nfund);
                temp_V = zeros(cS.ncash, cS.nfund);

                parfor i_fund = 1:cS.nfund
                    local_C = zeros(cS.ncash, 1);
                    local_A = zeros(cS.ncash, 1);
                    local_Q = zeros(cS.ncash, 1);
                    local_Q_withd = zeros(cS.ncash, 1);
                    local_V = zeros(cS.ncash, 1);

                    gfund_val = cS.gfund(i_fund);
                    for i_cash = 1:cS.ncash
                        gcash_val = cS.gcash(i_cash);

                        % 2. 放弃在 lb 中施加单调性硬约束，Q 的下界永远是 0
                        lb = [1e-3, 0, 0, 0];
                        % *** 核心修改 ***
                        if i_cash > 1
                            x0 = [local_C(i_cash-1), local_A(i_cash-1), local_Q(i_cash-1), local_Q_withd(i_cash-1)];
                            % lb = [local_C(i_cash-1)-1e-6, 0, local_Q(i_cash-1)-1e-6];
                        else
                            x0 = [gcash_val * 0.5, 1-1e-06, 1e-6 , 1e-6]; % C, alpha, Q
                        end

                        % 3. ub 只包含物理和政策约束
                        ub = [gcash_val, 1, min(cS.Q_max,gcash_val/(1 - cS.tau_y)), gfund_val];


                        % *** 修改: 将插值对象传递给目标函数 ***
                        obj_fun = @(x) utils_pps_fullliquidity.fun_valuefunc_work(x, gcash_val, gfund_val, V_next_interp_obj, cS, t);

                        A_ineq = [1, 0, 1 - cS.tau_y, -1];
                        b_ineq = gcash_val;

                        [policy, fval] = fmincon(obj_fun, x0, A_ineq, b_ineq, [], [], lb, ub, [], options);

                        local_C(i_cash) = policy(1);
                        % if abs(A_ineq*policy'-b_ineq)<1e-05
                        %     local_A(i_cash) = nan; % C+（1-tau_y）Q = Cash
                        % else
                        local_A(i_cash) = policy(2);
                        % end
                        local_Q(i_cash) = policy(3);
                        local_Q_withd(i_cash) = policy(4);
                        % if abs(policy(3))<0.0001
                        %     policy(3)
                        % end
                        local_V(i_cash) = -fval;
                    end
                    temp_C(:, i_fund) = local_C;
                    temp_A(:, i_fund) = local_A;
                    temp_Q(:, i_fund) = local_Q;
                    temp_Q_withd(:, i_fund) = local_Q_withd;
                    temp_V(:, i_fund) = local_V;
                end
                C_policy(:, :, t) = temp_C;
                A_policy(:, :, t) = temp_A;
                Q_policy(:, :, t) = temp_Q;
                Q_policy_withd(:, :, t) = temp_Q_withd;
                V(:, :, t) = temp_V;
            end
            fprintf('工作期求解完成，耗时 %.2f 分钟\n', toc(work_start_time)/60);

            % --- 封装结果 ---
            results.V = V;
            results.C_policy = C_policy;
            results.A_policy = A_policy;
            results.Q_policy = Q_policy;
            results.Q_policy_withd = Q_policy_withd;
            % 平滑处理
            % results.A_policy(find(abs(cS.gcash-results.C_policy-(1-cS.tau_y)*results.Q_policy)<1e-05))=nan;
        end
        
        
        %% =====================================================================
        %                VFI - 退休期值函数
        %  =====================================================================
        function v = fun_valuefunc_retired(x, cash, pension_payout, i_fund_K, V_next_interp_obj, cS, t)
            % x = [消费比例 c_rate, 风险资产比例 alpha]
            % pension_payout: 这是一个根据F_K计算出的固定值
            % i_fund_K: 这是F_K在网格上的索引，用于插值V_next，因为它在退休期间不变

            alpha = x(2);

            consumption = x(1);
            if consumption <= 1e-8, v = 1e10; return; end

            sav = cash - consumption;
            if sav < 0, v = 1e10; return; end

            % Epstein-Zin 瞬时效用项
            u_term = (1 - cS.beta) * (consumption^cS.psi_1);

            % 下一期现金持有量 (归一化)
            ret_fac_norm = cS.ret_fac;
            cash_1 = (cS.rf * (1 - alpha) + cS.gret * alpha) * sav + ret_fac_norm + pension_payout;
            % cash_1 = max(min(cash_1, cS.gcash(end)), cS.gcash(1));

            % 下一期养老金状态 F_{t+1} 就是 F_K
            fund_1 = repmat(cS.gfund(i_fund_K), cS.n_shocks, 1);

            % *** 修改: 使用 griddedInterpolant 对象进行插值 ***
            % 调用方式 F({Yq, Xq})
            int_V = V_next_interp_obj(cash_1);
            int_V(int_V <= 0) = 1e-9;

            % 计算期望
            expected_term = cS.shock_weig' * (int_V.^(1 - cS.gamma));

            % 完整的 Epstein-Zin 值函数 (退休后 gyp=1)
            value = (u_term + cS.beta * cS.survprob(t) * (expected_term)^(cS.psi_1 / (1 - cS.gamma)))^cS.psi_2;

            v = -value; % fmincon 最小化
        end

        %% =====================================================================
        %                VFI - 工作期值函数
        %  =====================================================================

        function v = fun_valuefunc_work(x, cash, fund, V_next_interp_obj, cS, t)
            % x = [绝对消费 C, 风险资产比例 alpha, 绝对养老金缴费 Q]
            consumption = x(1);
            alpha       = x(2);
            pension_contrib = x(3);
            pension_withdraw = x(4);

            if consumption <= 1e-8
                v = 1e10;
                return;
            end

            liquid_sav = cash - consumption - (1 - cS.tau_y) * pension_contrib + pension_withdraw;

            if liquid_sav < -1e-6
                v = 1e10;
                return;
            end

            % Epstein-Zin 瞬时效用项
            u_term = (1 - cS.beta) * (consumption^cS.psi_1);

            % 构造下一期状态的网格 (三维冲击)
            n = cS.n_shocks;
            gyp_3d = repmat(reshape(cS.gyp(:,:,t), [n, 1, n]), [1, n, 1]);
            gret_3d = repmat(reshape(cS.gret, [1, 1, n]), [n, n, 1]);
            yh_3d = repmat(reshape(cS.yh, [1, n, n]), [n, 1, 1]);

            % 下一期归一化状态变量
            portfolio_return = cS.rf * (1 - alpha) + gret_3d * alpha;
            cash_1 = portfolio_return .* (liquid_sav ./ gyp_3d) + (1-cS.tau_y) .* yh_3d;
            fund_1 = (fund + pension_contrib - pension_withdraw) * cS.rp ./ gyp_3d;

            % cash_1 = max(min(cash_1, cS.gcash(end)), cS.gcash(1));
            % fund_1 = max(min(fund_1, cS.gfund(end)), cS.gfund(1));

            % *** 修改: 使用 griddedInterpolant 对象进行插值 ***
            % 调用方式 F({Yq, Xq})
            int_V = V_next_interp_obj(cash_1, fund_1);
            int_V(int_V <= 0) = 1e-9;

            % 构造期望项 (包含归一化因子的增长)
            term_inside_exp = (gyp_3d .* int_V).^(1 - cS.gamma);
            expected_term = sum(cS.nweig1 .* term_inside_exp, 'all');

            % 完整的 Epstein-Zin 值函数
            value = (u_term + cS.beta * cS.survprob(t) * (expected_term)^(cS.psi_1 / (1 - cS.gamma)))^cS.psi_2;

            v = -value; % fmincon 最小化
        end

        %% =====================================================================
        %                       3. 模拟器函数
        %  =====================================================================
        function simulate_model(results, cS)
            % --- 初始化 ---
            fprintf('模拟开始...\n');
            rng(42); % 可复现性
            nsim = 10000; % 模拟个体数量

            % 解包策略函数
            C_policy = results.C_policy;
            A_policy = results.A_policy;
            Q_policy = results.Q_policy;

            % 初始化模拟数组 (所有变量首先在归一化单位下计算)
            % 冲击和外生过程
            simGPY = ones(cS.tn, nsim);       % 持久收入增长因子 G_t * exp(z_t)
            simY_norm = zeros(cS.tn, nsim);    % 归一化暂时/基础收入 exp(u_t) 或 ret_fac
            simR = zeros(cS.tn, nsim);         % 风险回报率

            % 归一化单位下的决策和状态变量
            simW_norm = zeros(cS.tn, nsim);    % 归一化流动性财富 w_t = W_t/P_t
            simF_norm = zeros(cS.tn, nsim);    % 归一化养老金 f_t = F_t/P_t
            simC_norm = zeros(cS.tn, nsim);    % 归一化消费 c_t = C_t/P_t
            simQ_norm = zeros(cS.tn, nsim);    % 归一化缴费 q_t = Q_t/P_t
            simA = zeros(cS.tn, nsim);         % 风险资产配置 alpha_t
            simS_norm = zeros(cS.tn, nsim);    % 归一化风险资产 s_t
            simB_norm = zeros(cS.tn, nsim);    % 归一化无风险资产 b_t
            simQ_Cash_ratio = zeros(cS.tn, nsim); % 缴费与现金流比率 q/cash

            % --- 1. 模拟外生冲击和收入过程 ---
            fprintf('  生成收入和回报率路径...\n');
            work_periods = cS.tr - cS.tb;

            % 使用 Antithetic Variates 方法减少模拟误差
            for i1 = 1:floor(nsim/2)
                z_shocks = randn(work_periods, 1) * cS.smav;
                u_shocks_log = randn(work_periods, 1) * cS.smay;
                r_shocks = randn(cS.tn, 1) * cS.sigr;

                % 正向冲击路径
                simGPY(2:work_periods+1, i1) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(z_shocks);
                simY_norm(1:work_periods, i1) = exp(u_shocks_log);
                simR(:, i1) = cS.rf + cS.mu + r_shocks;

                % 反向冲击路径 (镜像)
                i2 = nsim/2 + i1;
                simGPY(2:work_periods+1, i2) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(-z_shocks);
                simY_norm(1:work_periods, i2) = exp(-u_shocks_log);
                simR(:, i2) = cS.rf + cS.mu - r_shocks;
            end

            % 退休后基础养老金 (替代率)
            norm_base_pension = cS.ret_fac ;
            simY_norm(work_periods+1:cS.tn, :) = norm_base_pension;

            % --- 2. 迭代模拟生命周期决策 (在归一化单位下) ---
            fprintf('  模拟生命周期决策 (归一化单位)...\n');
            for t = 1:cS.tn
                if t <= work_periods % 工作期
                    norm_cash = simW_norm(t, :) + (1-cS.tau_y) * simY_norm(t, :);
                else % 退休期
                    norm_pension_payout = (1-cS.tau_q) * simF_norm(t, :) * cS.pension_rate;
                    norm_cash = simW_norm(t, :) + simY_norm(t, :) + norm_pension_payout;
                end

                % 为当前期 t 的所有策略函数创建插值器
                Fc_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, C_policy(:,:,t), 'spline', 'linear');
                Fa_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, A_policy(:,:,t), 'spline', 'linear');
                Fq_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, Q_policy(:,:,t), 'spline', 'linear');

                % 对所有 nsim 个体进行插值，得到决策
                simC_norm(t, :) = Fc_interpolant(norm_cash, simF_norm(t, :));
                simA(t, :) = Fa_interpolant(norm_cash, simF_norm(t, :));
                simQ_norm(t, :) = Fq_interpolant(norm_cash, simF_norm(t, :));

                % 应用约束和边界条件
                simA(t, :) = max(min(simA(t, :), 1), 0);
                simC_norm(t, :) = max(min(simC_norm(t, :), norm_cash), 0);
                simQ_norm(t, :) = max(min(simQ_norm(t, :), cS.Q_max), 0);

                if t <= work_periods % 工作期
                    total_outflow = simC_norm(t, :) + (1-cS.tau_y)*simQ_norm(t, :);
                    idx_exceed = total_outflow > norm_cash;
                    if any(idx_exceed)
                        scale_factor = norm_cash(idx_exceed) ./ total_outflow(idx_exceed) * 0.9999;
                        simC_norm(t, idx_exceed) = simC_norm(t, idx_exceed) .* scale_factor;
                        simQ_norm(t, idx_exceed) = simQ_norm(t, idx_exceed) .* scale_factor;
                    end
                    simQ_Cash_ratio(t, :) = simQ_norm(t, :) ./ norm_cash;
                    simQ_Cash_ratio(t, isnan(simQ_Cash_ratio(t,:)) | isinf(simQ_Cash_ratio(t,:))) = 0;
                    liquid_sav_norm = norm_cash - simC_norm(t, :) - (1-cS.tau_y)*simQ_norm(t, :);
                else % 退休期
                    simQ_norm(t,:) = 0;
                    simC_norm(t,:) = min(simC_norm(t,:), norm_cash * 0.9999);
                    liquid_sav_norm = norm_cash - simC_norm(t, :);
                end

                simS_norm(t, :) = simA(t, :) .* liquid_sav_norm;
                simB_norm(t, :) = liquid_sav_norm - simS_norm(t, :);

                % 更新下一期状态 (归一化单位)
                if t < cS.tn
                    portfolio_return_next = simB_norm(t, :) * cS.rf + simS_norm(t, :) .* simR(t, :);
                    simW_norm(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);

                    if t < work_periods
                        simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
                    elseif t == work_periods % 退休前最后一年
                        simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
                    else % 退休后
                        simF_norm(t+1, :) = simF_norm(t, :);
                    end
                end
            end
            fprintf('  归一化单位模拟完成。\n');

            % --- 3. 将结果转换为以人民币计价的真实数值 ---
            fprintf('  将模拟结果转换为真实人民币单位...\n');

            % **锚定参数**
            initial_income_yuan = 60000; % 假设22岁时平均年持久性收入为 60,000 元

            % 初始化真实单位数组
            simP_real = zeros(cS.tn, nsim); % 真实持久性收入 (CNY)
            simY_real = zeros(cS.tn, nsim); % 真实总收入 (CNY)
            simW_real = zeros(cS.tn, nsim); % 真实流动性财富 (CNY)
            simF_real = zeros(cS.tn, nsim); % 真实养老金财富 (CNY)
            simC_real = zeros(cS.tn, nsim); % 真实消费 (CNY)
            simQ_real = zeros(cS.tn, nsim); % 真实养老金缴费 (CNY)

            % 模拟真实持久性收入路径 P_t
            simP_real(1, :) = initial_income_yuan; % 锚定初始值
            for t = 1:(cS.tn - 1)
                simP_real(t+1, :) = simP_real(t, :) .* simGPY(t+1, :);
            end

            % "去归一化" 所有相关变量
            for t = 1:cS.tn
                simW_real(t, :) = simW_norm(t, :) .* simP_real(t, :);
                simF_real(t, :) = simF_norm(t, :) .* simP_real(t, :);
                simC_real(t, :) = simC_norm(t, :) .* simP_real(t, :);
                simQ_real(t, :) = simQ_norm(t, :) .* simP_real(t, :);
                % 计算真实总收入
                if t <= work_periods
                    simY_real(t, :) = simP_real(t, :) .* simY_norm(t, :);
                else
                    simY_real(t, :) = simP_real(t, :);
                end
            end

            % --- 4. 汇总真实结果并绘图 (单位: 万元) ---
            fprintf('  汇总真实结果并绘图 (单位: 万元)...\n');
            yuan_to_wanyuan = 1 / 10000;
            ages = (cS.tb:cS.td)';
            x_fill = [ages; flipud(ages)]; % 用于填充置信区间的x坐标

            % --- 数据处理 ---
            % 均值
            meanC_plot = mean(simC_real, 2) * yuan_to_wanyuan;
            meanY_plot = mean(simY_real, 2) * yuan_to_wanyuan;
            meanTotalW_plot = mean(simW_real + simF_real, 2) * yuan_to_wanyuan;
            meanLiquidW_plot = mean(simW_real, 2) * yuan_to_wanyuan;
            meanF_plot = mean(simF_real, 2) * yuan_to_wanyuan;
            meanA_plot = nanmean(simA, 2);
            meanQ_plot = mean(simQ_real, 2) * yuan_to_wanyuan;

            % 标准差
            stdC_plot = std(simC_real, 0, 2) * yuan_to_wanyuan;
            stdY_plot = std(simY_real, 0, 2) * yuan_to_wanyuan;
            stdTotalW_plot = std(simW_real + simF_real, 0, 2) * yuan_to_wanyuan;
            stdLiquidW_plot = std(simW_real, 0, 2) * yuan_to_wanyuan;
            stdF_plot = std(simF_real, 0, 2) * yuan_to_wanyuan;
            stdA_plot = nanstd(simA, 0, 2);
            stdQ_plot = std(simQ_real, 0, 2) * yuan_to_wanyuan;

            % --- 绘图 ---
            figure('Position', [100, 100, 1400, 900]);

            % 子图 1: 生命周期剖面均值
            subplot(2,2,1);
            hold on;
            % 颜色定义
            color_total_w = [0, 0.4470, 0.7410]; % Blue
            color_liquid_w = [0.3010, 0.7450, 0.9330]; % Cyan
            color_c = [0.8500, 0.3250, 0.0980]; % Red
            color_y = [0, 0, 0]; % Black

            % 绘制置信区间 (fill)
            fill(x_fill, [meanTotalW_plot - stdTotalW_plot; flipud(meanTotalW_plot + stdTotalW_plot)], color_total_w, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanLiquidW_plot - stdLiquidW_plot; flipud(meanLiquidW_plot + stdLiquidW_plot)], color_liquid_w, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanC_plot - stdC_plot; flipud(meanC_plot + stdC_plot)], color_c, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanY_plot - stdY_plot; flipud(meanY_plot + stdY_plot)], color_y, 'FaceAlpha', 0.1, 'EdgeColor', 'none');

            % 绘制均值线
            plot(ages, meanTotalW_plot, 'Color', color_total_w, 'LineWidth', 2, 'DisplayName', '总财富 (流动+养老金)');
            plot(ages, meanLiquidW_plot, 'Color', color_liquid_w, 'LineStyle', '--', 'LineWidth', 1.5, 'DisplayName', '流动性财富');
            plot(ages, meanC_plot, 'Color', color_c, 'LineWidth', 2, 'DisplayName', '消费');
            plot(ages, meanY_plot, 'Color', color_y, 'LineStyle', ':', 'LineWidth', 2, 'DisplayName', '收入');
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('生命周期剖面 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('人民币 (万元)');
            legend('show', 'Location', 'northwest'); grid on; xlim([cS.tb, cS.td]);

            % 子图 2: 投资与养老金缴费决策
            subplot(2,2,2);
            hold on;
            % 颜色定义
            color_a = [0.4940, 0.1840, 0.5560]; % Purple
            color_q = [0.4660, 0.6740, 0.1880]; % Green

            % 左轴
            yyaxis left;
            % 置信区间
            fill(x_fill, [meanA_plot - stdA_plot; flipud(meanA_plot + stdA_plot)], color_a, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanQ_plot - stdQ_plot; flipud(meanQ_plot + stdQ_plot)], color_q, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanA_plot, 'Color', color_a, 'LineWidth', 2, 'DisplayName', '风险资产配置比例 (\alpha)');
            plot(ages(1:work_periods), meanQ_plot(1:work_periods), 'Color', color_q, 'LineWidth', 2, 'DisplayName', '个人养老金年缴费 (万元)');
            ylabel('比例 / 人民币 (万元)');
            ylim([-0.05, 1.05]);

            % 右轴
            yyaxis right;
            simQY_ratio_real = simQ_real ./ simY_real;
            simQY_ratio_real(isinf(simQY_ratio_real) | isnan(simQY_ratio_real)) = NaN;
            meanQY_ratio_plot = nanmean(simQY_ratio_real, 2);
            stdQY_ratio_plot = nanstd(simQY_ratio_real, 0, 2);
            % 置信区间
            fill(x_fill(1:2*work_periods), [meanQY_ratio_plot(1:work_periods) - stdQY_ratio_plot(1:work_periods); flipud(meanQY_ratio_plot(1:work_periods) + stdQY_ratio_plot(1:work_periods))], color_q, 'FaceAlpha', 0.1, 'EdgeColor', 'none');
            % 均值线
            plot(ages(1:work_periods), meanQY_ratio_plot(1:work_periods), 'Color', color_q, 'LineStyle', '--', 'LineWidth', 1.5, 'DisplayName', '缴费/收入比');
            ylabel('缴费收入比');

            hold off;
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            title('投资与养老金缴费决策 (均值 \pm 1\sigma)'); xlabel('年龄');
            grid on; xlim([cS.tb, cS.td]);
            legend('show', 'Location', 'best');

            % 子图 3: 个人养老金账户财富动态
            subplot(2,2,3);
            hold on;
            color_f = [0.9290, 0.6940, 0.1250]; % Orange
            % 置信区间
            fill(x_fill, [meanF_plot - stdF_plot; flipud(meanF_plot + stdF_plot)], color_f, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanF_plot, 'Color', color_f, 'LineWidth', 2, 'DisplayName', '养老金账户资产');
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('个人养老金账户财富动态 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('人民币 (万元)');
            legend('show', 'Location', 'northwest'); grid on; xlim([cS.tb, cS.td]);

            % 子图 4: 总财富-收入比均值
            subplot(2,2,4);
            hold on;
            color_ratio = [0.6350, 0.0780, 0.1840]; % Maroon
            simWY_ratio_real = (simW_real + simF_real) ./ simY_real;
            simWY_ratio_real(isinf(simWY_ratio_real) | isnan(simWY_ratio_real)) = NaN;
            meanWY_ratio_plot = nanmean(simWY_ratio_real, 2);
            stdWY_ratio_plot = nanstd(simWY_ratio_real, 0, 2);
            % 置信区间
            fill(x_fill, [meanWY_ratio_plot - stdWY_ratio_plot; flipud(meanWY_ratio_plot + stdWY_ratio_plot)], color_ratio, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanWY_ratio_plot, 'Color', color_ratio, 'LineWidth', 2);
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('总财富-收入比 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('比率');
            grid on; xlim([cS.tb, cS.td]);

            sgtitle('生命周期模拟结果 (以2024年人民币计价)', 'FontSize', 16, 'FontWeight', 'bold');

            output_filename = fullfile(cS.save_path, 'vfi_simulation_results_real_cny_with_ci.png');
            print(gcf, output_filename, '-dpng', '-r300');
            fprintf('真实单位模拟图形已保存到 %s\n', output_filename);
        end
        % =====================================================================
        %                辅助函数: 计算归一化的养老金缴费上限
        % =====================================================================
        function q_max_normalized = calculate_q_max(absolute_q_max_yuan, anchor_income_yuan, f_y_profile)
            % 描述:
            % 本函数根据现实世界的绝对缴费上限(如12000元)和代表性收入水平，
            % 计算出适用于模型归一化单位的缴费上限比率 cS.Q_max。
            %
            % 输入:
            %   absolute_q_max_yuan - 政策规定的年度个人养老金缴费绝对上限 (人民币)
            %   anchor_income_yuan  - 用于锚定模型收入水平的初始工作年收入 (人民币)
            %   f_y_profile         - 模型计算出的整个工作生涯的确定性收入剖面 (无单位)
            %
            % 输出:
            %   q_max_normalized    - 归一化后的缴费上限 (相对于平均持久性收入的比率)

            fprintf('  正在计算归一化的养老金缴费上限 (Q_max)...\n');

            % 1. 检查输入是否有效
            if isempty(f_y_profile) || f_y_profile(1) <= 0
                error('输入的收入剖面 f_y_profile 无效。');
            end

            % 2. 计算转换因子，将模型的抽象收入单位与人民币挂钩
            % 因子 = 人民币 / 模型单位
            yuan_per_model_unit = anchor_income_yuan / f_y_profile(1);
            fprintf('    锚定收入: 初始年收入 %.0f 元 对应模型单位 %.4f\n', anchor_income_yuan, f_y_profile(1));

            % 3. 计算整个工作生涯的确定性收入路径 (人民币计价)
            work_income_yuan = f_y_profile * yuan_per_model_unit;

            % 4. 计算工作生涯的平均年收入 (代表性的持久性收入水平)
            average_work_income_yuan = mean(work_income_yuan);
            fprintf('    模型隐含的工作生涯平均年收入为: %.0f 元\n', average_work_income_yuan);

            % 5. 计算归一化的 Q_max
            q_max_normalized = absolute_q_max_yuan / average_work_income_yuan;
            fprintf('    政策上限 %.0f 元 对应的归一化 Q_max 为: %.4f\n', absolute_q_max_yuan, q_max_normalized);

        end
        %  =====================================================================
        %  =====================================================================
        function [z_grid, P] = discretizeAR1_Tauchen(mew, rho, sigma, znum, Tauchen_q, ~)
            % Tauchen方法离散化AR(1)过程
            if znum == 1
                z_grid = mew / (1 - rho);
                P = 1;
                return;
            end

            zstar = mew / (1 - rho);
            sigmaz = sigma / sqrt(1 - rho^2);

            z_grid = zstar + linspace(-Tauchen_q * sigmaz, Tauchen_q * sigmaz, znum)';
            omega = z_grid(2) - z_grid(1);

            P = zeros(znum, znum);
            for i = 1:znum
                for j = 1:znum
                    if j == 1
                        P(i, j) = normcdf((z_grid(j) + omega/2 - rho * z_grid(i) - mew) / sigma);
                    elseif j == znum
                        P(i, j) = 1 - normcdf((z_grid(j) - omega/2 - rho * z_grid(i) - mew) / sigma);
                    else
                        P(i, j) = normcdf((z_grid(j) + omega/2 - rho * z_grid(i) - mew) / sigma) - ...
                            normcdf((z_grid(j) - omega/2 - rho * z_grid(i) - mew) / sigma);
                    end
                end
            end
        end


    end

end

### utils_pps_notax_incentive: 无税收激励的PPS(TTE)

In [ ]:
classdef utils_pps_notax_incentive
    methods (Static)


        %% =====================================================================
        %                       1. 参数设置函数
        %  =====================================================================
        function cS = setup_parameters(varargin)
            fprintf('===== 步骤 1: 设置模型参数 =====\n');

            cS = struct();

            % --- A. 生命周期与人口结构 ---
            cS.tb = 22;       % 初始工作年龄
            cS.tr = 61;       % 退休年龄
            cS.td = 100;      % 最高寿命
            cS.tn = cS.td - cS.tb + 1; % 总期数

            % --- B. 经济主体偏好 (Epstein-Zin) ---
            cS.beta = 0.95;     % 折现因子
            cS.gamma = 3.84;    % 相对风险规避系数
            cS.psi = 0.15;      % 跨期替代弹性

            % --- C. 收入过程 ---
            % 确定性部分 g(t) = exp(aa + b1*t + b2*t^2 + b3*t^3)
            cS.aa = -2.170042 + 2.700381;
            cS.b1 = 0.16818;
            cS.b2 = -0.0323371 / 10;
            cS.b3 = 0.0019704 / 100;
            % 随机部分
            cS.smay = sqrt(0.169993); % 暂时冲击标准差 (u_t)
            cS.smav = sqrt(0.112572); % 持久冲击标准差 (z_t)
            cS.corr_z_epsilon = 0; % 持久冲击与风险收益的相关性 (未使用)
            cS.corr_u_epsilon = 0; % 暂时冲击与风险收益的相关性 (未使用)

            % --- D. 资产与回报率 ---
            cS.rf = 1.02;     % 无风险总回报率
            cS.mu = 0.04;     % 风险资产超额回报率
            cS.sigr = 0.27;   % 风险资产回报率标准差
            cS.rp = 1.04;     % 个人养老金账户(PPS)回报率
            cS.sigrrp = 0.27; % 个人养老金账户(PPS)风险资产回报率标准差

            % --- E. 养老金与税收 ---
            cS.ret_fac = 0.6827;      % 退休后基础养老金 (替代率)
            cS.tau_y = 0.06;          % 工资税率
            % cS.Q_max 将在下方通过新函数计算


            % --- F. 数值求解参数 ---
            cS.n_shocks = 5;      % 离散化的随机冲击节点数
            cS.ncash = 51;        % 现金持有量网格数
            cS.nfund = 51;        % 养老金账户网格数
            cS.maxcash = 200;     % 现金网格上限 (归一化)
            cS.mincash = 0.25;    % 现金网格下限 (归一化)
            cS.maxfund = 200;     % 养老金网格上限 (归一化)
            cS.minfund = 0.01;    % 养老金网格下限 (归一化)
            cS.save_path = 'result/'; % 结果保存路径

                        % 生存概率
            cS.survprob = zeros(cS.tn - 1, 1);
            cS.survprob_data = [0.99975, 0.99974, 0.99973, 0.99972, 0.99971, 0.99969, 0.99968, 0.99966, 0.99963, 0.99961, ...
                0.99958, 0.99954, 0.99951, 0.99947, 0.99943, 0.99938, 0.99934, 0.99928, 0.99922, 0.99916, ...
                0.99908, 0.999, 0.99891, 0.99881, 0.9987, 0.99857, 0.99843, 0.99828, 0.99812, 0.99794, ...
                0.99776, 0.99756, 0.99735, 0.99713, 0.9969, 0.99666, 0.9964, 0.99613, 0.99583, 0.99551, ...
                0.99515, 0.99476, 0.99432, 0.99383, 0.9933, 0.9927, 0.99205, 0.99133, 0.99053, 0.98961, ...
                0.98852, 0.98718, 0.98553, 0.98346, 0.98089, 0.97772, 0.97391, 0.96943, 0.96429, 0.95854, ...
                0.95221, 0.94537, 0.93805, 0.93027, 0.92202, 0.91327, 0.90393, 0.89389, 0.88304, 0.87126, ...
                0.85846, 0.84452, 0.82935, 0.81282, 0.79485, 0.77543, 0.75458, 0.7324, 0.70893, 0.68424, ...
                0.68424, 0.68424];

                        % *** 新增: 调用函数计算归一化的 Q_max ***
            % cS.Q_max = utils_pps_notax_incentive.calculate_q_max(absolute_q_max_yuan, anchor_income_yuan, cS.f_y);
            cS.Q_max =99999;

            if isempty(varargin) % 便于敏感性分析时改动参数而不进一步计算派生参数
                cS = utils_pps_notax_incentive.setup_process(cS);
            end
           
        end

        function cS = setup_process(cS)
            % --- G. 派生参数与预计算 ---
            cS.pension_rate = 1 / (cS.td - cS.tr + 1); % 退休后年金化给付比率
            % 效用函数参数
            cS.psi_1 = 1.0 - 1.0 / cS.psi;
            cS.psi_2 = 1.0 / cS.psi_1;
            cS.theta = (1.0 - cS.gamma) / (1.0 - cS.psi_1);


            for i = 1:min(length(cS.survprob_data), length(cS.survprob))
                cS.survprob(i) = cS.survprob_data(i);
            end

            % 离散化正态分布 (Tauchen方法)
            tauchenoptions.parallel=0;
            [grid, weig_matrix] = utils_pps_notax_incentive.discretizeAR1_Tauchen(0, 0, 1, cS.n_shocks, 2, tauchenoptions);
            cS.shock_grid = grid;
            cS.shock_weig = diag(weig_matrix);

            % 风险资产回报率网格
            cS.gret = cS.rf + cS.mu + cS.shock_grid * cS.sigr;
            cS.gret_rp = cS.rp + cS.shock_grid * cS.sigrrp;

            % 状态转移概率 (三维冲击)
            cS.nweig1 = zeros(cS.n_shocks, cS.n_shocks, cS.n_shocks);
            for i6 = 1:cS.n_shocks
                for i7 = 1:cS.n_shocks
                    for i8 = 1:cS.n_shocks
                        cS.nweig1(i6, i7, i8) = cS.shock_weig(i6) * cS.shock_weig(i7) * cS.shock_weig(i8);
                    end
                end
            end

            % 状态变量网格
            lgcash = linspace(log(cS.mincash), log(cS.maxcash), cS.ncash)';
            cS.gcash = exp(lgcash);
            lgfund = linspace(log(cS.minfund), log(cS.maxfund), cS.nfund)';
            cS.gfund = exp(lgfund);
            cS.gfund(1) = 0; % 确保可以从0开始

            % --- 收入过程计算 ---
            % 确定性收入剖面
            cS.f_y = zeros(cS.tr - cS.tb + 1, 1);
            for i1 = cS.tb:cS.tr
                age = i1;
                cS.f_y(age - cS.tb + 1) = exp(cS.aa + cS.b1*age + cS.b2*age^2 + cS.b3*age^3);
            end


            % 收入过程网格
            cS.yh = zeros(cS.n_shocks, cS.n_shocks); % 暂时冲击 exp(u_t)
            cS.yp = zeros(cS.n_shocks, cS.n_shocks); % 持久冲击增长 exp(z_t)
            cS.gyp = zeros(cS.n_shocks, cS.n_shocks, cS.tn - 1); % 总收入增长

            for i1 = 1:cS.n_shocks
                grid2_u = cS.shock_grid(i1) .* cS.corr_u_epsilon + cS.shock_grid .* sqrt(1 - cS.corr_u_epsilon^2);
                cS.yh(:, i1) = exp(grid2_u * cS.smay);

                grid2_z = cS.shock_grid(i1) .* cS.corr_z_epsilon + cS.shock_grid .* sqrt(1 - cS.corr_z_epsilon^2);
                cS.yp(:, i1) = exp(grid2_z * cS.smav);
            end

            % 工作期收入增长
            work_periods = cS.tr - cS.tb;
            for t = 1:work_periods
                G_t = cS.f_y(t+1) / cS.f_y(t); % 确定性增长
                cS.gyp(:,:,t) = repmat(G_t, cS.n_shocks, cS.n_shocks) .* cS.yp;
            end

            % 退休期收入增长 (无增长)
            cS.gyp(:,:,(work_periods+1):(cS.tn-1)) = 1.0;

            fprintf('参数设置完成。\n');
        end

        %
        %% =====================================================================
        %                       2. VFI 求解器函数 (修正版)
        %  =====================================================================
        function results = solve_model_vfi(cS)
            % --- 初始化 ---
            % 启动并行池
            if isempty(gcp('nocreate'))
                parpool('local');
            end

            % 初始化策略和价值函数数组
            V = zeros(cS.ncash, cS.nfund, cS.tn);
            C_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            A_policy = zeros(cS.ncash, cS.nfund, cS.tn);
            Q_policy = zeros(cS.ncash, cS.nfund, cS.tn);

            % --- 终端期 (t=T) ---
            % Epstein-Zin效用下的终端价值 V_T = ((1-beta)*C_T^(1-1/psi))^(1/(1-1/psi))
            % 其中 C_T = W_T (即 cS.gcash), V_T = (1-beta)^psi_2 * W_T
            for i_fund = 1:cS.nfund
                V(:, i_fund, cS.tn) = cS.gcash * (1-cS.beta)^cS.psi_2;
                C_policy(:, i_fund, cS.tn) = cS.gcash;
            end
            A_policy(:, :, cS.tn) = 0.0;
            Q_policy(:, :, cS.tn) = 0.0;

            % --- 逆向归纳求解 ---
            fprintf('开始逆向归纳求解...\n');
            options = optimoptions('fmincon', 'Display', 'off', 'Algorithm', 'interior-point', ...
                'MaxFunctionEvaluations', 1000, 'MaxIterations', 400, ...
                'OptimalityTolerance', 1e-10, 'StepTolerance', 1e-10, ...
                'ConstraintTolerance', 1e-10);

            % -- 退休期 (t = T-1, ..., K) --
            fprintf('求解退休期...\n');
            retire_start_time = tic;
            for t = (cS.tn-1):-1:(cS.tr-cS.tb+1)
                fprintf('  t = %d (年龄 %d)\n', t, t + cS.tb - 1);
                V_next = squeeze(V(:,:,t+1)); % 下一期值函数

                % *** 修改: 创建 griddedInterpolant 对象 ***
                % V_next 维度 (ncash, nfund)。行对应gcash, 列对应gfund
                % griddedInterpolant({Y,X}, V)


                temp_C = zeros(cS.ncash, cS.nfund);
                temp_A = zeros(cS.ncash, cS.nfund);
                temp_V = zeros(cS.ncash, cS.nfund);

                parfor i_fund = 1:cS.nfund

                    local_C = zeros(cS.ncash, 1);
                    local_A = zeros(cS.ncash, 1);
                    local_V = zeros(cS.ncash, 1);

                    gfund_at_retirement = cS.gfund(i_fund);
                    constant_pension_payout = gfund_at_retirement * cS.pension_rate;

                    V_next_interp_obj = griddedInterpolant(cS.gcash, V_next(:,i_fund),  'spline', 'linear');

                    for i_cash = 1:cS.ncash
                        gcash_val = cS.gcash(i_cash);

                        lb = [1e-6, 0];
                        x0 = [0.5 * gcash_val, 1-1e-06];
                        if i_cash > 1 && local_C(i_cash-1) > 0
                            x0 = [local_C(i_cash-1), local_A(i_cash-1)];
                            % lb = [local_C(i_cash-1)-1e-06, 0];
                        end

                        ub = [gcash_val, 1];

                        % *** 修改: 将插值对象传递给目标函数 ***
                        obj_fun = @(x) utils_pps_notax_incentive.fun_valuefunc_retired(x, gcash_val, constant_pension_payout, i_fund, V_next_interp_obj, cS, t);

                        [policy, fval] = fmincon(obj_fun, x0, [], [], [], [], lb, ub, [], options);

                        local_C(i_cash) = policy(1);
                        % if abs(policy(1)-gcash_val)<1e-05
                        %     local_A(i_cash) = nan; % C = Cash
                        % else
                        local_A(i_cash) = policy(2);
                        % end
                        local_V(i_cash) = -fval;
                    end
                    temp_C(:, i_fund) = local_C;
                    temp_A(:, i_fund) = local_A;
                    temp_V(:, i_fund) = local_V;
                end
                C_policy(:, :, t) = temp_C;
                A_policy(:, :, t) = temp_A;
                V(:, :, t) = temp_V;
            end
            fprintf('退休期求解完成，耗时 %.2f 分钟\n', toc(retire_start_time)/60);

            % -- 工作期 (t = K-1, ..., 1) --
            fprintf('求解工作期...\n');
            work_start_time = tic;
            for t = (cS.tr - cS.tb):-1:1
                fprintf('  t = %d (年龄 %d)\n', t, t + cS.tb - 1);
                V_next = squeeze(V(:,:,t+1));

                % *** 修改: 创建 griddedInterpolant 对象 ***
                [X,Y] = ndgrid(cS.gcash,cS.gfund);
                V_next_interp_obj = griddedInterpolant(X, Y, V_next,  'spline','linear');

                temp_C = zeros(cS.ncash, cS.nfund);
                temp_A = zeros(cS.ncash, cS.nfund);
                temp_Q = zeros(cS.ncash, cS.nfund);
                temp_V = zeros(cS.ncash, cS.nfund);

                parfor i_fund = 1:cS.nfund
                    local_C = zeros(cS.ncash, 1);
                    local_A = zeros(cS.ncash, 1);
                    local_Q = zeros(cS.ncash, 1);
                    local_V = zeros(cS.ncash, 1);

                    gfund_val = cS.gfund(i_fund);
                    for i_cash = 1:cS.ncash
                        gcash_val = cS.gcash(i_cash);

                        % 2. 放弃在 lb 中施加单调性硬约束，Q 的下界永远是 0
                        lb = [1e-3, 0, 0];
                        % *** 核心修改 ***
                        if i_cash > 1
                            x0 = [local_C(i_cash-1), local_A(i_cash-1), local_Q(i_cash-1)];
                            % lb = [local_C(i_cash-1)-1e-6, 0, local_Q(i_cash-1)-1e-6];
                        else
                            x0 = [gcash_val * 0.5, 1-1e-06, 1e-6]; % C, alpha, Q
                        end

                        % 3. ub 只包含物理和政策约束
                        ub = [gcash_val, 1, min(cS.Q_max, gcash_val)];


                        % *** 修改: 将插值对象传递给目标函数 ***
                        obj_fun = @(x) utils_pps_notax_incentive.fun_valuefunc_work(x, gcash_val, gfund_val, V_next_interp_obj, cS, t);

                        A_ineq = [1, 0, 1];
                        b_ineq = gcash_val;

                        [policy, fval] = fmincon(obj_fun, x0, A_ineq, b_ineq, [], [], lb, ub, [], options);

                        local_C(i_cash) = policy(1);
                        % if abs(A_ineq*policy'-b_ineq)<1e-05
                        %     local_A(i_cash) = nan; % C+（1-tau_y）Q = Cash
                        % else
                        local_A(i_cash) = policy(2);
                        % end
                        local_Q(i_cash) = policy(3);
                        % if abs(policy(3))<0.0001
                        %     policy(3)
                        % end
                        local_V(i_cash) = -fval;
                    end
                    temp_C(:, i_fund) = local_C;
                    temp_A(:, i_fund) = local_A;
                    temp_Q(:, i_fund) = local_Q;
                    temp_V(:, i_fund) = local_V;
                end
                C_policy(:, :, t) = temp_C;
                A_policy(:, :, t) = temp_A;
                Q_policy(:, :, t) = temp_Q;
                V(:, :, t) = temp_V;
            end
            fprintf('工作期求解完成，耗时 %.2f 分钟\n', toc(work_start_time)/60);

            % --- 封装结果 ---
            results.V = V;
            results.C_policy = C_policy;
            results.A_policy = A_policy;
            results.Q_policy = Q_policy;
            % 平滑处理
            % results.A_policy(find(abs(cS.gcash-results.C_policy-(1-cS.tau_y)*results.Q_policy)<1e-05))=nan;
        end
        
        
        %% =====================================================================
        %                VFI - 退休期值函数
        %  =====================================================================
        function v = fun_valuefunc_retired(x, cash, pension_payout, i_fund_K, V_next_interp_obj, cS, t)
            % x = [消费比例 c_rate, 风险资产比例 alpha]
            % pension_payout: 这是一个根据F_K计算出的固定值
            % i_fund_K: 这是F_K在网格上的索引，用于插值V_next，因为它在退休期间不变

            alpha = x(2);

            consumption = x(1);
            if consumption <= 1e-8, v = 1e10; return; end

            sav = cash - consumption;
            if sav < 0, v = 1e10; return; end

            % Epstein-Zin 瞬时效用项
            u_term = (1 - cS.beta) * (consumption^cS.psi_1);

            % 下一期现金持有量 (归一化)
            ret_fac_norm = cS.ret_fac;
            cash_1 = (cS.rf * (1 - alpha) + cS.gret * alpha) * sav + ret_fac_norm + pension_payout;
            % cash_1 = max(min(cash_1, cS.gcash(end)), cS.gcash(1));

            % 下一期养老金状态 F_{t+1} 就是 F_K
            fund_1 = repmat(cS.gfund(i_fund_K), cS.n_shocks, 1);

            % *** 修改: 使用 griddedInterpolant 对象进行插值 ***
            % 调用方式 F({Yq, Xq})
            int_V = V_next_interp_obj(cash_1);
            int_V(int_V <= 0) = 1e-9;

            % 计算期望
            expected_term = cS.shock_weig' * (int_V.^(1 - cS.gamma));

            % 完整的 Epstein-Zin 值函数 (退休后 gyp=1)
            value = (u_term + cS.beta * cS.survprob(t) * (expected_term)^(cS.psi_1 / (1 - cS.gamma)))^cS.psi_2;

            v = -value; % fmincon 最小化
        end

        %% =====================================================================
        %                VFI - 工作期值函数
        %  =====================================================================

        function v = fun_valuefunc_work(x, cash, fund, V_next_interp_obj, cS, t)
            % x = [绝对消费 C, 风险资产比例 alpha, 绝对养老金缴费 Q]
            consumption = x(1);
            alpha       = x(2);
            pension_contrib = x(3);

            if consumption <= 1e-8
                v = 1e10;
                return;
            end

            liquid_sav = cash - consumption - pension_contrib;

            if liquid_sav < -1e-6
                v = 1e10;
                return;
            end

            % Epstein-Zin 瞬时效用项
            u_term = (1 - cS.beta) * (consumption^cS.psi_1);

            % 构造下一期状态的网格 (三维冲击)
            n = cS.n_shocks;
            gyp_3d = repmat(reshape(cS.gyp(:,:,t), [n, 1, n]), [1, n, 1]);
            gret_3d = repmat(reshape(cS.gret, [1, 1, n]), [n, n, 1]);
            yh_3d = repmat(reshape(cS.yh, [1, n, n]), [n, 1, 1]);

            % 下一期归一化状态变量
            portfolio_return = cS.rf * (1 - alpha) + gret_3d * alpha;
            cash_1 = portfolio_return .* (liquid_sav ./ gyp_3d) + (1-cS.tau_y) .* yh_3d;
            fund_1 = (fund + pension_contrib) * cS.rp ./ gyp_3d;

            % cash_1 = max(min(cash_1, cS.gcash(end)), cS.gcash(1));
            % fund_1 = max(min(fund_1, cS.gfund(end)), cS.gfund(1));

            % *** 修改: 使用 griddedInterpolant 对象进行插值 ***
            % 调用方式 F({Yq, Xq})
            int_V = V_next_interp_obj(cash_1, fund_1);
            int_V(int_V <= 0) = 1e-9;

            % 构造期望项 (包含归一化因子的增长)
            term_inside_exp = (gyp_3d .* int_V).^(1 - cS.gamma);
            expected_term = sum(cS.nweig1 .* term_inside_exp, 'all');

            % 完整的 Epstein-Zin 值函数
            value = (u_term + cS.beta * cS.survprob(t) * (expected_term)^(cS.psi_1 / (1 - cS.gamma)))^cS.psi_2;

            v = -value; % fmincon 最小化
        end

        %% =====================================================================
        %                       3. 模拟器函数
        %  =====================================================================
        function simulate_model(results, cS)
            % --- 初始化 ---
            fprintf('模拟开始...\n');
            rng(42); % 可复现性
            nsim = 10000; % 模拟个体数量

            % 解包策略函数
            C_policy = results.C_policy;
            A_policy = results.A_policy;
            Q_policy = results.Q_policy;

            % 初始化模拟数组 (所有变量首先在归一化单位下计算)
            % 冲击和外生过程
            simGPY = ones(cS.tn, nsim);       % 持久收入增长因子 G_t * exp(z_t)
            simY_norm = zeros(cS.tn, nsim);    % 归一化暂时/基础收入 exp(u_t) 或 ret_fac
            simR = zeros(cS.tn, nsim);         % 风险回报率

            % 归一化单位下的决策和状态变量
            simW_norm = zeros(cS.tn, nsim);    % 归一化流动性财富 w_t = W_t/P_t
            simF_norm = zeros(cS.tn, nsim);    % 归一化养老金 f_t = F_t/P_t
            simC_norm = zeros(cS.tn, nsim);    % 归一化消费 c_t = C_t/P_t
            simQ_norm = zeros(cS.tn, nsim);    % 归一化缴费 q_t = Q_t/P_t
            simA = zeros(cS.tn, nsim);         % 风险资产配置 alpha_t
            simS_norm = zeros(cS.tn, nsim);    % 归一化风险资产 s_t
            simB_norm = zeros(cS.tn, nsim);    % 归一化无风险资产 b_t
            simQ_Cash_ratio = zeros(cS.tn, nsim); % 缴费与现金流比率 q/cash

            % --- 1. 模拟外生冲击和收入过程 ---
            fprintf('  生成收入和回报率路径...\n');
            work_periods = cS.tr - cS.tb;

            % 使用 Antithetic Variates 方法减少模拟误差
            for i1 = 1:floor(nsim/2)
                z_shocks = randn(work_periods, 1) * cS.smav;
                u_shocks_log = randn(work_periods, 1) * cS.smay;
                r_shocks = randn(cS.tn, 1) * cS.sigr;

                % 正向冲击路径
                simGPY(2:work_periods+1, i1) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(z_shocks);
                simY_norm(1:work_periods, i1) = exp(u_shocks_log);
                simR(:, i1) = cS.rf + cS.mu + r_shocks;

                % 反向冲击路径 (镜像)
                i2 = nsim/2 + i1;
                simGPY(2:work_periods+1, i2) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(-z_shocks);
                simY_norm(1:work_periods, i2) = exp(-u_shocks_log);
                simR(:, i2) = cS.rf + cS.mu - r_shocks;
            end

            % 退休后基础养老金 (替代率)
            norm_base_pension = cS.ret_fac ;
            simY_norm(work_periods+1:cS.tn, :) = norm_base_pension;

            % --- 2. 迭代模拟生命周期决策 (在归一化单位下) ---
            fprintf('  模拟生命周期决策 (归一化单位)...\n');
            for t = 1:cS.tn
                if t <= work_periods % 工作期
                    norm_cash = simW_norm(t, :) + (1-cS.tau_y) * simY_norm(t, :);
                else % 退休期
                    norm_pension_payout =  simF_norm(t, :) * cS.pension_rate;
                    norm_cash = simW_norm(t, :) + simY_norm(t, :) + norm_pension_payout;
                end

                % 为当前期 t 的所有策略函数创建插值器
                Fc_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, C_policy(:,:,t), 'spline', 'linear');
                Fa_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, A_policy(:,:,t), 'spline', 'linear');
                Fq_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, Q_policy(:,:,t), 'spline', 'linear');

                % 对所有 nsim 个体进行插值，得到决策
                simC_norm(t, :) = Fc_interpolant(norm_cash, simF_norm(t, :));
                simA(t, :) = Fa_interpolant(norm_cash, simF_norm(t, :));
                simQ_norm(t, :) = Fq_interpolant(norm_cash, simF_norm(t, :));

                % 应用约束和边界条件
                simA(t, :) = max(min(simA(t, :), 1), 0);
                simC_norm(t, :) = max(min(simC_norm(t, :), norm_cash), 0);
                simQ_norm(t, :) = max(min(simQ_norm(t, :), cS.Q_max), 0);

                if t <= work_periods % 工作期
                    total_outflow = simC_norm(t, :) + simQ_norm(t, :);
                    idx_exceed = total_outflow > norm_cash;
                    if any(idx_exceed)
                        scale_factor = norm_cash(idx_exceed) ./ total_outflow(idx_exceed) * 0.9999;
                        simC_norm(t, idx_exceed) = simC_norm(t, idx_exceed) .* scale_factor;
                        simQ_norm(t, idx_exceed) = simQ_norm(t, idx_exceed) .* scale_factor;
                    end
                    simQ_Cash_ratio(t, :) = simQ_norm(t, :) ./ norm_cash;
                    simQ_Cash_ratio(t, isnan(simQ_Cash_ratio(t,:)) | isinf(simQ_Cash_ratio(t,:))) = 0;
                    liquid_sav_norm = norm_cash - simC_norm(t, :) - simQ_norm(t, :);
                else % 退休期
                    simQ_norm(t,:) = 0;
                    simC_norm(t,:) = min(simC_norm(t,:), norm_cash * 0.9999);
                    liquid_sav_norm = norm_cash - simC_norm(t, :);
                end

                simS_norm(t, :) = simA(t, :) .* liquid_sav_norm;
                simB_norm(t, :) = liquid_sav_norm - simS_norm(t, :);

                % 更新下一期状态 (归一化单位)
                if t < cS.tn
                    portfolio_return_next = simB_norm(t, :) * cS.rf + simS_norm(t, :) .* simR(t, :);
                    simW_norm(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);

                    if t < work_periods
                        simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
                    elseif t == work_periods % 退休前最后一年
                        simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
                    else % 退休后
                        simF_norm(t+1, :) = simF_norm(t, :);
                    end
                end
            end
            fprintf('  归一化单位模拟完成。\n');

            % --- 3. 将结果转换为以人民币计价的真实数值 ---
            fprintf('  将模拟结果转换为真实人民币单位...\n');

            % **锚定参数**
            initial_income_yuan = 60000; % 假设22岁时平均年持久性收入为 60,000 元

            % 初始化真实单位数组
            simP_real = zeros(cS.tn, nsim); % 真实持久性收入 (CNY)
            simY_real = zeros(cS.tn, nsim); % 真实总收入 (CNY)
            simW_real = zeros(cS.tn, nsim); % 真实流动性财富 (CNY)
            simF_real = zeros(cS.tn, nsim); % 真实养老金财富 (CNY)
            simC_real = zeros(cS.tn, nsim); % 真实消费 (CNY)
            simQ_real = zeros(cS.tn, nsim); % 真实养老金缴费 (CNY)

            % 模拟真实持久性收入路径 P_t
            simP_real(1, :) = initial_income_yuan; % 锚定初始值
            for t = 1:(cS.tn - 1)
                simP_real(t+1, :) = simP_real(t, :) .* simGPY(t+1, :);
            end

            % "去归一化" 所有相关变量
            for t = 1:cS.tn
                simW_real(t, :) = simW_norm(t, :) .* simP_real(t, :);
                simF_real(t, :) = simF_norm(t, :) .* simP_real(t, :);
                simC_real(t, :) = simC_norm(t, :) .* simP_real(t, :);
                simQ_real(t, :) = simQ_norm(t, :) .* simP_real(t, :);
                % 计算真实总收入
                if t <= work_periods
                    simY_real(t, :) = simP_real(t, :) .* simY_norm(t, :);
                else
                    simY_real(t, :) = simP_real(t, :);
                end
            end

            % --- 4. 汇总真实结果并绘图 (单位: 万元) ---
            fprintf('  汇总真实结果并绘图 (单位: 万元)...\n');
            yuan_to_wanyuan = 1 / 10000;
            ages = (cS.tb:cS.td)';
            x_fill = [ages; flipud(ages)]; % 用于填充置信区间的x坐标

            % --- 数据处理 ---
            % 均值
            meanC_plot = mean(simC_real, 2) * yuan_to_wanyuan;
            meanY_plot = mean(simY_real, 2) * yuan_to_wanyuan;
            meanTotalW_plot = mean(simW_real + simF_real, 2) * yuan_to_wanyuan;
            meanLiquidW_plot = mean(simW_real, 2) * yuan_to_wanyuan;
            meanF_plot = mean(simF_real, 2) * yuan_to_wanyuan;
            meanA_plot = nanmean(simA, 2);
            meanQ_plot = mean(simQ_real, 2) * yuan_to_wanyuan;

            % 标准差
            stdC_plot = std(simC_real, 0, 2) * yuan_to_wanyuan;
            stdY_plot = std(simY_real, 0, 2) * yuan_to_wanyuan;
            stdTotalW_plot = std(simW_real + simF_real, 0, 2) * yuan_to_wanyuan;
            stdLiquidW_plot = std(simW_real, 0, 2) * yuan_to_wanyuan;
            stdF_plot = std(simF_real, 0, 2) * yuan_to_wanyuan;
            stdA_plot = nanstd(simA, 0, 2);
            stdQ_plot = std(simQ_real, 0, 2) * yuan_to_wanyuan;

            % --- 绘图 ---
            figure('Position', [100, 100, 1400, 900]);

            % 子图 1: 生命周期剖面均值
            subplot(2,2,1);
            hold on;
            % 颜色定义
            color_total_w = [0, 0.4470, 0.7410]; % Blue
            color_liquid_w = [0.3010, 0.7450, 0.9330]; % Cyan
            color_c = [0.8500, 0.3250, 0.0980]; % Red
            color_y = [0, 0, 0]; % Black

            % 绘制置信区间 (fill)
            fill(x_fill, [meanTotalW_plot - stdTotalW_plot; flipud(meanTotalW_plot + stdTotalW_plot)], color_total_w, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanLiquidW_plot - stdLiquidW_plot; flipud(meanLiquidW_plot + stdLiquidW_plot)], color_liquid_w, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanC_plot - stdC_plot; flipud(meanC_plot + stdC_plot)], color_c, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanY_plot - stdY_plot; flipud(meanY_plot + stdY_plot)], color_y, 'FaceAlpha', 0.1, 'EdgeColor', 'none');

            % 绘制均值线
            plot(ages, meanTotalW_plot, 'Color', color_total_w, 'LineWidth', 2, 'DisplayName', '总财富 (流动+养老金)');
            plot(ages, meanLiquidW_plot, 'Color', color_liquid_w, 'LineStyle', '--', 'LineWidth', 1.5, 'DisplayName', '流动性财富');
            plot(ages, meanC_plot, 'Color', color_c, 'LineWidth', 2, 'DisplayName', '消费');
            plot(ages, meanY_plot, 'Color', color_y, 'LineStyle', ':', 'LineWidth', 2, 'DisplayName', '收入');
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('生命周期剖面 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('人民币 (万元)');
            legend('show', 'Location', 'northwest'); grid on; xlim([cS.tb, cS.td]);

            % 子图 2: 投资与养老金缴费决策
            subplot(2,2,2);
            hold on;
            % 颜色定义
            color_a = [0.4940, 0.1840, 0.5560]; % Purple
            color_q = [0.4660, 0.6740, 0.1880]; % Green

            % 左轴
            yyaxis left;
            % 置信区间
            fill(x_fill, [meanA_plot - stdA_plot; flipud(meanA_plot + stdA_plot)], color_a, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            fill(x_fill, [meanQ_plot - stdQ_plot; flipud(meanQ_plot + stdQ_plot)], color_q, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanA_plot, 'Color', color_a, 'LineWidth', 2, 'DisplayName', '风险资产配置比例 (\alpha)');
            plot(ages(1:work_periods), meanQ_plot(1:work_periods), 'Color', color_q, 'LineWidth', 2, 'DisplayName', '个人养老金年缴费 (万元)');
            ylabel('比例 / 人民币 (万元)');
            ylim([-0.05, 1.05]);

            % 右轴
            yyaxis right;
            simQY_ratio_real = simQ_real ./ simY_real;
            simQY_ratio_real(isinf(simQY_ratio_real) | isnan(simQY_ratio_real)) = NaN;
            meanQY_ratio_plot = nanmean(simQY_ratio_real, 2);
            stdQY_ratio_plot = nanstd(simQY_ratio_real, 0, 2);
            % 置信区间
            fill(x_fill(1:2*work_periods), [meanQY_ratio_plot(1:work_periods) - stdQY_ratio_plot(1:work_periods); flipud(meanQY_ratio_plot(1:work_periods) + stdQY_ratio_plot(1:work_periods))], color_q, 'FaceAlpha', 0.1, 'EdgeColor', 'none');
            % 均值线
            plot(ages(1:work_periods), meanQY_ratio_plot(1:work_periods), 'Color', color_q, 'LineStyle', '--', 'LineWidth', 1.5, 'DisplayName', '缴费/收入比');
            ylabel('缴费收入比');

            hold off;
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            title('投资与养老金缴费决策 (均值 \pm 1\sigma)'); xlabel('年龄');
            grid on; xlim([cS.tb, cS.td]);
            legend('show', 'Location', 'best');

            % 子图 3: 个人养老金账户财富动态
            subplot(2,2,3);
            hold on;
            color_f = [0.9290, 0.6940, 0.1250]; % Orange
            % 置信区间
            fill(x_fill, [meanF_plot - stdF_plot; flipud(meanF_plot + stdF_plot)], color_f, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanF_plot, 'Color', color_f, 'LineWidth', 2, 'DisplayName', '养老金账户资产');
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('个人养老金账户财富动态 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('人民币 (万元)');
            legend('show', 'Location', 'northwest'); grid on; xlim([cS.tb, cS.td]);

            % 子图 4: 总财富-收入比均值
            subplot(2,2,4);
            hold on;
            color_ratio = [0.6350, 0.0780, 0.1840]; % Maroon
            simWY_ratio_real = (simW_real + simF_real) ./ simY_real;
            simWY_ratio_real(isinf(simWY_ratio_real) | isnan(simWY_ratio_real)) = NaN;
            meanWY_ratio_plot = nanmean(simWY_ratio_real, 2);
            stdWY_ratio_plot = nanstd(simWY_ratio_real, 0, 2);
            % 置信区间
            fill(x_fill, [meanWY_ratio_plot - stdWY_ratio_plot; flipud(meanWY_ratio_plot + stdWY_ratio_plot)], color_ratio, 'FaceAlpha', 0.15, 'EdgeColor', 'none');
            % 均值线
            plot(ages, meanWY_ratio_plot, 'Color', color_ratio, 'LineWidth', 2);
            xline(cS.tr, '--', '退休', 'LineWidth', 1.5);
            hold off;
            title('总财富-收入比 (均值 \pm 1\sigma)'); xlabel('年龄'); ylabel('比率');
            grid on; xlim([cS.tb, cS.td]);

            sgtitle('生命周期模拟结果 (以2024年人民币计价)', 'FontSize', 16, 'FontWeight', 'bold');

            output_filename = fullfile(cS.save_path, 'vfi_simulation_results_real_cny_with_ci.png');
            print(gcf, output_filename, '-dpng', '-r300');
            fprintf('真实单位模拟图形已保存到 %s\n', output_filename);
        end
        % =====================================================================
        %                辅助函数: 计算归一化的养老金缴费上限
        % =====================================================================
        function q_max_normalized = calculate_q_max(absolute_q_max_yuan, anchor_income_yuan, f_y_profile)
            % 描述:
            % 本函数根据现实世界的绝对缴费上限(如12000元)和代表性收入水平，
            % 计算出适用于模型归一化单位的缴费上限比率 cS.Q_max。
            %
            % 输入:
            %   absolute_q_max_yuan - 政策规定的年度个人养老金缴费绝对上限 (人民币)
            %   anchor_income_yuan  - 用于锚定模型收入水平的初始工作年收入 (人民币)
            %   f_y_profile         - 模型计算出的整个工作生涯的确定性收入剖面 (无单位)
            %
            % 输出:
            %   q_max_normalized    - 归一化后的缴费上限 (相对于平均持久性收入的比率)

            fprintf('  正在计算归一化的养老金缴费上限 (Q_max)...\n');

            % 1. 检查输入是否有效
            if isempty(f_y_profile) || f_y_profile(1) <= 0
                error('输入的收入剖面 f_y_profile 无效。');
            end

            % 2. 计算转换因子，将模型的抽象收入单位与人民币挂钩
            % 因子 = 人民币 / 模型单位
            yuan_per_model_unit = anchor_income_yuan / f_y_profile(1);
            fprintf('    锚定收入: 初始年收入 %.0f 元 对应模型单位 %.4f\n', anchor_income_yuan, f_y_profile(1));

            % 3. 计算整个工作生涯的确定性收入路径 (人民币计价)
            work_income_yuan = f_y_profile * yuan_per_model_unit;

            % 4. 计算工作生涯的平均年收入 (代表性的持久性收入水平)
            average_work_income_yuan = mean(work_income_yuan);
            fprintf('    模型隐含的工作生涯平均年收入为: %.0f 元\n', average_work_income_yuan);

            % 5. 计算归一化的 Q_max
            q_max_normalized = absolute_q_max_yuan / average_work_income_yuan;
            fprintf('    政策上限 %.0f 元 对应的归一化 Q_max 为: %.4f\n', absolute_q_max_yuan, q_max_normalized);

        end
        %  =====================================================================
        %  =====================================================================
        function [z_grid, P] = discretizeAR1_Tauchen(mew, rho, sigma, znum, Tauchen_q, ~)
            % Tauchen方法离散化AR(1)过程
            if znum == 1
                z_grid = mew / (1 - rho);
                P = 1;
                return;
            end

            zstar = mew / (1 - rho);
            sigmaz = sigma / sqrt(1 - rho^2);

            z_grid = zstar + linspace(-Tauchen_q * sigmaz, Tauchen_q * sigmaz, znum)';
            omega = z_grid(2) - z_grid(1);

            P = zeros(znum, znum);
            for i = 1:znum
                for j = 1:znum
                    if j == 1
                        P(i, j) = normcdf((z_grid(j) + omega/2 - rho * z_grid(i) - mew) / sigma);
                    elseif j == znum
                        P(i, j) = 1 - normcdf((z_grid(j) - omega/2 - rho * z_grid(i) - mew) / sigma);
                    else
                        P(i, j) = normcdf((z_grid(j) + omega/2 - rho * z_grid(i) - mew) / sigma) - ...
                            normcdf((z_grid(j) - omega/2 - rho * z_grid(i) - mew) / sigma);
                    end
                end
            end
        end


    end

end

## 脚本

### main_PPS: 生成基准结果

In [ ]:


% --- 初始化 ---
clear;
close all;
clc;
warning off

% --- 主流程控制器 ---
fprintf('开始执行生命周期模型...\n');

% 1. 设置所有模型参数
cS = utils_pps.setup_parameters();
save_path = cS.save_path;
file_path = fullfile(save_path, 'vfi_results_rikyrp.mat');

% 2. 求解模型 (值函数迭代)
fprintf('\n===== 步骤 2: 求解模型 (VFI) =====\n');
if ~exist(file_path,'file')
    vfi_results = utils_pps.solve_model_vfi(cS);
    if ~exist(save_path, 'dir'), mkdir(save_path); end
    save(file_path, 'vfi_results', 'cS');
    fprintf('模型求解结果已保存到 %s\n', file_path);
else
    load(file_path)
end

% 保存结果


% 3. 模拟模型
fprintf('\n===== 步骤 3: 模拟模型 =====\n');
utils_pps.simulate_model(vfi_results, cS);

fprintf('\n模型执行完毕。\n');



### main_PPS_riskyrp: 生成不同风险性PPS结果

In [ ]:
%% 批量求解不同参数设定下的生命周期模型
%
% 描述:
%   本脚本旨在为一系列关键模型参数生成VFI求解结果，用于后续的
%   敏感性分析。脚本会遍历每个指定参数的一组典型值，为每个值调用
%   VFI求解器，并将结果分别保存到 'result/riskyrp/' 文件夹下。
%
%   脚本会自动跳过基准参数值的计算，以避免重复工作。
%

%
% 输出:
%   - result/riskyrp/vfi_results_riskyrp_sigrrp_0p1000.mat
%   - result/riskyrp/vfi_results_riskyrp_sigrrp_0p2000.mat


% --- 初始化 ---
clear;
close all;
clc;
warning off;

% --- 1. 定义敏感性分析的配置 ---
% 每个cell元素代表一个要分析的参数
% 'name': 用于文件命名的简称
% 'field': 在cS结构体中对应的字段名(字符串形式)
% 'values': 需要进行分析的一系列数值
sensitivity_params = {
    % 参数 1: 
    struct('name', 'sigrrp', ...
           'field', 'cS.sigrrp', ...
           'values', [0, 0.05, 0.1, 0.2,0.27, 0.3]), ...
};

% --- 2. 循环遍历每个参数及其设定值 ---
for p_idx = 1:length(sensitivity_params)
    param_info = sensitivity_params{p_idx};
    param_name = param_info.name;
    param_field = param_info.field;
    param_values = param_info.values;

    for v_idx = 1:length(param_values)
        current_value = param_values(v_idx);

        fprintf('\n\n======================================================\n');
        fprintf('===== 开始敏感性分析 =====\n');
        fprintf('===== 参数: %s, 当前值: %f =====\n', upper(param_name), current_value);
        fprintf('======================================================\n');

        % --- a. 获取基准参数 ---
        % 每次都重新加载，以确保从一个干净的基准开始
        cS = utils_pps.setup_parameters('sensitivity');
        cS.ncash = 51;
        cS.nfund = 51;

        % --- c. 覆盖当前分析所需的参数值 ---
        % 使用 eval 动态修改 cS 结构体中对应的参数
        eval(sprintf('%s = %f;', param_field, current_value));


        % --- d. 定义文件保存路径和名称 ---
        save_dir = fullfile('result', 'riskyrp');
        if ~exist(save_dir, 'dir')
            mkdir(save_dir);
        end

        value_str = strrep(sprintf('%.4f', current_value), '.', 'p');
        file_name = sprintf('vfi_results_riskyrp_%s_%s.mat', param_name, value_str);
        file_path = fullfile(save_dir, file_name);

        % --- e. 求解或加载模型 ---
        fprintf('\n--- 正在为 %s = %f 求解模型 ---\n', param_name, current_value);
        cS = utils_pps.setup_process(cS); % 派生参数设置
        if ~exist(file_path, 'file')
            fprintf('未找到结果文件，开始进行值函数迭代...\n');
            vfi_results = utils_pps.solve_model_vfi_riskyrp(cS);

            % utils_pps.simulate_model(vfi_results, cS);

            save(file_path, 'vfi_results', 'cS');
            fprintf('模型求解结果已成功保存到: %s\n', file_path);
        else
            fprintf('已在路径 %s 找到对应的结果文件，跳过计算。\n', file_path);
        end
    end
end

fprintf('\n\n所有指定的敏感性分析已全部完成。\n');

### main_q_max: 生成不同PPS上限结果

In [ ]:
%% 批量求解不同 Q_max 设定下的生命周期模型
%
% 描述:
%   本脚本旨在生成一系列在不同个人养老金年度缴费上限(Q_max)设定下的
%   模型求解结果(VFI results)。脚本会循环遍历一组预定义的、以人民币计价的
%   缴费上限，为每个值调用VFI求解器，并将结果分别保存到不同的 .mat 文件中。
%
%   这些生成的文件将作为后续进行效用成本分析(utility cost analysis)的
%   基础数据。
%
%   本脚本假设 Q_max=0 (对应 vfi_results_nopps.mat) 和
%   Q_max=99999 (对应 vfi_results.mat) 的结果已经存在，因此不会重复计算。
%
% 输出:
%   - result/vfi_results_qmax_6000.mat
%   - result/vfi_results_qmax_12000.mat
%   - result/vfi_results_qmax_24000.mat
%   - result/vfi_results_qmax_36000.mat
%   (以及对应的 cS 结构体)

% --- 初始化 ---
clear;
close all;
clc;
warning off;

% --- 1. 定义要进行敏感性分析的 Q_max 值 (人民币绝对值) ---
% 我们选择当前政策(12000元)及其倍数作为分析点，以评估政策变化的潜在影响。
absolute_q_max_yuan_vec = [3000, 6000, 12000, 24000, 36000];  % 

% --- 2. 循环求解每个 Q_max 设定 ---
for i = 1:length(absolute_q_max_yuan_vec)

    current_q_max_abs = absolute_q_max_yuan_vec(i);
    fprintf('\n\n======================================================\n');
    fprintf('===== 开始处理 Q_max = %d 元的设定 =====\n', current_q_max_abs);
    fprintf('======================================================\n');

    % --- a. 获取基础参数 ---
    % 每次循环都重新获取，以确保 cS 是一个干净的基准。
    cS = utils_pps.setup_parameters();

    % --- b. 计算并覆盖当前循环的归一化 Q_max ---
    % 注意: 这里假设 calculate_q_max 也是 utils_pps 类的一个静态方法。
    % 如果不是，请确保该函数在MATLAB路径上。
    % 锚定收入需要与 setup_parameters 中的设定保持一致。
    anchor_income_yuan = 60000;
    try
        cS.Q_max = utils_pps.calculate_q_max(current_q_max_abs, anchor_income_yuan, cS.f_y);
    catch ME
        error('无法调用 utils_pps.calculate_q_max。请确保该辅助函数已移入 utils_pps.m 静态类中。错误信息: %s', ME.message);
    end

    % --- c. 定义当前设定的文件保存路径 ---
    save_path = cS.save_path;

    file_name = sprintf('vfi_results_qmax_%d.mat', current_q_max_abs);
    if cS.Q_max==0
        file_name = 'vfi_results_nopps.mat';
    end
    file_path = fullfile(save_path, file_name);

    % --- d. 求解或加载模型 ---
    fprintf('\n--- 步骤 2: 求解模型 (VFI) for Q_max = %d ---\n', current_q_max_abs);

    if ~exist(file_path, 'file')
        fprintf('未找到结果文件，开始进行值函数迭代...\n');
        vfi_results = utils_pps.solve_model_vfi(cS);

        if ~exist(save_path, 'dir')
            mkdir(save_path);
        end

        save(file_path, 'vfi_results', 'cS');
        fprintf('模型求解结果已成功保存到: %s\n', file_path);
    else
        fprintf('已在路径 %s 找到对应的结果文件，跳过计算。\n', file_path);
    end
end

fprintf('\n\n所有指定的 Q_max 值的模型求解已全部完成。\n');

### main_sensitivity：敏感性分析

In [ ]:
%% 批量求解不同参数设定下的生命周期模型 (敏感性分析)
%
% 描述:
%   本脚本旨在为一系列关键模型参数生成VFI求解结果，用于后续的
%   敏感性分析。脚本会遍历每个指定参数的一组典型值，为每个值调用
%   VFI求解器，并将结果分别保存到 'result/sensitivity/' 文件夹下。
%
%   脚本会自动跳过基准参数值的计算，以避免重复工作。
%
% 分析的参数:
%   1. 永久性收入冲击标准差 (smav)
%   2. 相对风险规避系数 (gamma)
%   3. 风险资产波动率 (sigr)
%   4. 个人养老金账户回报率 (rp)
%
% 输出:
%   - result/sensitivity/vfi_results_smav_0p1000.mat
%   - result/sensitivity/vfi_results_gamma_2p0000.mat
%   - ... (以此类推)

% --- 初始化 ---
clear;
close all;
clc;
warning off;

% --- 1. 定义敏感性分析的配置 ---
% 每个cell元素代表一个要分析的参数
% 'name': 用于文件命名的简称
% 'field': 在cS结构体中对应的字段名(字符串形式)
% 'values': 需要进行分析的一系列数值
sensitivity_params = {
    % 参数 1: 永久性收入冲击标准差 (smav)，基于方差设定
    struct('name', 'smav', ...
           'field', 'cS.smav', ...
           'values', [sqrt(0.02), sqrt(0.2), sqrt(0.4), sqrt(0.6)]), ...

    % 参数 2: 相对风险规避系数 (gamma)
    struct('name', 'gamma', ...
           'field', 'cS.gamma', ...
           'values', [2.0, 5.0, 7.5, 10.0]), ...

    % 参数 3: 风险资产波动率 (sigr)
    struct('name', 'sigr', ...
           'field', 'cS.sigr', ...
           'values', [0.01, 0.15, 0.35, 0.45]), ...

    % 参数 4: 个人养老金账户回报率 (rp)
    struct('name', 'rp', ...
           'field', 'cS.rp', ...
           'values', [1.01, 1.02,1.06, 1.08]),...
               
    % 参数 5: 退休年龄 (tr)           
    struct('name', 'tr', ...
           'field', 'cS.tr', ...
           'values', [58, 62, 63, 64, 65])
};

% --- 2. 循环遍历每个参数及其设定值 ---
for p_idx = 1:length(sensitivity_params)
    param_info = sensitivity_params{p_idx};
    param_name = param_info.name;
    param_field = param_info.field;
    param_values = param_info.values;

    for v_idx = 1:length(param_values)
        current_value = param_values(v_idx);

        fprintf('\n\n======================================================\n');
        fprintf('===== 开始敏感性分析 =====\n');
        fprintf('===== 参数: %s, 当前值: %f =====\n', upper(param_name), current_value);
        fprintf('======================================================\n');

        % --- a. 获取基准参数 ---
        % 每次都重新加载，以确保从一个干净的基准开始
        cS = utils_pps.setup_parameters('sensitivity');
        cS.ncash = 31;
        cS.nfund = 31;


        % --- b. 检查当前值是否为基准值，若是则跳过 ---
        % temp = strsplit(param_field, '.');
        % baseline_value = getfield(cS, temp{2} ); % 动态获取基准值
        % if abs(current_value - baseline_value) < 1e-9
        %     fprintf('当前值为此参数的基准值 (%f)，跳过计算。\n', baseline_value);
        %     continue;
        % end

        % --- c. 覆盖当前分析所需的参数值 ---
        % 使用 eval 动态修改 cS 结构体中对应的参数
        eval(sprintf('%s = %f;', param_field, current_value));


        % --- d. 定义文件保存路径和名称 ---
        save_dir = fullfile('result', 'sensitivity');
        if ~exist(save_dir, 'dir')
            mkdir(save_dir);
        end

        value_str = strrep(sprintf('%.4f', current_value), '.', 'p');
        file_name = sprintf('vfi_results_%s_%s.mat', param_name, value_str);
        file_path = fullfile(save_dir, file_name);

        % --- e. 求解或加载模型 ---
        fprintf('\n--- 正在为 %s = %f 求解模型 ---\n', param_name, current_value);
        cS = utils_pps.setup_process(cS); % 派生参数设置
        if ~exist(file_path, 'file')
            fprintf('未找到结果文件，开始进行值函数迭代...\n');
            vfi_results = utils_pps.solve_model_vfi(cS);

            utils_pps.simulate_model(vfi_results, cS);

            save(file_path, 'vfi_results', 'cS');
            fprintf('模型求解结果已成功保存到: %s\n', file_path);
        else
            fprintf('已在路径 %s 找到对应的结果文件，跳过计算。\n', file_path);
        end
    end
end

fprintf('\n\n所有指定的敏感性分析已全部完成。\n');

### main_PPS_decompose: Q的分解

In [ ]:


% --- 初始化 ---
clear;
close all;
clc;
warning off

% --- 主流程控制器 ---
fprintf('开始执行生命周期模型...\n');

%%  无税收激励
% 1. 设置所有模型参数
cS = utils_pps_notax_incentive.setup_parameters();
save_path = cS.save_path;
file_path = fullfile(save_path, 'breakdown','vfi_results_notaxincentive.mat');

% 2. 求解模型 (值函数迭代)
fprintf('\n===== 步骤 2: 求解模型 (VFI) =====\n');
if ~exist(file_path,'file')
    vfi_results = utils_pps_notax_incentive.solve_model_vfi(cS);
    if ~exist(save_path, 'dir'), mkdir(save_path); end
    save(file_path, 'vfi_results', 'cS');
    fprintf('模型求解结果已保存到 %s\n', file_path);
else
    load(file_path)
end

%%  完全流动性个人养老金
% 1. 设置所有模型参数
cS = utils_pps_fullliquidity.setup_parameters();
save_path = cS.save_path;
file_path = fullfile(save_path, 'breakdown','vfi_results_fullliquidity.mat');

% 2. 求解模型 (值函数迭代)
fprintf('\n===== 步骤 2: 求解模型 (VFI) =====\n');
if ~exist(file_path,'file')
    vfi_results = utils_pps_fullliquidity.solve_model_vfi(cS);
    if ~exist(save_path, 'dir'), mkdir(save_path); end
    save(file_path, 'vfi_results', 'cS');
    fprintf('模型求解结果已保存到 %s\n', file_path);
else
    load(file_path)
end


%% 无收入不确定性

% 1. 设置所有模型参数
cS = utils_pps.setup_parameters('test');
cS.smav = 0;
cS.smay = 0;
cS = utils_pps.setup_process(cS);
save_path = cS.save_path;
file_path = fullfile(save_path, 'breakdown','vfi_results_noincomerisk.mat');

% 2. 求解模型 (值函数迭代)
fprintf('\n===== 步骤 2: 求解模型 (VFI) =====\n');
if ~exist(file_path,'file')
    vfi_results = utils_pps.solve_model_vfi(cS);
    if ~exist(save_path, 'dir'), mkdir(save_path); end
    save(file_path, 'vfi_results', 'cS');
    fprintf('模型求解结果已保存到 %s\n', file_path);
else
    load(file_path)
end





# 论文图表结果生成

## 策略函数

### 图4.1-1 -- 不同年龄下的消费

In [ ]:
%% 绘制固定年龄下的缴费决策策略函数 (Q(W,F))
%
% 描述:
%   本脚本读取 VFI 求解器保存的 vfi_results.mat 文件，
%   并为三个特定年龄 (22, 40, 60岁) 分别绘制个人养老金缴费决策 (Q)
%   如何随现金资产 (W_t) 和养老金账户余额 (F_t) 变化的 3D 策略曲面。
%
% 输出:
%   - figs/fig4.1_C_policy_F_fixed.png: 灰度图，400 DPI, 宽度严格为 14.99cm

% --- 初始化环境 ---
clear;
close all;
clc;

% --- 1. 加载数据 ---
fprintf('正在加载模型求解结果...\n');
try
    data = load('result/vfi_results.mat', 'vfi_results', 'cS');
    vfi_results = data.vfi_results;
    cS = data.cS;
    fprintf('数据加载成功。\n');
catch ME
    error('无法加载 result/vfi_results.mat 文件。请确保已成功运行 main_PPS.m 并生成了该文件。错误信息: %s', ME.message);
end

% --- 2. 设置绘图参数 ---
fprintf('正在设置绘图参数...\n');

% 提取所需变量
C_policy = vfi_results.C_policy;


gcash = cS.gcash;                % 现金网格
gfund = cS.gfund;                % 养老金网格
tb = cS.tb;                      % 初始工作年龄
tr = cS.tr;                      % 退休年龄

% 平滑处理

% 定义要展示的特定年龄
target_ages = [22, 40, 70];

% 检查并修正年龄范围
if target_ages(1) < tb
    fprintf('注意: 目标年龄 %d 小于模型起始年龄 %d。将使用起始年龄 %d 代替。\n', target_ages(1), tb, tb);
    target_ages(1) = tb;
end

% 将年龄转换为策略矩阵中的时间索引 t (t = age - tb + 1)
time_indices = target_ages - tb + 1;

% --- 3. 准备绘图数据 ---
fprintf('正在准备绘图数据...\n');

% 创建用于绘图的网格 (X: gcash, Y: gfund)
[W_grid, F_grid] = meshgrid(gcash(1:30), gfund(1:30));

% --- 4. 创建并保存图像 ---
fprintf('正在生成图像...\n');

% 创建图形窗口
fig = figure('Name', 'A Policy Surface at Fixed Ages');

% 设置图形窗口尺寸 (宽度严格为14.99cm)
figure_width_cm = 14.99;
figure_height_cm = 4.5; % 根据宽高比调整高度
set(fig, 'Units', 'centimeters', 'Position', [5, 5, figure_width_cm, figure_height_cm]);

% 为所有子图设置一个统一的、信息丰富的视角
common_view = [-49.4982   26.4119];

% 循环创建三个子图
for i = 1:3
    ax = subplot(1, 3, i, 'Parent', fig);
    
    % 提取当前年龄对应的 Q 策略数据
    current_t_idx = time_indices(i);
    % C_policy 维度: (W, F, T) -> (ncash, nfund, tn)
    % surf 需要的 Z 矩阵维度: (rows, cols) -> (nfund, ncash)
    % 因此需要对切片进行转置
    C_slice = squeeze(C_policy(:, :, current_t_idx))';
    
    % 绘制3D表面图
    surf(ax, W_grid, F_grid, C_slice(1:30,1:30), 'EdgeColor', 'none');
    
    % 应用平滑着色，消除黑色网格线
    shading(ax, 'interp');
    
    % 设置统一视角
    view(ax, common_view);
    
    % 设置坐标轴标签
    xlabel(ax, '$W_t$', 'FontSize', 12,'Interpreter','latex');
    ylabel(ax, '$F_t$', 'FontSize', 12,'Interpreter','latex');
    if i == 1
        % 创建zlabel并获取其句柄
        zl = zlabel(ax, '$C_t$', 'FontSize', 12,'Interpreter','latex','Rotation',0);
        % *** 修改: 调整zlabel的位置 ***
        % 根据当前视角，向x轴负方向移动标签，使其远离坐标轴
        currentPos = get(zl, 'Position');
        offset = 0; % 这个值可以根据需要微调
        set(zl, 'Position', [-4.4111 4.2367 0.8135]);
    end
    
    % 设置子图标题，标明当前年龄
    title(ax, sprintf('(%c) %d岁', char(97 + i - 1), target_ages(i)), 'FontSize', 12, 'FontWeight', 'normal');
    
    % 设置坐标轴范围
    zlim(ax, [0, 2]);
    
    zticks(ax, 0:0.5:2);
    yticks(ax, 0:1:3);
    xticks(ax, 0:3:10);
    
    % 设置字体和网格线
    set(ax, 'FontSize', 10, 'Box', 'on');

    % *** 新增: 强制不显示指数并减少边距，使刻度数字更靠近坐标轴 ***
    ax.XAxis.Exponent = 0;
    ax.YAxis.Exponent = 0;
end

% 应用反转的灰度颜色图
colormap(flipud(gray));

% --- 5. 保存图像 ---
% 检查输出目录是否存在，如果不存在则创建
output_dir = 'tex/fig/ch04';
% if ~exist(output_dir, 'dir')
%     mkdir(output_dir);
%     fprintf('创建目录: %s\n', output_dir);
% end

% 定义输出文件名 (文件名保持不变)
output_filename = fullfile(output_dir, 'fig4.1-1_C_policy_different_age.png');

% 以指定的分辨率保存图像
print(fig, output_filename, '-dpng', '-r400');

fprintf('图像已成功保存至: %s\n', output_filename);

### 图 4.2 -- 生命周期视角下的个人养老金策略

In [ ]:
%% 绘制生命周期下的缴费决策策略函数 Q(t|W,F)
%
% 描述:
%   本脚本读取 VFI 求解器保存的 vfi_results.mat 文件，
%   并为三个固定的养老金账户余额 (F_t) 水平 (低、中、高) 分别绘制子图。
%   在每个子图中，展示个人养老金缴费决策 (Q) 如何随年龄 (t) 变化。
%   每张子图包含三条曲线，分别对应三个现金资产 (W_t) 水平 (低、中、高)。
%
% 输出:
%   - tex/fig/ch04/fig4.3_Q_policy_lifecycle.png: 灰度图，400 DPI, 宽度严格为 14.99cm

% --- 初始化环境 ---
clear;
close all;
clc;

% --- 1. 加载数据 ---
fprintf('正在加载模型求解结果...\n');
try
    data = load('result/vfi_results.mat', 'vfi_results', 'cS');
    vfi_results = data.vfi_results;
    cS = data.cS;
    fprintf('数据加载成功。\n');
catch ME
    error('无法加载 result/vfi_results.mat 文件。请确保已成功运行 main_PPS.m 并生成了该文件。错误信息: %s', ME.message);
end

% --- 2. 设置绘图参数 ---
fprintf('正在设置绘图参数...\n');

% 提取所需变量
Q_policy = vfi_results.Q_policy; % 缴费策略函数 (ncash, nfund, tn)
gcash = cS.gcash;                % 现金网格
gfund = cS.gfund;                % 养老金网格
tb = cS.tb;                      % 初始工作年龄
tr = cS.tr;                      % 退休年龄
ncash = cS.ncash;                % 现金网格点数量
nfund = cS.nfund;                % 养老金网格点数量

% 定义工作期年龄和对应的时间索引
% Q_policy 只在工作期 (t=1 到 t=tr-tb) 有非零值
work_ages = tb:(tr - 1);
time_indices = 1:(tr - tb);

% 定义要展示的 "低、中、高" 水平在网格上的索引
% 选择了网格中大约 20%, 50%, 80% 的位置来代表低、中、高
cash_indices = [round(ncash*0.2), round(ncash*0.3),  round(ncash*0.5)];
fund_indices = [round(nfund*0.01), round(nfund*0.45), round(nfund*0.55), round(nfund*0.8)];

% 获取对应的 W_t 和 F_t 的实际数值，用于图例和标题
cash_levels = gcash(cash_indices);
fund_levels = gfund(fund_indices);

% --- 3. 准备绘图数据 ---
% 数据提取将在绘图循环中直接进行，此处无需额外准备

% --- 4. 创建并保存图像 ---
fprintf('正在生成图像...\n');

% 创建图形窗口
fig = figure('Name', 'Q Policy Over Lifecycle for different W and F');

% 设置图形窗口尺寸 (宽度严格为14.99cm)
figure_width_cm = 14.99;
figure_height_cm = 4.0; % 调整高度以适应 1x3 子图布局
set(fig, 'Units', 'centimeters', 'Position', [5, 5, figure_width_cm, figure_height_cm]);

% 定义曲线样式
line_styles = {'--', '-', '-.'};
line_colors = {'k', 'k', 'k'}; % 使用黑色，靠线型区分
line_widths = {1, 2, 1};

% 预先计算所有子图统一的Y轴范围，以保证可比性
q_data_subset = Q_policy(cash_indices, fund_indices, time_indices);
max_q_value = max(q_data_subset(:));
y_limit = [0, 3];
% 如果所有Q都为0，设置一个默认的非零上限
if y_limit(2) == 0
    y_limit(2) = 1;
end


% 计算 ASCII 值并转换为字符
% 'a' 的 ASCII 值是 97。当 inputNum=1 时, 97+1-1=97 -> 'a'


% 循环创建三个子图 (每个子图代表一个固定的 F_t 水平)
for i = 1:4
    ax = subplot(1, 4, i, 'Parent', fig);
    hold(ax, 'on'); % 激活 hold on 以便绘制多条曲线
    
    current_fund_idx = fund_indices(i);
    
    % 在当前 F_t 水平下，循环绘制三条曲线 (每个曲线代表一个固定的 W_t 水平)
    for j = 1:3
        current_cash_idx = cash_indices(j);
        
        % 提取当前 W_t 和 F_t 组合下，随年龄变化的 Q 决策
        % Q_policy(cash_idx, fund_idx, time_indices)
        q_over_time = squeeze(Q_policy(current_cash_idx, current_fund_idx, time_indices));
        
        % 绘制曲线
        plot(ax, work_ages, q_over_time, ...
            'LineStyle', line_styles{j}, ...
            'Color', line_colors{j}, ...
            'LineWidth', line_widths{j});
    end
    
    hold(ax, 'off'); % 关闭 hold
    
    % 设置坐标轴标签和标题
    % if i == 2 || i==3
    xlabel(ax, '年龄', 'FontSize', 10); % 
    % end
    if i == 1
        ylabel(ax, '$Q_t$', 'FontSize', 10, 'Interpreter', 'latex','Rotation',0);
    end
    title(ax, sprintf('(%c) $F_t \\approx %.2f$', char(97 + i - 1),fund_levels(i)), ...
          'FontSize', 11, 'FontWeight', 'normal', 'Interpreter', 'latex');
          
    % 设置坐标轴范围和刻度
    xlim(ax, [tb, tr - 1]);
    ylim(ax, y_limit);
    
    % 设置字体和网格线
    set(ax, 'FontSize', 9, 'Box', 'on');
    grid(ax, 'on');
end

% 在最后一个子图的右侧添加一个全局图例
legend_labels = {sprintf('$W_t \\approx %.2f$', cash_levels(1)), ...
                 sprintf('$W_t \\approx %.2f$', cash_levels(2)), ...
                 sprintf('$W_t \\approx %.2f$', cash_levels(3))};
             
% --- 现有代码 ---
lgd = legend(ax, legend_labels, 'Interpreter', 'latex', 'FontSize', 8,'Box','off');

% --- 在此处添加以下代码 ---
% 将示例线的长度设置为15点 (默认值约为30)
lgd.ItemTokenSize(1) = 15;% 调整图例位置到整个图的右侧外部
% lgd.Layout.Tile = 'East';

% --- 5. 保存图像 ---
% 检查输出目录是否存在，如果不存在则创建
output_dir = 'tex/fig/ch04';
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('创建目录: %s\n', output_dir);
end

% 定义输出文件名
output_filename = fullfile(output_dir, 'fig4.2_Q_policy_differentFandW.png');

% 以指定的分辨率保存图像
print(fig, output_filename, '-dpng', '-r400');

fprintf('图像已成功保存至: %s\n', output_filename);

### 图 4.3 -- 不同年龄下的个人养老金策略

In [ ]:
%% 绘制固定年龄下的缴费决策策略函数 (Q(W,F))
%
% 描述:
%   本脚本读取 VFI 求解器保存的 vfi_results.mat 文件，
%   并为三个特定年龄 (22, 40, 60岁) 分别绘制个人养老金缴费决策 (Q)
%   如何随现金资产 (W_t) 和养老金账户余额 (F_t) 变化的 3D 策略曲面。
%
% 输出:
%   - figs/fig4.1_Q_policy_F_fixed.png: 灰度图，400 DPI, 宽度严格为 14.99cm

% --- 初始化环境 ---
clear;
close all;
clc;

% --- 1. 加载数据 ---
fprintf('正在加载模型求解结果...\n');
try
    data = load('result/vfi_results.mat', 'vfi_results', 'cS');
    vfi_results = data.vfi_results;
    cS = data.cS;
    fprintf('数据加载成功。\n');
catch ME
    error('无法加载 result/vfi_results.mat 文件。请确保已成功运行 main_PPS.m 并生成了该文件。错误信息: %s', ME.message);
end

% --- 2. 设置绘图参数 ---
fprintf('正在设置绘图参数...\n');

% 提取所需变量
Q_policy = vfi_results.Q_policy; % 缴费策略函数 (ncash, nfund, tn)
gcash = cS.gcash;                % 现金网格
gfund = cS.gfund;                % 养老金网格
tb = cS.tb;                      % 初始工作年龄
tr = cS.tr;                      % 退休年龄

% 定义要展示的特定年龄
target_ages = [22, 30, 40];

% 检查并修正年龄范围
if target_ages(1) < tb
    fprintf('注意: 目标年龄 %d 小于模型起始年龄 %d。将使用起始年龄 %d 代替。\n', target_ages(1), tb, tb);
    target_ages(1) = tb;
end
if target_ages(end) >= tr
    fprintf('注意: 目标年龄 %d 大于或等于退休年龄 %d。将使用退休前最后一年龄 %d 代替。\n', target_ages(end), tr, tr-1);
    target_ages(end) = tr-1;
end

% 将年龄转换为策略矩阵中的时间索引 t (t = age - tb + 1)
time_indices = target_ages - tb + 1;

% --- 3. 准备绘图数据 ---
fprintf('正在准备绘图数据...\n');

% 创建用于绘图的网格 (X: gcash, Y: gfund)
[W_grid, F_grid] = meshgrid(gcash(1:30), gfund(1:30));

% --- 4. 创建并保存图像 ---
fprintf('正在生成图像...\n');

% 创建图形窗口
fig = figure('Name', 'Q Policy Surface at Fixed Ages');

% 设置图形窗口尺寸 (宽度严格为14.99cm)
figure_width_cm = 14.99;
figure_height_cm = 4.5; % 根据宽高比调整高度
set(fig, 'Units', 'centimeters', 'Position', [5, 5, figure_width_cm, figure_height_cm]);

% 为所有子图设置一个统一的、信息丰富的视角
common_view = [-45, 30];

% 循环创建三个子图
for i = 1:3
    ax = subplot(1, 3, i, 'Parent', fig);
    
    % 提取当前年龄对应的 Q 策略数据
    current_t_idx = time_indices(i);
    % Q_policy 维度: (W, F, T) -> (ncash, nfund, tn)
    % surf 需要的 Z 矩阵维度: (rows, cols) -> (nfund, ncash)
    % 因此需要对切片进行转置
    Q_slice = squeeze(Q_policy(:, :, current_t_idx))';
    
    % 绘制3D表面图
    surf(ax, W_grid, F_grid, Q_slice(1:30,1:30), 'EdgeColor', 'none');
    
    % 应用平滑着色，消除黑色网格线
    shading(ax, 'interp');
    
    % 设置统一视角
    view(ax, common_view);
    
    % 设置坐标轴标签
    xlabel(ax, '$W_t$', 'FontSize', 12,'Interpreter','latex');
    ylabel(ax, '$F_t$', 'FontSize', 12,'Interpreter','latex');
    if i == 1
        % 创建zlabel并获取其句柄
        zl = zlabel(ax, '$Q_t$', 'FontSize', 12,'Interpreter','latex','Rotation',0);
        % *** 修改: 调整zlabel的位置 ***
        % 根据当前视角，向x轴负方向移动标签，使其远离坐标轴
        currentPos = get(zl, 'Position');
        offset = 0.7; % 这个值可以根据需要微调
        set(zl, 'Position', currentPos + [-offset, offset, 0]);
    end
    
    % 设置子图标题，标明当前年龄
    title(ax, sprintf('(%c) %d岁', char(97 + i - 1), target_ages(i)), 'FontSize', 12, 'FontWeight', 'normal');
    
    % 设置坐标轴范围
    zlim(ax, [0, 5]);
    
    zticks(ax, 0:1:5);
    yticks(ax, 0:1:3);
    xticks(ax, 0:3:10);
    
    % 设置字体和网格线
    set(ax, 'FontSize', 10, 'Box', 'on');

    % *** 新增: 强制不显示指数并减少边距，使刻度数字更靠近坐标轴 ***
    ax.XAxis.Exponent = 0;
    ax.YAxis.Exponent = 0;
end

% 应用反转的灰度颜色图
% colormap(flipud(gray));

% --- 5. 保存图像 ---
% 检查输出目录是否存在，如果不存在则创建
output_dir = 'tex/fig/ch04';
% if ~exist(output_dir, 'dir')
%     mkdir(output_dir);
%     fprintf('创建目录: %s\n', output_dir);
% end

% 定义输出文件名 (文件名保持不变)
output_filename = fullfile(output_dir, 'fig4.3_Q_policy_different_age.png');

% 以指定的分辨率保存图像
print(fig, output_filename, '-dpng', '-r400');

fprintf('图像已成功保存至: %s\n', output_filename);

### 表4.3 --个人养老金缴费决策驱动因素精确分解 

In [ ]:
% =========================================================================
% == SCRIPT: create_q_decomposition_table.m
% == 版本: [v3.0 - 个人养老金缴费决策驱动因素精确分解器]
% ==
% == 目的:
% ==   1. 严格按照指定的分解逻辑，量化缴费决策的驱动因素。
% ==   2. 核心逻辑: 理论潜力 - 总流动性成本 = 实际缴费。
% ==   3. 进一步分析收入风险如何调节（可能增加或减少）最终决策。
% ==   4. 生成一个逻辑严谨、脚注清晰的LaTeX分析表格，用于最终报告。
% ==
% == v3.0 更新:
% ==   - 完全重写列的定义、计算公式和脚注，以匹配新的、更精确的分析框架。
% ==   - 明确了“收入风险的调节效应”的正负号含义。
% ==   - 调整了校验和的定义，使其直接验证核心分解逻辑。
% ==
% == 前置条件:
% ==   - 已成功运行main脚本并生成以下所有.mat文件:
% ==     - result/vfi_results.mat (基准模型)
% ==     - result/breakdown/vfi_results_fullliquidity.mat (无流动性约束)
% ==     - result/breakdown/vfi_results_noincomerisk.mat (无收入风险)
% ==     - result/breakdown/vfi_results_notaxincentive.mat (无税收激励)
% =========================================================================

clear; close all; clc;
fprintf('=== [v3.0] 个人养老金缴费决策驱动因素精确分解脚本 ===\n\n');

%% --- 1. 用户设定与文件定义 ---
fprintf('--- 1. 用户设定与文件定义 ---\n');

% --- [核心] 定义要分析的模型文件并赋予清晰的别名 ---
model_files = {
    'baseline',         'result/vfi_results.mat';
    'full_liquidity',   'result/breakdown/vfi_results_fullliquidity.mat';
    'no_risk',          'result/breakdown/vfi_results_noincomerisk.mat';
    'no_tax',           'result/breakdown/vfi_results_notaxincentive.mat'
};

% --- [核心] 定义要评估的代表性状态点 ---
assessment_ages = [30, 40, 50];             % 年龄点
w_quantiles = [0.25, 0.50, 0.75];         % 现金持有量 (W) 的分位数
f_quantiles = [0.01, 0.50];               % 养老金存量 (F) 的分位数

% --- 输出路径 ---
output_dir = 'tex/tab/ch04/';
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
output_filename = fullfile(output_dir, 'table_q_decomposition.tex');
fprintf('   - LaTeX输出文件将保存至: %s\n', output_filename);


%% --- 2. 加载所有模型数据 ---
fprintf('\n--- 2. 正在加载所有VFI模型结果 ---\n');

num_models = size(model_files, 1);
model_data = struct();

for i = 1:num_models
    key = model_files{i, 1};
    file_path = model_files{i, 2};
    
    fprintf('   - 正在加载模型 [%s] 从: %s\n', upper(key), file_path);
    if ~exist(file_path, 'file')
        error('找不到必要的输入文件: %s。请先运行相关的main脚本生成此文件。', file_path);
    end
    
    data = load(file_path, 'vfi_results', 'cS');
    model_data.(key).Q_policy = data.vfi_results.Q_policy;
    model_data.(key).cS = data.cS;
end

% 使用基准模型的参数作为参考
cS_sample = model_data.baseline.cS;
fprintf('   ✅ 所有模型数据加载成功。\n');

%% --- 3. 确定代表性状态点的网格索引 ---
fprintf('\n--- 3. 正在确定代表性状态点的网格索引 ---\n');

% 年龄 -> 时间索引 (t = age - tb + 1)
age_indices = assessment_ages - cS_sample.tb + 1;

% 财富分位数 -> 网格索引
w_indices = zeros(size(w_quantiles));
for i = 1:length(w_quantiles)
    [~, w_indices(i)] = min(abs(cumsum(ones(size(cS_sample.gcash)))/length(cS_sample.gcash) - w_quantiles(i)));
end

f_indices = zeros(size(f_quantiles));
for i = 1:length(f_quantiles)
    [~, f_indices(i)] = min(abs(cumsum(ones(size(cS_sample.gfund)))/length(cS_sample.gfund) - f_quantiles(i)));
end

fprintf('   - 年龄: %s -> 时间索引: %s\n', mat2str(assessment_ages), mat2str(age_indices));
fprintf('   - 现金W索引: %s\n', mat2str(w_indices));
fprintf('   - 基金F索引: %s\n', mat2str(f_indices));
fprintf('   ✅ 状态点索引定位完成。\n');


%% --- 4. [核心] 生成LaTeX分析表格 ---
fprintf('\n--- 4. 正在生成LaTeX格式的决策分解表格 ---\n');

fileID = fopen(output_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开LaTeX输出文件。'); end

% --- LaTeX 表格前导码 ---
fprintf(fileID, '%% =============================================================\n');
fprintf(fileID, '%%  此文件由 create_q_decomposition_table.m (v3.0) 自动生成\n');
fprintf(fileID, '%% =============================================================\n');
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{个人养老金缴费决策的驱动因素分解}\n');
fprintf(fileID, '\\label{tab:q_decomposition}\n');
fprintf(fileID, '\\sisetup{table-format=1.3, round-mode=places, round-precision=3, table-number-alignment = center, table-sign-mantissa}\n');
fprintf(fileID, '\\begin{threeparttable}\n');

% 定义表格列格式
col_def = 'c c c S S S S S';
fprintf(fileID, '\\begin{tabular}{%s}\n', col_def);
fprintf(fileID, '\\toprule\n');

% 表头 (使用 \makecell 实现换行)
fprintf(fileID, '{年龄} & {现金(W)} & {基金(F)} & {\\makecell{理论缴费\\\\潜力 \\tnote{a}}} & {\\makecell{总流动性\\\\成本 \\tnote{b}}} & {\\makecell{收入风险的\\\\调节效应 \\tnote{c}}} & {\\makecell{实际最优\\\\缴费 \\tnote{d}}} & {\\makecell{无激励\\\\缴费 \\tnote{e}}} \\\\\n');
fprintf(fileID, '\\midrule\n');

% --- 循环填充表格内容 ---
for i_age = 1:length(assessment_ages)
    age = assessment_ages(i_age);
    t = age_indices(i_age);
    
    for i_w = 1:length(w_quantiles)
        w_idx = w_indices(i_w);
        w_val = cS_sample.gcash(w_idx);
        w_label = sprintf('%.0f\\%%(%.2f)', w_quantiles(i_w)*100, w_val);
        
        for i_f = 1:length(f_quantiles)
            f_idx = f_indices(i_f);
            f_val = cS_sample.gfund(f_idx);
            f_label = sprintf('%.0f\\%%(%.2f)', f_quantiles(i_f)*100, f_val);

            % 从各模型中提取Q值
            Q_baseline = model_data.baseline.Q_policy(w_idx, f_idx, t);
            Q_full_liquidity = model_data.full_liquidity.Q_policy(w_idx, f_idx, t);
            Q_no_risk  = model_data.no_risk.Q_policy(w_idx, f_idx, t);
            Q_no_tax   = model_data.no_tax.Q_policy(w_idx, f_idx, t);

            % 按照prompt中的精确定义进行计算
            potential_q = Q_full_liquidity;
            total_liquidity_cost = Q_full_liquidity - Q_baseline;
            income_risk_effect = Q_baseline - Q_no_risk;
            optimal_q = Q_baseline;
            no_incentive_q = Q_no_tax;

            % 写入行数据
            if i_w == 1 && i_f == 1
                fprintf(fileID, '%d & %s & %s & %f & %f & %f & %f & %f \\\\\n', ...
                    age, w_label, f_label, ...
                    potential_q, total_liquidity_cost, income_risk_effect, optimal_q, no_incentive_q);
            else
                fprintf(fileID, ' & %s & %s & %f & %f & %f & %f & %f \\\\\n', ...
                    w_label, f_label, ...
                    potential_q, total_liquidity_cost, income_risk_effect, optimal_q, no_incentive_q);
            end
        end
    end
    
    if i_age < length(assessment_ages)
        fprintf(fileID, '\\addlinespace\n');
    end
end

% --- LaTeX 表格结尾与脚注 (严格按照prompt提供的内容) ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
% 使用[para,flushleft]并严格复制用户提供的脚注文本
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\footnotesize\n');
fprintf(fileID, '  \\item[注] 表中所有缴费数值均为归一化单位。状态点中的百分比表示该点在对应网格中的分位数位置，括号内为该点的实际数值。各分解项定义如下：\n');
fprintf(fileID, '  \\item[\\textsuperscript{a}] \\textbf{理论缴费潜力 ($Q_{\\text{潜力}}$):} 在一个反事实世界中的最优缴费，该世界中个人养老金账户完全流动（无锁定），但个体依然面临现实的收入风险。该值主要由税收激励驱动。\n');
fprintf(fileID, '  \\item[\\textsuperscript{b}] \\textbf{总流动性成本:} 衡量因资金将被锁定至退休（即引入流动性约束）而导致个体放弃的缴费总额。计算公式：$Q_{\\text{潜力}} - Q_{\\text{最优}}$。\n');
fprintf(fileID, '  \\item[\\textsuperscript{c}] \\textbf{收入风险的调节效应:} 衡量收入不确定性对缴费决策的净影响。计算公式：$Q_{\\text{最优}} - Q_{\\text{无风险}}$。值为正，表示收入风险的存在促使个体增加对作为安全港的个人养老金的配置（避险效应），因此移除该风险后缴费额会下降。\n');
fprintf(fileID, '  \\item[\\textsuperscript{d}] \\textbf{实际最优缴费 ($Q^{*}$):} 基准模型中，综合考虑所有约束和风险后的最终最优决策。\n');
fprintf(fileID, '  \\item[\\textsuperscript{e}] \\textbf{无激励缴费:} 作为参照，在移除税收优惠后的最优缴费决策，其预期值接近于零。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');

fclose(fileID);

fprintf('   ✅ 成功生成精确分解后的 LaTeX 表格文件: %s\n', output_filename);
fprintf('\n--- 脚本执行完毕 ---\n');

### 图 4.4 -- 生命周期视角下的风险资产比例

In [ ]:
%% 绘制生命周期下的缴费决策策略函数 Q(t|W,F)
%
% 描述:
%   本脚本读取 VFI 求解器保存的 vfi_results.mat 文件，
%   并为三个固定的养老金账户余额 (F_t) 水平 (低、中、高) 分别绘制子图。
%   在每个子图中，展示个人养老金缴费决策 (Q) 如何随年龄 (t) 变化。
%   每张子图包含三条曲线，分别对应三个现金资产 (W_t) 水平 (低、中、高)。
%
% 输出:
%   - tex/fig/ch04/fig4.3_A_policy_lifecycle.png: 灰度图，400 DPI, 宽度严格为 14.99cm

% --- 初始化环境 ---
clear;
close all;
clc;

% --- 1. 加载数据 ---
fprintf('正在加载模型求解结果...\n');
try
    data = load('result/vfi_results.mat', 'vfi_results', 'cS');
    vfi_results = data.vfi_results;
    cS = data.cS;
    fprintf('数据加载成功。\n');
catch ME
    error('无法加载 result/vfi_results.mat 文件。请确保已成功运行 main_PPS.m 并生成了该文件。错误信息: %s', ME.message);
end

% --- 2. 设置绘图参数 ---
fprintf('正在设置绘图参数...\n');

% 提取所需变量
A_policy = vfi_results.A_policy; % 缴费策略函数 (ncash, nfund, tn)
gcash = cS.gcash;                % 现金网格
gfund = cS.gfund;                % 养老金网格
tb = cS.tb;                      % 初始工作年龄
tr = cS.tr;                      % 退休年龄
td = cS.td;
ncash = cS.ncash;                % 现金网格点数量
nfund = cS.nfund;                % 养老金网格点数量

% 定义工作期年龄和对应的时间索引
% A_policy 只在工作期 (t=1 到 t=tr-tb) 有非零值
work_ages = tb:td;
time_indices = 1:td-tb+1;

% 定义要展示的 "低、中、高" 水平在网格上的索引
% 选择了网格中大约 20%, 50%, 80% 的位置来代表低、中、高
cash_indices = [round(ncash*0.2), round(ncash*0.3),  round(ncash*0.5)];
fund_indices = [round(nfund*0.01), round(nfund*0.4), round(nfund*0.6), round(nfund*0.7)];

% 获取对应的 W_t 和 F_t 的实际数值，用于图例和标题
cash_levels = gcash(cash_indices);
fund_levels = gfund(fund_indices);

% --- 3. 准备绘图数据 ---
% 数据提取将在绘图循环中直接进行，此处无需额外准备

% --- 4. 创建并保存图像 ---
fprintf('正在生成图像...\n');

% 创建图形窗口
fig = figure('Name', 'Q Policy Over Lifecycle for different W and F');

% 设置图形窗口尺寸 (宽度严格为14.99cm)
figure_width_cm = 14.99;
figure_height_cm = 4; % 调整高度以适应 1x3 子图布局
set(fig, 'Units', 'centimeters', 'Position', [5, 5, figure_width_cm, figure_height_cm]);

% 定义曲线样式
line_styles = {'-.', '-', ':'};
line_colors = {'k', 'k', 'k'}; % 使用黑色，靠线型区分
line_widths = {0.7, 1.2, 1};

% 预先计算所有子图统一的Y轴范围，以保证可比性
q_data_subset = A_policy(cash_indices, fund_indices, time_indices);
max_q_value = max(q_data_subset(:));
y_limit = [0.2, 1.1];
% 如果所有Q都为0，设置一个默认的非零上限
if y_limit(2) == 0
    y_limit(2) = 1;
end


% 计算 ASCII 值并转换为字符
% 'a' 的 ASCII 值是 97。当 inputNum=1 时, 97+1-1=97 -> 'a'
% 在最后一个子图的右侧添加一个全局图例
legend_labels = {sprintf('$W_t \\approx %.2f$', cash_levels(1)), ...
    sprintf('$W_t \\approx %.2f$', cash_levels(2)), ...
    sprintf('$W_t \\approx %.2f$', cash_levels(3))};

% 循环创建三个子图 (每个子图代表一个固定的 F_t 水平)
for i = 1:4
    ax = subplot(1, 4, i, 'Parent', fig);
    hold(ax, 'on'); % 激活 hold on 以便绘制多条曲线

    current_fund_idx = fund_indices(i);

    % 在当前 F_t 水平下，循环绘制三条曲线 (每个曲线代表一个固定的 W_t 水平)
    for j = 1:3
        current_cash_idx = cash_indices(j);

        % 提取当前 W_t 和 F_t 组合下，随年龄变化的 Q 决策
        % A_policy(cash_idx, fund_idx, time_indices)
        q_over_time = squeeze(A_policy(current_cash_idx, current_fund_idx, time_indices));

        % 绘制曲线
        plot(ax, work_ages, q_over_time, ...
            'LineStyle', line_styles{j}, ...
            'Color', line_colors{j}, ...
            'LineWidth', line_widths{j});
    end

    hold(ax, 'off'); % 关闭 hold

    % 设置坐标轴标签和标题
    if i == 2
        xlabel(ax, '年龄', 'FontSize', 10); %
                lgd = legend(ax, legend_labels, 'Interpreter', 'latex', 'FontSize', 8,'Box','off',...
                    'Position',[0.5585 -0.0131 0.3465 0.0988],'Orientation', 'horizontal');

        % --- 在此处添加以下代码 ---
        % 将示例线的长度设置为15点 (默认值约为30)
        lgd.ItemTokenSize(1) = 6;% 调整图例位置到整个图的右侧外部
    end
    if i == 1
        ylabel(ax, '$\alpha_t$', 'FontSize', 10, 'Interpreter', 'latex','Rotation',0);
    end


    title(ax, sprintf('(%c) $F_t \\approx %.2f$', char(97 + i - 1),fund_levels(i)), ...
        'FontSize', 11, 'FontWeight', 'normal', 'Interpreter', 'latex');

    % 设置坐标轴范围和刻度
    xlim(ax, [tb, td]);
    xticks([30,45,60,75,90]);
    ax.XTickLabelRotation = 45;
    ax.TickLength = [0.02, 0.025]; 
    % ax.FontName = 'Times New Roman';
    ylim(ax, y_limit);

    % 设置字体和网格
    set(ax, 'FontSize', 9, 'Box', 'on');
    grid(ax, 'on');
end




% --- 5. 保存图像 ---
% 检查输出目录是否存在，如果不存在则创建
output_dir = 'tex/fig/ch04';
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('创建目录: %s\n', output_dir);
end

% 定义输出文件名
output_filename = fullfile(output_dir, 'fig4.4_A_policy_differentFandW.png');

% 以指定的分辨率保存图像
print(fig, output_filename, '-dpng', '-r400');

fprintf('图像已成功保存至: %s\n', output_filename);

### 图 4.5 -- 不同年龄下的风险资产比例

In [ ]:
%% 绘制固定年龄下的缴费决策策略函数 (Q(W,F))
%
% 描述:
%   本脚本读取 VFI 求解器保存的 vfi_results.mat 文件，
%   并为三个特定年龄 (22, 40, 60岁) 分别绘制个人养老金缴费决策 (Q)
%   如何随现金资产 (W_t) 和养老金账户余额 (F_t) 变化的 3D 策略曲面。
%
% 输出:
%   - figs/fig4.1_A_policy_F_fixed.png: 灰度图，400 DPI, 宽度严格为 14.99cm

% --- 初始化环境 ---
clear;
close all;
clc;

% --- 1. 加载数据 ---
fprintf('正在加载模型求解结果...\n');
try
    data = load('result/vfi_results.mat', 'vfi_results', 'cS');
    vfi_results = data.vfi_results;
    cS = data.cS;
    fprintf('数据加载成功。\n');
catch ME
    error('无法加载 result/vfi_results.mat 文件。请确保已成功运行 main_PPS.m 并生成了该文件。错误信息: %s', ME.message);
end

% --- 2. 设置绘图参数 ---
fprintf('正在设置绘图参数...\n');

% 提取所需变量
A_policy = vfi_results.A_policy; % 缴费策略函数 (ncash, nfund, tn)
Q_policy = vfi_results.Q_policy; % 缴费策略函数 (ncash, nfund, tn)
C_policy = vfi_results.C_policy;


gcash = cS.gcash;                % 现金网格
gfund = cS.gfund;                % 养老金网格
tb = cS.tb;                      % 初始工作年龄
tr = cS.tr;                      % 退休年龄

% 平滑处理
A_policy(find(abs(cS.gcash-C_policy-(1-cS.tau_y)*Q_policy)<1e-04))=1;

% 定义要展示的特定年龄
target_ages = [22, 40, 70];

% 检查并修正年龄范围
if target_ages(1) < tb
    fprintf('注意: 目标年龄 %d 小于模型起始年龄 %d。将使用起始年龄 %d 代替。\n', target_ages(1), tb, tb);
    target_ages(1) = tb;
end

% 将年龄转换为策略矩阵中的时间索引 t (t = age - tb + 1)
time_indices = target_ages - tb + 1;

% --- 3. 准备绘图数据 ---
fprintf('正在准备绘图数据...\n');

% 创建用于绘图的网格 (X: gcash, Y: gfund)
[W_grid, F_grid] = meshgrid(gcash, gfund);

% --- 4. 创建并保存图像 ---
fprintf('正在生成图像...\n');

% 创建图形窗口
fig = figure('Name', 'A Policy Surface at Fixed Ages');

% 设置图形窗口尺寸 (宽度严格为14.99cm)
figure_width_cm = 14.99;
figure_height_cm = 4.5; % 根据宽高比调整高度
set(fig, 'Units', 'centimeters', 'Position', [5, 5, figure_width_cm, figure_height_cm]);

% 为所有子图设置一个统一的、信息丰富的视角
common_view = [45, 30];

% 循环创建三个子图
for i = 1:3
    ax = subplot(1, 3, i, 'Parent', fig);
    
    % 提取当前年龄对应的 Q 策略数据
    current_t_idx = time_indices(i);
    % A_policy 维度: (W, F, T) -> (ncash, nfund, tn)
    % surf 需要的 Z 矩阵维度: (rows, cols) -> (nfund, ncash)
    % 因此需要对切片进行转置
    A_slice = squeeze(A_policy(:, :, current_t_idx))';
    
    % 绘制3D表面图
    surf(ax, W_grid, F_grid, A_slice, 'EdgeColor', 'none');
    
    % 应用平滑着色，消除黑色网格线
    shading(ax, 'interp');
    
    % 设置统一视角
    view(ax, common_view);
    
    % 设置坐标轴标签
    xlabel(ax, '$W_t$', 'FontSize', 12,'Interpreter','latex');
    ylabel(ax, '$F_t$', 'FontSize', 12,'Interpreter','latex');
    if i == 1
        % 创建zlabel并获取其句柄
        zl = zlabel(ax, '$\alpha_t$', 'FontSize', 12,'Interpreter','latex','Rotation',0);
        % *** 修改: 调整zlabel的位置 ***
        % 根据当前视角，向x轴负方向移动标签，使其远离坐标轴
        currentPos = get(zl, 'Position');
        offset = 0; % 这个值可以根据需要微调
        set(zl, 'Position', [-75.0759 -89.0171 0.3904]);
    end
    
    % 设置子图标题，标明当前年龄
    title(ax, sprintf('(%c) %d岁', char(97 + i - 1), target_ages(i)), 'FontSize', 12, 'FontWeight', 'normal');
    
    % 设置坐标轴范围
    zlim(ax, [0, 1]);
    
    % zticks(ax, 0:1:5);
    % yticks(ax, 0:1:3);
    % xticks(ax, 0:3:10);
    
    % 设置字体和网格线
    set(ax, 'FontSize', 10, 'Box', 'on');

    % *** 新增: 强制不显示指数并减少边距，使刻度数字更靠近坐标轴 ***
    ax.XAxis.Exponent = 0;
    ax.YAxis.Exponent = 0;
end

% 应用反转的灰度颜色图
colormap(flipud(gray));

% --- 5. 保存图像 ---
% 检查输出目录是否存在，如果不存在则创建
output_dir = 'tex/fig/ch04';
% if ~exist(output_dir, 'dir')
%     mkdir(output_dir);
%     fprintf('创建目录: %s\n', output_dir);
% end

% 定义输出文件名 (文件名保持不变)
output_filename = fullfile(output_dir, 'fig4.5_A_policy_different_age.png');

% 以指定的分辨率保存图像
print(fig, output_filename, '-dpng', '-r400');

fprintf('图像已成功保存至: %s\n', output_filename);

## 数值模拟

### 图 4.6 -- 基本结果

In [ ]:
%% 绘制生命周期模拟的关键路径图 (多面板) - 灰度版
%
% 描述:
%   本脚本加载 VFI 结果，重新运行蒙特卡洛模拟，并生成三面板图：
%   (a) 消费、风险资产和养老金购买的比例 (中位数与10-90分位区间)。
%   (b) 消费、养老金缴费和收入的绝对值 (中位数，单位万元)。
%   (c) 收入与财富/养老金账户的比率 (中位数与10-90分位区间)。
%
% 输出:
%   - tex/fig/ch04/fig4.6_simu_overall_grayscale.png: PNG格式图，400 DPI, 宽度严格为 14.99cm

% --- 初始化环境 ---
clear;
close all;
clc;

% --- 1. 加载数据 ---
fprintf('正在加载模型求解结果...\n');
try
    data = load('result/vfi_results.mat', 'vfi_results', 'cS');
    vfi_results = data.vfi_results;
    cS = data.cS;
    fprintf('数据加载成功。\n');
catch ME
    error('无法加载 result/vfi_results.mat 文件。请确保已成功运行 main_PPS.m 并生成了该文件。错误信息: %s', ME.message);
end

% --- 2. 重新运行模拟以捕获所有轨迹 ---
fprintf('正在重新运行蒙特卡洛模拟以获取完整数据分布...\n');

% --- 2.1. 初始化 ---
rng(42); 
nsim = 10000; 
C_policy = vfi_results.C_policy;
A_policy = vfi_results.A_policy;
Q_policy = vfi_results.Q_policy;
simGPY = ones(cS.tn, nsim);       
simY_norm = zeros(cS.tn, nsim);    
simR = zeros(cS.tn, nsim);         
simW_norm = zeros(cS.tn, nsim);    
simF_norm = zeros(cS.tn, nsim);    
simC_norm = zeros(cS.tn, nsim);    
simQ_norm = zeros(cS.tn, nsim);    
simA = zeros(cS.tn, nsim);         

% --- 2.2. 模拟外生冲击和收入过程 ---
work_periods = cS.tr - cS.tb;
for i1 = 1:floor(nsim/2)
    z_shocks = randn(work_periods, 1) * cS.smav;
    u_shocks_log = randn(work_periods, 1) * cS.smay;
    r_shocks = randn(cS.tn, 1) * cS.sigr;
    simGPY(2:work_periods+1, i1) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(z_shocks);
    simY_norm(1:work_periods, i1) = exp(u_shocks_log);
    simR(:, i1) = cS.rf + cS.mu + r_shocks;
    i2 = nsim/2 + i1;
    simGPY(2:work_periods+1, i2) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(-z_shocks);
    simY_norm(1:work_periods, i2) = exp(-u_shocks_log);
    simR(:, i2) = cS.rf + cS.mu - r_shocks;
end
simY_norm(work_periods+1:cS.tn, :) = cS.ret_fac;

% --- 2.3. 迭代模拟生命周期决策 (归一化单位) ---
sim_norm_cash = zeros(cS.tn, nsim); 
for t = 1:cS.tn
    if t <= work_periods 
        norm_cash = simW_norm(t, :) + (1-cS.tau_y) * simY_norm(t, :);
    else 
        norm_pension_payout = (1-cS.tau_q) * simF_norm(t, :) * cS.pension_rate;
        norm_cash = simW_norm(t, :) + simY_norm(t, :) + norm_pension_payout;
    end
    sim_norm_cash(t, :) = norm_cash;
    
    Fc_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, C_policy(:,:,t), 'spline', 'linear');
    Fa_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, A_policy(:,:,t), 'spline', 'linear');
    Fq_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, Q_policy(:,:,t), 'spline', 'linear');

    simC_norm(t, :) = Fc_interpolant(norm_cash, simF_norm(t, :));
    simA(t, :) = Fa_interpolant(norm_cash, simF_norm(t, :));
    simQ_norm(t, :) = Fq_interpolant(norm_cash, simF_norm(t, :));

    simA(t, :) = max(min(simA(t, :), 1), 0);
    simC_norm(t, :) = max(min(simC_norm(t, :), norm_cash), 0);
    simQ_norm(t, :) = max(min(simQ_norm(t, :), cS.Q_max), 0);
    
    if t <= work_periods
        total_outflow = simC_norm(t, :) + (1-cS.tau_y)*simQ_norm(t, :);
        idx_exceed = total_outflow > norm_cash;
        if any(idx_exceed)
            scale_factor = norm_cash(idx_exceed) ./ total_outflow(idx_exceed) * 0.9999;
            simC_norm(t, idx_exceed) = simC_norm(t, idx_exceed) .* scale_factor;
            simQ_norm(t, idx_exceed) = simQ_norm(t, idx_exceed) .* scale_factor;
        end
        liquid_sav_norm = norm_cash - simC_norm(t, :) - (1-cS.tau_y)*simQ_norm(t, :);
    else 
        simQ_norm(t,:) = 0;
        simC_norm(t,:) = min(simC_norm(t,:), norm_cash * 0.9999);
        liquid_sav_norm = norm_cash - simC_norm(t, :);
    end

    simS_norm = simA(t, :) .* liquid_sav_norm;
    simB_norm = liquid_sav_norm - simS_norm;

    if t < cS.tn
        portfolio_return_next = simB_norm * cS.rf + simS_norm .* simR(t, :);
        simW_norm(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);
        if t <= work_periods
            simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
        else
            simF_norm(t+1, :) = simF_norm(t, :);
        end
    end
end

% --- 2.4. 转换为真实人民币单位 ---
initial_income_yuan = 60000;
simP_real = zeros(cS.tn, nsim);
simP_real(1, :) = initial_income_yuan;
for t = 1:(cS.tn - 1)
    simP_real(t+1, :) = simP_real(t, :) .* simGPY(t+1, :);
end
simW_real = simW_norm .* simP_real;
simF_real = simF_norm .* simP_real;
simC_real = simC_norm .* simP_real;
simQ_real = simQ_norm .* simP_real;
% 计算真实收入（工作期为劳动收入，退休期为基础养老金）
simY_real = simY_norm .* simP_real;

fprintf('模拟数据生成完毕。\n');

% --- 3. 计算绘图所需变量 ---
fprintf('正在计算中位数和分位区间...\n');
ages = (cS.tb:cS.td)';
x_fill = [ages; flipud(ages)]; 
yuan_to_wanyuan = 1 / 10000;

% (a) 子图变量：比例
sim_c_ratio = simC_norm ./ sim_norm_cash;
sim_c_ratio(isinf(sim_c_ratio) | isnan(sim_c_ratio)) = 0;
med_c_ratio = median(sim_c_ratio, 2);
p10_c_ratio = prctile(sim_c_ratio, 10, 2);
p90_c_ratio = prctile(sim_c_ratio, 90, 2);

med_alpha = nanmedian(simA, 2);
p10_alpha = prctile(simA, 10, 2);
p90_alpha = prctile(simA, 90, 2);

sim_q_ratio = simQ_norm ./ sim_norm_cash;
sim_q_ratio(isinf(sim_q_ratio) | isnan(sim_q_ratio)) = 0;
med_q_ratio = median(sim_q_ratio, 2);
p10_q_ratio = prctile(sim_q_ratio, 10, 2);
p90_q_ratio = prctile(sim_q_ratio, 90, 2);

% (b) 子图变量：真实数值 (万元)，仅中位数
med_C_real = median(simC_real, 2) * yuan_to_wanyuan;
med_Q_real = median(simQ_real, 2) * yuan_to_wanyuan;
med_Y_real = median(simY_real, 2) * yuan_to_wanyuan;

% (c) 子图变量：比率
% 个人养老金给付（真实值）
sim_pension_payout_real = zeros(cS.tn, nsim);
sim_pension_payout_real(work_periods+1:end, :) = (1-cS.tau_q) * simF_real(work_periods+1:end, :) * cS.pension_rate;

% 比率1: (收入+个人养老金收入)/财富W
ratio1 = (simY_real + sim_pension_payout_real) ./ simW_real;
ratio1(isinf(ratio1) | isnan(ratio1)) = NaN;
med_r1 = nanmedian(ratio1, 2);
p10_r1 = prctile(ratio1, 10, 2);
p90_r1 = prctile(ratio1, 90, 2);

% 比率2: (收入)/财富W
ratio2 = simY_real ./ simW_real;
ratio2(isinf(ratio2) | isnan(ratio2)) = NaN;
med_r2 = nanmedian(ratio2, 2);
p10_r2 = prctile(ratio2, 10, 2);
p90_r2 = prctile(ratio2, 90, 2);

% 比率3: 收入/个人养老金账户F
ratio3 = simY_real ./ simF_real;
ratio3(isinf(ratio3) | isnan(ratio3)) = NaN;
med_r3 = nanmedian(ratio3, 2);
p10_r3 = prctile(ratio3, 10, 2);
p90_r3 = prctile(ratio3, 90, 2);

% --- 4. 创建并保存图像 ---
fprintf('正在生成图像...\n');
fig = figure('Name', 'Lifecycle Simulation Profiles (Grayscale)');
figure_width_cm = 14.99;
figure_height_cm = 4.5;
set(fig, 'Units', 'centimeters', 'Position', [5, 5, figure_width_cm, figure_height_cm]);
pause(0.1); 
retirement_age_line = cS.tr - 1;

% --- 灰度颜色和样式定义 ---
gray_dark = [0.2 0.2 0.2];
gray_medium = [0.5 0.5 0.5];
gray_light = [0.7 0.7 0.7];
fill_alpha = 0.2; % 填充区域的透明度
line_style_solid = '-';
line_style_dashed = '--';
line_style_dashdot = '-.';

% --- 子图 (a) ---
ax1 = subplot(1, 3, 1);
hold(ax1, 'on');
fill(x_fill, [p10_c_ratio; flipud(p90_c_ratio)], gray_dark, 'FaceAlpha', fill_alpha, 'EdgeColor', 'none', 'HandleVisibility', 'off');
fill(x_fill, [p10_alpha; flipud(p90_alpha)], gray_medium, 'FaceAlpha', fill_alpha, 'EdgeColor', 'none', 'HandleVisibility', 'off');
fill(x_fill, [p10_q_ratio; flipud(p90_q_ratio)], gray_light, 'FaceAlpha', fill_alpha, 'EdgeColor', 'none', 'HandleVisibility', 'off');
plot(ax1, ages, med_c_ratio, 'LineWidth', 1.5, 'Color', gray_dark, 'LineStyle', line_style_solid, 'DisplayName', '$C_t/W_t$');
plot(ax1, ages(1:work_periods+1), med_q_ratio(1:work_periods+1), 'LineWidth', 1.5, 'Color', gray_light, 'LineStyle', line_style_dashdot, 'DisplayName', '$Q_t/W_t$');
plot(ax1, ages, med_alpha, 'LineWidth', 1.5, 'Color', gray_medium, 'LineStyle', line_style_dashed, 'DisplayName', '$\alpha_t$');
xline(ax1, retirement_age_line, '--', {'退休年龄'}, 'LineWidth', 1, 'Color', gray_medium, 'LabelColor', gray_medium, 'HandleVisibility', 'off');
hold(ax1, 'off');
grid(ax1, 'on');
title(ax1, '(a) 决策比例');
xlabel(ax1, '年龄');
xlim(ax1, [cS.tb, cS.td]);
ylim(ax1, [0, 1.05]);
z1 = legend(ax1, 'show', 'box', 'off', 'interpreter', 'latex', 'Location', 'northwest');
z1.ItemTokenSize(1) = 10;
set(ax1, 'FontSize', 8);

% --- 子图 (b) ---
ax2 = subplot(1, 3, 2);
hold(ax2, 'on');
plot(ax2, ages, med_C_real, 'LineWidth', 1.5, 'Color', gray_dark, 'LineStyle', line_style_solid, 'DisplayName', '$C_t$');
plot(ax2, ages(1:work_periods+1), med_Q_real(1:work_periods+1), 'LineWidth', 1.5, 'Color', gray_light, 'LineStyle', line_style_dashdot, 'DisplayName', '$Q_t$');
plot(ax2, ages, med_Y_real, 'LineWidth', 1.5, 'Color', gray_medium, 'LineStyle', line_style_dashed, 'DisplayName', '$Y_t$ ');
xline(ax2, retirement_age_line, '--', 'LineWidth', 1, 'Color', gray_medium, 'LabelColor', gray_medium, 'HandleVisibility', 'off');
hold(ax2, 'off');
grid(ax2, 'on');
title(ax2, '(b) 实际决策值');
xlabel(ax2, '年龄');
ylabel(ax2, '人民币（万元）');
xlim(ax2, [cS.tb, cS.td]);
z2 = legend(ax2, 'show', 'box', 'off', 'interpreter', 'latex', 'Location', 'northwest');
z2.ItemTokenSize(1) = 10;
set(ax2, 'FontSize', 8);

% --- 子图 (c) ---
ax3 = subplot(1, 3, 3);
hold(ax3, 'on');
fill(x_fill, [p10_r1; flipud(p90_r1)], gray_dark, 'FaceAlpha', fill_alpha, 'EdgeColor', 'none', 'HandleVisibility', 'off');
fill(x_fill, [p10_r2; flipud(p90_r2)], gray_medium, 'FaceAlpha', fill_alpha, 'EdgeColor', 'none', 'HandleVisibility', 'off');
fill(x_fill, [p10_r3; flipud(p90_r3)], gray_light, 'FaceAlpha', fill_alpha, 'EdgeColor', 'none', 'HandleVisibility', 'off');
plot(ax3, ages, med_r1, 'LineWidth', 1.5, 'Color', gray_dark, 'LineStyle', line_style_solid, 'DisplayName', '$(Y+Y_p)/W$');
plot(ax3, ages, med_r2, 'LineWidth', 1.5, 'Color', gray_medium, 'LineStyle', line_style_dashed, 'DisplayName', '$Y/W$');
plot(ax3, ages, med_r3, 'LineWidth', 1.5, 'Color', gray_light, 'LineStyle', line_style_dashdot, 'DisplayName', '$Y/F$');
xline(ax3, retirement_age_line, '--', 'LineWidth', 1, 'Color', gray_medium, 'LabelColor', gray_medium, 'HandleVisibility', 'off');
hold(ax3, 'off');
grid(ax3, 'on');
title(ax3, '(c) 收入-财富比');
xlabel(ax3, '年龄');
xlim(ax3, [cS.tb, cS.td]);
ylim(ax3, [0, 5]); 
z3 = legend(ax3, 'show', 'box', 'off', 'interpreter', 'latex', 'Location', 'northwest');
z3.ItemTokenSize(1) = 10;
set(ax3, 'FontSize', 8);

% --- 5. 保存图像 ---
output_dir = 'tex/fig/ch04';
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('创建目录: %s\n', output_dir);
end
output_filename = fullfile(output_dir, 'fig4.6_simu_overall.png'); % 新文件名
print(fig, output_filename, '-dpng', '-r400');
fprintf('灰度图像已成功保存至: %s\n', output_filename);

### 图 4.7 -- 与无PPS模型的比较

In [ ]:
%% 对比有无个人养老金(PPS)的生命周期模拟路径 (仅中位数，自定义图例) - 灰度版
%
% 描述:
%   本脚本加载两个模型（有PPS和无PPS）的VFI求解结果，
%   分别进行蒙特卡洛模拟，并生成一个 1x3 的对比图，旨在凸显
%   引入个人养老金账户(PPS)对生命周期行为的影响。
%
%   (a) 财富路径: 对比总财富和流动性财富的积累。
%   (b) 风险决策: 对比流动性资产中风险资产的配置比例 (alpha)。
%   (c) 消费路径: 对比消费的生命周期模式。
%
%   所有路径均只显示中位数。
%
% 前提:
%   - 运行 main_PPS.m   生成 result/vfi_results.mat
%   - 运行 main_noPPS.m 生成 result/vfi_results_nopps.mat
%
% 输出:
%   - tex/fig/ch04/fig_pps_vs_nopps_comparison.png: PNG格式图，400 DPI, 宽度严格为 14.99cm

% --- 初始化环境 ---
clear;
close all;
clc;
fprintf('开始生成PPS模型对比图 (自定义图例)...\n');

% --- 1. 加载数据 ---
% 1.1 加载包含PPS的模型结果
fprintf('正在加载包含PPS的模型结果...\n');
try
    data_pps = load('result/vfi_results.mat', 'vfi_results', 'cS');
    vfi_results_pps = data_pps.vfi_results;
    cS_pps = data_pps.cS;
    fprintf('  [成功] vfi_results.mat 加载完毕。\n');
catch ME
    error('无法加载 result/vfi_results.mat 文件。请确保已成功运行 main_PPS.m。错误信息: %s', ME.message);
end

% 1.2 加载不包含PPS的模型结果
fprintf('正在加载不包含PPS的模型结果...\n');
try
    data_nopps = load('result/vfi_results_nopps.mat', 'vfi_results', 'cS');
    vfi_results_nopps = data_nopps.vfi_results;
    cS_nopps = data_nopps.cS;
    fprintf('  [成功] vfi_results_nopps.mat 加载完毕。\n');
catch ME
    error('无法加载 result/vfi_results_nopps.mat 文件。请确保已成功运行 main_noPPS.m。错误信息: %s', ME.message);
end

% --- 2. 模拟设置 ---
rng(42);
nsim = 10000;
work_periods = cS_pps.tr - cS_pps.tb;
tn = cS_pps.tn;
initial_income_yuan = 60000;
yuan_to_wanyuan = 1 / 10000;
ages = (cS_pps.tb:cS_pps.td)';

% --- 2.1 模拟外生冲击 (两个模型使用完全相同的冲击路径以保证可比性) ---
fprintf('正在生成外生冲击路径...\n');
simGPY = ones(tn, nsim);
simY_norm_work = zeros(work_periods, nsim);
simR = zeros(tn, nsim);

for i1 = 1:floor(nsim/2)
    z_shocks = randn(work_periods, 1) * cS_pps.smav;
    u_shocks_log = randn(work_periods, 1) * cS_pps.smay;
    r_shocks = randn(tn, 1) * cS_pps.sigr;
    i2 = nsim/2 + i1;
    
    % 正向路径
    simGPY(2:work_periods+1, i1) = cS_pps.f_y(2:work_periods+1) ./ cS_pps.f_y(1:work_periods) .* exp(z_shocks);
    simY_norm_work(:, i1) = exp(u_shocks_log);
    simR(:, i1) = cS_pps.rf + cS_pps.mu + r_shocks;
    % 反向路径
    simGPY(2:work_periods+1, i2) = cS_pps.f_y(2:work_periods+1) ./ cS_pps.f_y(1:work_periods) .* exp(-z_shocks);
    simY_norm_work(:, i2) = exp(-u_shocks_log);
    simR(:, i2) = cS_pps.rf + cS_pps.mu - r_shocks;
end

simY_norm = zeros(tn, nsim);
simY_norm(1:work_periods, :) = simY_norm_work;
simY_norm(work_periods+1:tn, :) = cS_pps.ret_fac;

% 计算真实持久性收入 (两个模型共享)
simP_real = zeros(tn, nsim);
simP_real(1, :) = initial_income_yuan;
for t = 1:(tn - 1)
    simP_real(t+1, :) = simP_real(t, :) .* simGPY(t+1, :);
end
fprintf('  冲击路径生成完毕。\n');

% --- 3. 运行PPS模型的蒙特卡洛模拟 ---
fprintf('正在运行包含PPS模型的模拟...\n');
simW_norm_pps = zeros(tn, nsim);
simF_norm_pps = zeros(tn, nsim);
simC_norm_pps = zeros(tn, nsim);
simA_pps = zeros(tn, nsim);

for t = 1:tn
    if t <= work_periods
        norm_cash = simW_norm_pps(t, :) + (1-cS_pps.tau_y) * simY_norm(t, :);
    else
        norm_pension_payout = (1-cS_pps.tau_q) * simF_norm_pps(t, :) * cS_pps.pension_rate;
        norm_cash = simW_norm_pps(t, :) + simY_norm(t, :) + norm_pension_payout;
    end
    
    Fc_interpolant = griddedInterpolant({cS_pps.gcash, cS_pps.gfund}, vfi_results_pps.C_policy(:,:,t), 'spline', 'linear');
    Fa_interpolant = griddedInterpolant({cS_pps.gcash, cS_pps.gfund}, vfi_results_pps.A_policy(:,:,t), 'spline', 'linear');
    Fq_interpolant = griddedInterpolant({cS_pps.gcash, cS_pps.gfund}, vfi_results_pps.Q_policy(:,:,t), 'spline', 'linear');

    simC_norm = Fc_interpolant(norm_cash, simF_norm_pps(t, :));
    simA = Fa_interpolant(norm_cash, simF_norm_pps(t, :));
    simQ_norm = Fq_interpolant(norm_cash, simF_norm_pps(t, :));

    simA_pps(t, :) = max(min(simA, 1), 0);
    simC_norm = max(min(simC_norm, norm_cash), 0);
    simQ_norm = max(min(simQ_norm, cS_pps.Q_max), 0);
    
    if t <= work_periods
        total_outflow = simC_norm + (1-cS_pps.tau_y)*simQ_norm;
        idx_exceed = total_outflow > norm_cash;
        if any(idx_exceed)
            scale_factor = norm_cash(idx_exceed) ./ total_outflow(idx_exceed) * 0.9999;
            simC_norm(idx_exceed) = simC_norm(idx_exceed) .* scale_factor;
            simQ_norm(idx_exceed) = simQ_norm(idx_exceed) .* scale_factor;
        end
        liquid_sav_norm = norm_cash - simC_norm - (1-cS_pps.tau_y)*simQ_norm;
    else
        simQ_norm(:) = 0;
        simC_norm = min(simC_norm, norm_cash * 0.9999);
        liquid_sav_norm = norm_cash - simC_norm;
    end
    simC_norm_pps(t, :) = simC_norm;

    simS_norm = simA_pps(t, :) .* liquid_sav_norm;
    simB_norm = liquid_sav_norm - simS_norm;

    if t < tn
        portfolio_return_next = simB_norm * cS_pps.rf + simS_norm .* simR(t, :);
        simW_norm_pps(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);
        if t <= work_periods
            simF_norm_pps(t+1, :) = ((simF_norm_pps(t, :) + simQ_norm) * cS_pps.rp) ./ simGPY(t+1, :);
        else
            simF_norm_pps(t+1, :) = simF_norm_pps(t, :);
        end
    end
end
simW_real_pps = simW_norm_pps .* simP_real;
simF_real_pps = simF_norm_pps .* simP_real;
simC_real_pps = simC_norm_pps .* simP_real;
fprintf('  PPS模型模拟完成。\n');

% --- 4. 运行No-PPS模型的蒙特卡洛模拟 ---
fprintf('正在运行不包含PPS模型的模拟...\n');
simW_norm_nopps = zeros(tn, nsim);
simC_norm_nopps = zeros(tn, nsim);
simA_nopps = zeros(tn, nsim);

for t = 1:tn
    if t <= work_periods
        norm_cash = simW_norm_nopps(t, :) + (1-cS_nopps.tau_y) * simY_norm(t, :);
    else
        norm_cash = simW_norm_nopps(t, :) + simY_norm(t, :);
    end
    
    Fc_interpolant = griddedInterpolant(cS_nopps.gcash, vfi_results_nopps.C_policy(:,t), 'spline', 'linear');
    Fa_interpolant = griddedInterpolant(cS_nopps.gcash, vfi_results_nopps.A_policy(:,t), 'spline', 'linear');

    simC_norm = Fc_interpolant(norm_cash);
    simA = Fa_interpolant(norm_cash);
    
    simA_nopps(t, :) = max(min(simA, 1), 0);
    simC_norm = max(min(simC_norm, norm_cash), 0);
    simC_norm_nopps(t, :) = simC_norm;

    liquid_sav_norm = norm_cash - simC_norm;
    
    simS_norm = simA_nopps(t, :) .* liquid_sav_norm;
    simB_norm = liquid_sav_norm - simS_norm;

    if t < tn
        portfolio_return_next = simB_norm * cS_nopps.rf + simS_norm .* simR(t, :);
        simW_norm_nopps(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);
    end
end
simW_real_nopps = simW_norm_nopps .* simP_real;
simC_real_nopps = simC_norm_nopps .* simP_real;
fprintf('  No-PPS模型模拟完成。\n');

% --- 5. 计算绘图所需变量 (仅中位数) ---
fprintf('正在计算中位数...\n');
% (a) 财富
med_TotalW_pps = median(simW_real_pps + simF_real_pps, 2) * yuan_to_wanyuan;
med_LiquidW_pps = median(simW_real_pps, 2) * yuan_to_wanyuan;
med_TotalW_nopps = median(simW_real_nopps, 2) * yuan_to_wanyuan;

% (b) 风险决策 Alpha
med_A_pps = median(simA_pps, 2);
med_A_nopps = median(simA_nopps, 2);

% (c) 消费
med_C_pps = median(simC_real_pps, 2) * yuan_to_wanyuan;
med_C_nopps = median(simC_real_nopps, 2) * yuan_to_wanyuan;

% *** 新增: 计算子图(c)所需的超额消费比例 ***
sim_C_excess_ratio = (simC_real_pps ./ simC_real_nopps - 1) * 100; % 转换为百分比
sim_C_excess_ratio(isinf(sim_C_excess_ratio) | isnan(sim_C_excess_ratio)) = NaN; % 处理除以0的情况

% --- 6. 创建并保存图像 ---
fprintf('正在生成图像...\n');
fig = figure('Name', 'Comparison: With vs Without PPS');
figure_width_cm = 14.99;
figure_height_cm = 4.5;
set(fig, 'Units', 'centimeters', 'Position', [5, 5, figure_width_cm, figure_height_cm]);
pause(0.1); 
retirement_age_line = cS_pps.tr - 1;

% --- 灰度颜色和样式定义 (修改点) ---
line_color = [0.5 0.5 0.5];
color_pps = [0.2 0.2 0.2];      % 深灰色 (替换蓝色)
color_nopps = [0.6 0.6 0.6];    % 浅灰色 (替换红色)
style_pps = '-';                % 实线
style_nopps = '--';             % 虚线

% --- 子图 (a): 财富路径 ---
ax1 = subplot(1, 3, 1);
hold(ax1, 'on');
plot(ax1, ages, med_TotalW_pps, 'Color', color_pps, 'LineStyle', style_pps, 'LineWidth', 1.5, 'HandleVisibility', 'off');
plot(ax1, ages, med_LiquidW_pps, 'Color', color_pps, 'LineStyle', ':', 'LineWidth', 1.5, 'DisplayName', '流动性财富 (含PPS)');
plot(ax1, ages, med_TotalW_nopps, 'Color', color_nopps, 'LineStyle', style_nopps, 'LineWidth', 1.5, 'HandleVisibility', 'off');
xline(ax1, retirement_age_line, '--', 'LineWidth', 1, 'Color', line_color, 'HandleVisibility', 'off');
hold(ax1, 'off');
grid(ax1, 'on');
title(ax1, '(a) 总财富路径');
% xlabel(ax1, '年龄');
ylabel(ax1, '人民币 (万元)');
xlim(ax1, [cS_pps.tb, cS_pps.td]);
z1 = legend(ax1, 'show','box','off', 'FontSize', 7,'Position', [0.1362 0.0070 0.1911 0.1000]);
z1.ItemTokenSize(1) = 10;
set(ax1, 'FontSize', 8);

% --- 子图 (b): 风险决策 ---
ax2 = subplot(1, 3, 2);
hold(ax2, 'on');
plot(ax2, ages, med_A_pps, 'Color', color_pps, 'LineStyle', style_pps, 'LineWidth', 1.5,'DisplayName','PPS');
plot(ax2, ages, med_A_nopps, 'Color', color_nopps, 'LineStyle', style_nopps, 'LineWidth', 1.5,'DisplayName','no-PPS');
xline(ax2, retirement_age_line, '--', 'LineWidth', 1, 'Color', line_color, 'HandleVisibility', 'off');

hold(ax2, 'off');
grid(ax2, 'on');
title(ax2, '(b) 风险决策');
xlabel(ax2, '年龄');
xlim(ax2, [cS_pps.tb, cS_pps.td]);
ylim(ax2, [0, 1.05]);
z2 = legend(ax2, 'show','box','off','Location','north', 'FontSize', 7);
z2.ItemTokenSize(1) = 15;
set(ax2, 'FontSize', 8);

% --- 子图 (c): 消费路径 ---
ax3 = subplot(1, 3, 3);
hold(ax3, 'on');

% 计算代表+-1个标准差的分位数 (16% 和 84%)
p20_C_excess = prctile(sim_C_excess_ratio, 20, 2);
p80_C_excess = prctile(sim_C_excess_ratio, 80, 2);
med_C_excess = nanmedian(sim_C_excess_ratio, 2);

% 填充分位区间
fill_color = [0.5 0.5 0.5]; % Gray color for the band
x_fill = [ages; flipud(ages)];
fill(ax3, x_fill, [p20_C_excess; flipud(p80_C_excess)], fill_color, 'FaceAlpha', 0.4, 'EdgeColor', 'none', 'DisplayName', '20-80分位区间');

% 绘制中位数线
plot(ax3, ages, med_C_excess, 'k-', 'LineWidth', 1.5, 'HandleVisibility', 'off');

% 添加0%参考线
yline(ax3, 0, '--', 'Color', [0.3 0.3 0.3], 'HandleVisibility', 'off');

% 添加退休年龄竖线
xline(ax3, retirement_age_line, '--', {'退休'}, 'LineWidth', 1, 'Color', line_color, 'HandleVisibility', 'off');

hold(ax3, 'off');
grid(ax3, 'on');
title(ax3, '(c) PPS模型超额消费');
ylabel(ax3, '超额消费 (%)');
xlim(ax3, [cS_pps.tb, cS_pps.td]);
z3 = legend(ax3, 'show','box','off','position',[0.695699201761592,0.009402064808382,0.199294529369058,0.099999997720999], 'FontSize', 7);
z3.ItemTokenSize(1) = 10;
set(ax3, 'FontSize', 8);

% --- 7. 保存图像 ---
output_dir = 'tex/fig/ch04';
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('创建目录: %s\n', output_dir);
end
output_filename = fullfile(output_dir, 'fig4.7_pps_vs_nopps_comparison.png');
print(fig, output_filename, '-dpng', '-r400');
fprintf('图像已成功保存至: %s\n', output_filename);

### 表：养老金缴费上限的福利损失

In [ ]:
% =========================================================================
% == SCRIPT: create_qmax_welfare_table_by_age_v3.m
% == 版本: [v3.0 - [Definitive] 分年龄段福利成本表格生成器]
% ==
% == 目的:
% ==   1. 修正v2版本中因错误地去归一化导致的逻辑问题。
% ==   2. 在模型求解所在的归一化空间内，进行一致且可靠的福利比较。
% ==   3. 采用标准的效用-CV转换公式，计算分年龄段的福利损失。
% ==   4. 生成最终的、结果符合经济学直觉的顶刊级别LaTeX表格。
% ==
% == 前置条件:
% ==   - 已成功运行所有相关的main脚本并生成所有vfi_results_*.mat文件。
% =========================================================================

clear; close all; clc;
fprintf('=== [Definitive] 分年龄段福利成本表格生成脚本 (v3.0) ===\n\n');

%% --- 1. 用户设定 ---
fprintf('--- 1. 用户设定与文件定义 ---\n');

% --- [核心] 定义要分析的模型设定 (基准模型放在第一个) ---
model_configs = {
    inf,   '',           '无上限';
    36000, 'qmax_36000', '36,000';
    24000, 'qmax_24000', '24,000';
    12000, 'qmax_12000', '12,000';
    6000,  'qmax_6000',  '6,000';
    3000,  'qmax_3000',  '3,000';
};

% --- [核心] 定义要评估的年龄点 ---
assessment_ages = {'终身', 30, 40, 50, 60, 70, 80}; 

% --- 输入/输出路径 ---
input_dir = 'result/';
output_dir = 'tex/tab/ch04/';
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
output_filename = fullfile(output_dir, 'tab4.1_welfare_cost_by_age_final.tex');


%% --- 2. [计算核心] 循环计算每个模型的、分年龄的期望归一化价值 ---
fprintf('\n--- 2. 正在计算分年龄期望归一化价值 E[V_hat] ---\n');

num_models = size(model_configs, 1);
num_ages = length(assessment_ages);
ev_hat_matrix = zeros(num_models, num_ages);

% 加载一个样本cS以获取年龄信息
temp_data = load(fullfile(input_dir, 'vfi_results.mat'), 'cS');
cS_sample = temp_data.cS;
age_indices = zeros(1, num_ages);
for i = 1:num_ages
    if ischar(assessment_ages{i}) && strcmpi(assessment_ages{i}, '终身')
        age_indices(i) = 1;
    else
        age_indices(i) = assessment_ages{i} - cS_sample.tb + 1;
    end
end

for i = 1:num_models
    config = model_configs(i, :);
    label = config{3};
    suffix = config{2};
    
    % 构建文件名并加载数据
    if isempty(suffix), file_name = 'vfi_results.mat'; else, file_name = ['vfi_results_', suffix, '.mat']; end
    file_path = fullfile(input_dir, file_name);
    fprintf('   正在处理模型: %s\n', label);
    if ~exist(file_path, 'file'), error('找不到输入文件: %s', file_path); end
    data = load(file_path, 'vfi_results', 'cS');
    vfi_results = data.vfi_results;
    cS = data.cS;
    
    % --- a. 运行模拟以获得状态分布 ---
    sim_data = get_simulation_paths(vfi_results, cS);
    
    % --- b. 遍历评估年龄，计算 E[V_hat_t] ---
    for j = 1:num_ages
        t = age_indices(j);
        
        W_hat_dist = sim_data.simW_norm(t, :);
        
        has_pps = isfield(cS, 'rp');
        
        if has_pps
            F_hat_dist = sim_data.simF_norm(t, :);
            V_hat_t_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, vfi_results.V(:,:,t), 'spline', 'linear');
            V_hat_t_dist = V_hat_t_interpolant(W_hat_dist, F_hat_dist);
        else
            V_hat_t_interpolant = griddedInterpolant(cS.gcash, vfi_results.V(:,t), 'spline', 'linear');
            V_hat_t_dist = V_hat_t_interpolant(W_hat_dist')';
        end
        
        ev_hat_matrix(i, j) = mean(V_hat_t_dist, 'omitnan');
    end
end
fprintf('   ✅ 所有模型的期望归一化价值计算完成。\n');

%% --- 3. 计算福利损失(CV) ---
fprintf('\n--- 3. 正在计算福利损失 (CV) ---\n');
% 基准模型的 E[V_hat] 在第一行
ev_hat_benchmark_row = ev_hat_matrix(1, :);
gamma = cS_sample.gamma; % 从样本cS中获取风险规避系数

% CV = (E[V_hat_constrained] / E[V_hat_benchmark])^(1 / (1 - gamma)) - 1
cv_matrix = (ev_hat_matrix ./ ev_hat_benchmark_row).^(1 / (1 - gamma)) - 1;

fprintf('   ✅ 福利损失计算完成。\n');

%% --- 4. [核心] 生成 LaTeX 表格 ---
fprintf('\n--- 4. 生成 LaTeX 格式的福利成本表格 ---\n');

fileID = fopen(output_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开LaTeX输出文件。'); end

fprintf(fileID, '%% =============================================================\n');
fprintf(fileID, '%%  此文件由 create_qmax_welfare_table_by_age_v3.m (v3.0) 自动生成\n');
fprintf(fileID, '%% =============================================================\n');
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{不同缴费上限下，各年龄段的期望剩余终身福利损失}\n');
fprintf(fileID, '\\label{tab:welfare_cost_by_age}\n');
fprintf(fileID, '\\begin{threeparttable}\n');

col_def = ['l', repmat('c', 1, num_ages)];
fprintf(fileID, '\\begin{tabular}{%s}\n', col_def);
fprintf(fileID, '\\toprule\n');

header_str = '年缴费上限 (元)';
for j = 1:num_ages
    header_str = [header_str, sprintf(' & %s岁', num2str(assessment_ages{j}))];
end
header_str = strrep(header_str, '终身岁', '终身');
fprintf(fileID, '%s \\\\\n', header_str);
fprintf(fileID, '\\midrule\n');

for i = 1:num_models
    label_str = model_configs{i, 3};
    row_str = label_str;
    for j = 1:num_ages
        if i == 1
            cell_str = '-\textsuperscript{a}';
        else
            cv_val = cv_matrix(i, j);
            cell_str = sprintf('%.2f\\%%', cv_val * 100);
        end
        row_str = [row_str, ' & ', cell_str];
    end
    fprintf(fileID, '%s \\\\\n', row_str);
    if i > 1 && model_configs{i,1} == 0
         fprintf(fileID, '\\addlinespace\n');
    end
end

fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\footnotesize\n');
fprintf(fileID, '  \\item[注] 表中数值为补偿变化(CV)，衡量在不同年龄点，个体因面临缴费上限（或无PPS制度）而产生的期望剩余终身福利损失，基准情形为无缴费上限的制度。');
fprintf(fileID, '例如，在30岁时，-1.50\\%% 表示一个30岁的普通个体愿意从此刻到生命终点，永久性地放弃其年消费的1.50\\%%，以换取进入一个无缴费上限的养老金制度。\n');
fprintf(fileID, '  \\item[\\textsuperscript{a}] 无缴费上限的情形被设定为计算福利成本的基准。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');

fclose(fileID);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_filename);
fprintf('\n--- 脚本执行完毕 ---\n');


%% =====================================================================
%                       [本地辅助函数] - (与v2版本一致，无需修改)
% =====================================================================
function sim_data = get_simulation_paths(results, cS)
    % 描述:
    %   本函数严格复制 main_PPS.m 和 temp.m 中的模拟逻辑，以确保一致性。
    %   它不进行任何绘图或分析，仅返回包含所有模拟路径的结构体。
    
    % --- 初始化 ---
    rng(42); 
    nsim = 10000;
    
    C_policy = results.C_policy;
    A_policy = results.A_policy;
    
    simGPY = ones(cS.tn, nsim); simY_norm = zeros(cS.tn, nsim); simR = zeros(cS.tn, nsim);
    simW_norm = zeros(cS.tn, nsim); simC_norm = zeros(cS.tn, nsim);
    simA = zeros(cS.tn, nsim);
    
    has_pps = isfield(cS, 'rp');
    if has_pps
        Q_policy = results.Q_policy;
        simF_norm = zeros(cS.tn, nsim);
        simQ_norm = zeros(cS.tn, nsim);
    end
    
    % --- 1. 模拟外生冲击和收入过程 ---
    work_periods = cS.tr - cS.tb;
    for i1 = 1:floor(nsim/2)
        z_shocks = randn(work_periods, 1) * cS.smav;
        u_shocks_log = randn(work_periods, 1) * cS.smay;
        r_shocks = randn(cS.tn, 1) * cS.sigr;
        simGPY(2:work_periods+1, i1) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(z_shocks);
        simY_norm(1:work_periods, i1) = exp(u_shocks_log);
        simR(:, i1) = cS.rf + cS.mu + r_shocks;
        i2 = nsim/2 + i1;
        simGPY(2:work_periods+1, i2) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(-z_shocks);
        simY_norm(1:work_periods, i2) = exp(-u_shocks_log);
        simR(:, i2) = cS.rf + cS.mu - r_shocks;
    end
    simY_norm(work_periods+1:cS.tn, :) = cS.ret_fac;

    % --- 2. 迭代模拟生命周期决策 (归一化单位) ---
    for t = 1:cS.tn
        if has_pps
            current_F = simF_norm(t, :);
            if t <= work_periods 
                norm_cash = simW_norm(t, :) + (1-cS.tau_y) * simY_norm(t, :);
            else 
                norm_pension_payout = (1-cS.tau_q) * current_F * cS.pension_rate;
                norm_cash = simW_norm(t, :) + simY_norm(t, :) + norm_pension_payout;
            end
            
            Fc_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, C_policy(:,:,t), 'spline', 'linear');
            Fa_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, A_policy(:,:,t), 'spline', 'linear');
            Fq_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, Q_policy(:,:,t), 'spline', 'linear');
            simC_norm(t, :) = Fc_interpolant(norm_cash, current_F);
            simA(t, :) = Fa_interpolant(norm_cash, current_F);
            simQ_norm(t, :) = Fq_interpolant(norm_cash, current_F);
        else % No PPS
            norm_cash = simW_norm(t, :) + (1-cS.tau_y) * simY_norm(t, :);
            if t > work_periods, norm_cash = simW_norm(t, :) + simY_norm(t, :); end
            Fc_interpolant = griddedInterpolant(cS.gcash, C_policy(:,t), 'spline', 'linear');
            Fa_interpolant = griddedInterpolant(cS.gcash, A_policy(:,t), 'spline', 'linear');
            simC_norm(t, :) = Fc_interpolant(norm_cash')';
            simA(t, :) = Fa_interpolant(norm_cash')';
        end

        simA(t, :) = max(min(simA(t, :), 1), 0);
        simC_norm(t, :) = max(min(simC_norm(t, :), norm_cash), 0);
        
        liquid_sav_norm = 0;
        if has_pps
            simQ_norm(t, :) = max(min(simQ_norm(t, :), cS.Q_max), 0);
            if t <= work_periods
                total_outflow = simC_norm(t, :) + (1-cS.tau_y)*simQ_norm(t, :);
                idx_exceed = total_outflow > norm_cash;
                if any(idx_exceed)
                    scale_factor = norm_cash(idx_exceed) ./ total_outflow(idx_exceed) * 0.9999;
                    simC_norm(t, idx_exceed) = simC_norm(t, idx_exceed) .* scale_factor;
                    simQ_norm(t, idx_exceed) = simQ_norm(t, idx_exceed) .* scale_factor;
                end
                liquid_sav_norm = norm_cash - simC_norm(t, :) - (1-cS.tau_y)*simQ_norm(t, :);
            else 
                simQ_norm(t,:) = 0;
                simC_norm(t,:) = min(simC_norm(t,:), norm_cash * 0.9999);
                liquid_sav_norm = norm_cash - simC_norm(t, :);
            end
        else % No PPS
            liquid_sav_norm = norm_cash - simC_norm(t, :);
        end
        
        if t < cS.tn
            simS_norm = simA(t, :) .* liquid_sav_norm;
            simB_norm = liquid_sav_norm - simS_norm;
            portfolio_return_next = simB_norm * cS.rf + simS_norm .* simR(t, :);
            simW_norm(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);
            if has_pps
                if t <= work_periods
                    simF_norm(t+1, :) = ((current_F + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
                else
                    simF_norm(t+1, :) = current_F;
                end
            end
        end
    end
    
    % --- 3. (仅用于v2版)转换为真实人民币单位 ---
    % 在v3版中不再需要，但保留以备查
    initial_income_yuan = 60000;
    simP_real = zeros(cS.tn, nsim);
    simP_real(1, :) = initial_income_yuan;
    for t = 1:(cS.tn - 1)
        simP_real(t+1, :) = simP_real(t, :) .* simGPY(t+1, :);
    end
    
    % --- 4. 封装结果 ---
    sim_data.simW_norm = simW_norm;
    sim_data.simP_real = simP_real; % 虽然v3不直接用，但保留无妨
    if has_pps
        sim_data.simF_norm = simF_norm;
    else
        sim_data.simF_norm = zeros(cS.tn, nsim);
    end
end

### 图4.8 风险性的PPS收益 

In [ ]:
%% 比较不同风险水平PPS的生命周期模拟路径
%
% 描述:
%   本脚本加载一系列具有不同风险水平(sigrrp)的个人养老金账户(PPS)
%   模型的VFI求解结果，进行蒙特卡洛模拟，并生成 1x3 对比图。
%
%   (a) Q_t/W_t: 养老金缴费占流动性财富比率
%   (b) alpha_t: 风险资产配置比例
%   (c) 超额消费: 以 sigrrp=0.27 为基准的超额消费百分比
%
% 前提:
%   - 已运行脚本生成一系列 vfi_results_riskyrp_*.mat 文件并存放于
%     'result/riskyrp/' 目录下。
%   - 其中必须包含基准文件 vfi_results_riskyrp_sigrrp_0p2700.mat。
%
% 输出:
%   - tex/fig/ch04/fig_riskyrp_comparison.png: PNG格式图，400 DPI, 宽度严格为 14.99cm

% --- 初始化环境 ---
clear;
close all;
clc;
fprintf('开始生成风险PPS模型对比图...\n');

% --- 1. 发现并加载所有相关的结果文件 ---
data_dir = fullfile('result', 'riskyrp');
if ~exist(data_dir, 'dir')
    error('目录 %s 不存在，请先运行数据生成脚本。', data_dir);
end
file_pattern = 'vfi_results_riskyrp_sigrrp_*.mat';
files = dir(fullfile(data_dir, file_pattern));
if isempty(files)
    error('在 %s 中未找到符合 %s 格式的文件。', data_dir, file_pattern);
end

num_scenarios = length(files);
results_data = cell(num_scenarios, 1);
fprintf('发现 %d 个情景文件，正在加载...\n', num_scenarios);

for i = 1:num_scenarios
    filename = files(i).name;
    % 从文件名解析 sigrrp 的值
    token = regexp(filename, 'sigrrp_(\d+p\d+)\.mat', 'tokens');
    if isempty(token)
        warning('无法从文件名 %s 中解析sigrrp值，跳过此文件。', filename);
        continue;
    end
    sigrrp_val = str2double(strrep(token{1}{1}, 'p', '.'));
    
    data = load(fullfile(data_dir, filename));
    results_data{i} = struct('vfi', data.vfi_results, 'cS', data.cS, 'sigrrp', sigrrp_val);
    fprintf('  [成功] 已加载 %s (sigrrp = %.4f)\n', filename, sigrrp_val);
end

% --- 2. 模拟设置 ---
rng(42);
nsim = 10000;
cS_base = results_data{1}.cS; % 使用第一个文件的参数作为基础
tb = cS_base.tb;
tr = cS_base.tr;
td = cS_base.td;
work_periods = cS_base.tr - cS_base.tb;
tn = cS_base.tn;
initial_income_yuan = 60000;
yuan_to_wanyuan = 1 / 10000;
ages = (cS_base.tb:cS_base.td)';

% --- 2.1 模拟外生冲击 (所有情景共享) ---
fprintf('正在生成通用的外生冲击路径...\n');
simGPY = ones(tn, nsim);
simY_norm = zeros(tn, nsim);
simR = zeros(tn, nsim);
% 生成基础的标准正态冲击，用于构建相关的风险资产回报
norm_shocks = randn(tn, nsim); 

for i1 = 1:floor(nsim/2)
    z_shocks = randn(work_periods, 1) * cS_base.smav;
    u_shocks_log = randn(work_periods, 1) * cS_base.smay;
    i2 = nsim/2 + i1;
    
    simGPY(2:work_periods+1, i1) = cS_base.f_y(2:work_periods+1) ./ cS_base.f_y(1:work_periods) .* exp(z_shocks);
    simGPY(2:work_periods+1, i2) = cS_base.f_y(2:work_periods+1) ./ cS_base.f_y(1:work_periods) .* exp(-z_shocks);
    simY_norm(1:work_periods, i1) = exp(u_shocks_log);
    simY_norm(1:work_periods, i2) = exp(-u_shocks_log);
end
simY_norm(work_periods+1:tn, :) = cS_base.ret_fac;
simR = cS_base.rf + cS_base.mu + norm_shocks * cS_base.sigr;

% 计算真实持久性收入
simP_real = zeros(tn, nsim);
simP_real(1, :) = initial_income_yuan;
for t = 1:(tn - 1)
    simP_real(t+1, :) = simP_real(t, :) .* simGPY(t+1, :);
end

% --- 3. 为每个情景运行蒙特卡洛模拟 ---
all_sim_W_real = cell(num_scenarios, 1);
all_sim_C_real = cell(num_scenarios, 1);
all_sim_Q_real = cell(num_scenarios, 1);
all_sim_A = cell(num_scenarios, 1);

for i = 1:num_scenarios
    cS_current = results_data{i}.cS;
    vfi_current = results_data{i}.vfi;
    fprintf('正在为 sigrrp = %.4f 运行模拟...\n', cS_current.sigrrp);

    % 生成与 simR 相关的 PPS 回报率
    simRp = cS_current.rp + norm_shocks * cS_current.sigrrp;

    simW_norm = zeros(tn, nsim);
    simF_norm = zeros(tn, nsim);
    simC_norm = zeros(tn, nsim);
    simQ_norm_path = zeros(tn, nsim);
    simA = zeros(tn, nsim);

    for t = 1:tn
        if t <= work_periods
            norm_cash = simW_norm(t, :) + (1-cS_current.tau_y) * simY_norm(t, :);
        else
            norm_pension_payout = (1-cS_current.tau_q) * simF_norm(t, :) * cS_current.pension_rate;
            norm_cash = simW_norm(t, :) + simY_norm(t, :) + norm_pension_payout;
        end
        
        Fc_interpolant = griddedInterpolant({cS_current.gcash, cS_current.gfund}, vfi_current.C_policy(:,:,t), 'spline', 'linear');
        Fa_interpolant = griddedInterpolant({cS_current.gcash, cS_current.gfund}, vfi_current.A_policy(:,:,t), 'spline', 'linear');
        Fq_interpolant = griddedInterpolant({cS_current.gcash, cS_current.gfund}, vfi_current.Q_policy(:,:,t), 'spline', 'linear');
        
        simC_t = Fc_interpolant(norm_cash, simF_norm(t, :));
        simA_t = Fa_interpolant(norm_cash, simF_norm(t, :));
        simQ_t = Fq_interpolant(norm_cash, simF_norm(t, :));

        simA(t, :) = max(min(simA_t, 1), 0);
        simC_t = max(min(simC_t, norm_cash), 0);
        simQ_t = max(min(simQ_t, cS_current.Q_max), 0);
        
        if t <= work_periods
            total_outflow = simC_t + (1-cS_current.tau_y)*simQ_t;
            idx_exceed = total_outflow > norm_cash;
            if any(idx_exceed)
                scale_factor = norm_cash(idx_exceed) ./ total_outflow(idx_exceed) * 0.9999;
                simC_t(idx_exceed) = simC_t(idx_exceed) .* scale_factor;
                simQ_t(idx_exceed) = simQ_t(idx_exceed) .* scale_factor;
            end
            liquid_sav_norm = norm_cash - simC_t - (1-cS_current.tau_y)*simQ_t;
        else
            simQ_t(:) = 0;
            simC_t = min(simC_t, norm_cash * 0.9999);
            liquid_sav_norm = norm_cash - simC_t;
        end
        simC_norm(t, :) = simC_t;
        simQ_norm_path(t, :) = simQ_t;

        simS_norm = simA(t, :) .* liquid_sav_norm;
        simB_norm = liquid_sav_norm - simS_norm;

        if t < tn
            portfolio_return_next = simB_norm * cS_current.rf + simS_norm .* simR(t, :);
            simW_norm(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);
            if t <= work_periods
                simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_t) .* simRp(t,:)) ./ simGPY(t+1, :);
            else
                simF_norm(t+1, :) = simF_norm(t, :);
            end
        end
    end
    
    all_sim_W_real{i} = simW_norm .* simP_real;
    all_sim_C_real{i} = simC_norm .* simP_real;
    all_sim_Q_real{i} = simQ_norm_path .* simP_real;
    all_sim_A{i} = simA;
end

% --- 4. 计算绘图所需变量 ---
fprintf('正在计算绘图变量...\n');
% 找到基准情景 (sigrrp = 0.27) 的索引
benchmark_sigrrp = 0;
benchmark_idx = find(cellfun(@(x) x.sigrrp, results_data) == benchmark_sigrrp);
if isempty(benchmark_idx)
    error('未找到基准情景 sigrrp = %.4f 的结果文件。', benchmark_sigrrp);
end
C_real_benchmark = all_sim_C_real{benchmark_idx};

plot_data = cell(num_scenarios, 1);
for i = 1:num_scenarios
    sim_W_real = all_sim_W_real{i};
    sim_Q_real = all_sim_Q_real{i};
    sim_A_path = all_sim_A{i};
    sim_C_real = all_sim_C_real{i};

    % (a) Q/W
    sim_QW_ratio = sim_Q_real ./ sim_W_real;
    sim_QW_ratio(isinf(sim_QW_ratio) | isnan(sim_QW_ratio) | sim_W_real < 1e-4) = NaN;
    med_QW_ratio = nanmedian(sim_QW_ratio, 2);
    
    % (b) Alpha
    med_A = nanmedian(sim_A_path, 2);
    
    % (c) 超额消费
    sim_C_excess_ratio = (C_real_benchmark ./ sim_C_real - 1) * 100;
    sim_C_excess_ratio(isinf(sim_C_excess_ratio) | isnan(sim_C_excess_ratio)) = NaN;
    med_C_excess = nanmedian(sim_C_excess_ratio, 2);
    
    plot_data{i} = struct('med_QW_ratio', med_QW_ratio, 'med_A', med_A, 'med_C_excess', med_C_excess);
end

% --- 5. 创建并保存图像 ---
fprintf('正在生成图像...\n');
fig = figure('Name', 'Risky PPS Comparison');

figure_width_cm = 14.99;
figure_height_cm = 4.5;
set(fig, 'Units', 'centimeters', 'Position', [5, 5, figure_width_cm, figure_height_cm]);
pause(0.1); 
retirement_age_line = cS_base.tr - 1;
line_color = [0.5 0.5 0.5];
% colors = lines(num_scenarios); % 彩色
colors = [0.4,0.4,0.4;
    0.7,0.7,0.7;
    0.1,0.1,0.1];

% --- 子图 (a): Q/W ---
ax1 = subplot(1, 3, 1);
hold(ax1, 'on');
for i = 1:num_scenarios
    sigrrp_val = results_data{i}.sigrrp;
    if i == benchmark_idx
    plot(ax1, ages(1:tr-tb+1), plot_data{i}.med_QW_ratio(1:tr-tb+1), 'Color', colors(i,:), 'LineWidth', 1.3, ...
        'DisplayName', sprintf('$\\sigma_{rp} = %.2f$ (Baseline)', sigrrp_val));
    else
    plot(ax1, ages(1:tr-tb+1), plot_data{i}.med_QW_ratio(1:tr-tb+1), 'Color', colors(i,:), 'LineWidth', 1.3, ...
        'DisplayName', sprintf('$\\sigma_{rp} = %.2f$', sigrrp_val));
    end
end
hold(ax1, 'off');
grid(ax1, 'on');
title(ax1, '(a) $Q_t/W_t$', 'Interpreter', 'latex');
xlabel(ax1, '年龄');
xlim(ax1, [cS_base.tb, cS_base.tr]);
ylim(ax1, [-0.1, 0.8]); % 设定一个合理的Y轴范围
z1=legend(ax1, 'show','box','off','Location','northwest', 'FontSize', 7, 'Interpreter', 'latex');
z1.ItemTokenSize(1) = 15;% 调整图例位置到整个图的右侧外部
set(ax1, 'FontSize', 8);

% --- 子图 (b): Alpha ---
ax2 = subplot(1, 3, 2);
hold(ax2, 'on');
for i = 1:num_scenarios
    sigrrp_val = results_data{i}.sigrrp;
    plot(ax2, ages, plot_data{i}.med_A, 'Color', colors(i,:), 'LineWidth', 1.5, ...
        'DisplayName', sprintf('$\\sigma_{rp} = %.2f$', sigrrp_val));
end
xline(ax2, retirement_age_line, '--', 'LineWidth', 1, 'Color', line_color, 'HandleVisibility', 'off', 'Interpreter', 'latex'); % {'退休'}
hold(ax2, 'off');
grid(ax2, 'on');
title(ax2, '(b) $\alpha_t$', 'Interpreter', 'latex');
xlabel(ax2, '年龄');
xlim(ax2, [cS_base.tb, cS_base.td]);
ylim(ax2, [0, 1.05]);
z2=legend(ax2, 'show','box','off','Location','best', 'FontSize', 7, 'Interpreter', 'latex');
z2.ItemTokenSize(1) = 15;% 调整图例位置到整个图的右侧外部
set(ax2, 'FontSize', 8);

% --- 子图 (c): 超额消费 ---
ax3 = subplot(1, 3, 3);
hold(ax3, 'on');
for i = 1:num_scenarios
    % 不绘制基准线自身 (超额为0)
    if i == benchmark_idx, continue; end
    sigrrp_val = results_data{i}.sigrrp;
    plot(ax3, ages, plot_data{i}.med_C_excess, 'Color', colors(i,:), 'LineWidth', 1.5, ...
        'DisplayName', sprintf('$\\sigma_{rp} = %.2f$', sigrrp_val));
end
yline(ax3, 0, '--', 'Color', [0.3 0.3 0.3], 'HandleVisibility', 'off');
xline(ax3, retirement_age_line, '--', 'LineWidth', 1, 'Color', line_color, 'HandleVisibility', 'off', 'Interpreter', 'latex');
hold(ax3, 'off');
grid(ax3, 'on');
title(ax3, '(c) 基准超额消费 (%)');
xlabel(ax3, '年龄');
xlim(ax3, [cS_base.tb, cS_base.td]);
z3=legend(ax3, 'show','box','off','Location','north', 'FontSize', 7, 'Interpreter', 'latex');
z3.ItemTokenSize(1) = 15;% 调整图例位置到整个图的右侧外部
set(ax3, 'FontSize', 8);



% --- 6. 保存图像 ---
output_dir = 'tex/fig/ch04';
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('创建目录: %s\n', output_dir);
end
output_filename = fullfile(output_dir, 'fig4.8_riskyrp_comparison.png');
print(fig, output_filename, '-dpng', '-r400');
fprintf('图像已成功保存至: %s\n', output_filename);

### 图x5 敏感性分析

In [ ]:
%% 敏感性分析结果可视化脚本
%
% 描述:
%   本脚本旨在自动读取所有已生成的敏感性分析结果(.mat文件)，
%   为每种参数类别生成一组具有经济学解释力的对比图表，并严格按照
%   博士论文的排版要求进行格式化。
%
% 流程:
%   1. 定义要进行可视化分析的参数列表。
%   2. 对每个参数，自动在 'result/sensitivity/' 目录中查找相关结果文件。
%   3. 加载基准模型和各敏感性模型的结果，并为每个模型重新运行一次
%      标准化的蒙特卡洛模拟以获取完整的生命周期路径数据。
%   4. 调用为该参数定制的绘图函数，选择最相关的指标进行对比展示。
%   5. 将生成的图像以高分辨率、指定尺寸保存到 'tex/fig/ch04/sensitivity/' 目录下。
%
% 前置条件:
%   - result/vfi_results.mat (基准模型) 文件已存在。
%   - result/sensitivity/ 文件夹下所有敏感性分析的 .mat 文件已生成。

% --- 初始化环境 ---
clear;
close all;
clc;
fprintf('===== 敏感性分析结果可视化脚本启动 =====\n\n');

%% --- 1. 全局配置 ---
% 定义要分析的参数及其绘图配置
% 'param_name': 参数在cS结构体中的字段名 (用于文件查找和图例)
% 'plot_title': 生成图表的总标题
% 'plot_func':  用于绘制该参数结果的函数句柄
param_configs = {
    struct('param_name', 'gamma', 'plot_func', @plot_gamma_sensitivity);
    struct('param_name', 'smav', 'plot_func', @plot_smav_sensitivity);
    struct('param_name', 'sigr', 'plot_func', @plot_sigr_sensitivity);
    struct('param_name', 'rp', 'plot_func', @plot_rp_sensitivity);
    struct('param_name', 'tr', 'plot_func', @plot_tr_sensitivity);
};

% --- 路径和绘图常量 ---
BASE_RESULT_FILE = 'result/vfi_results.mat'; % 已修正路径
SENSITIVITY_DIR = 'result/sensitivity/';
OUTPUT_DIR = 'tex/fig/ch04/sensitivity/';
FIGURE_WIDTH_CM = 14.99; % 严格遵循的宽度

if ~exist(OUTPUT_DIR, 'dir')
    mkdir(OUTPUT_DIR);
end

%% --- 2. 主循环：为每个参数生成图表 ---
for i = 1:length(param_configs)
    config = param_configs{i};
    param_name = config.param_name;
    
    fprintf('--- 开始处理参数: %s ---\n', upper(param_name));
    
    % --- a. 发现相关文件 ---
    fprintf('  1. 正在查找相关的结果文件...\n');
    search_pattern = sprintf('vfi_results_%s_*.mat', param_name);
    sensitivity_files = dir(fullfile(SENSITIVITY_DIR, search_pattern));
    
    if isempty(sensitivity_files)
        warning('未找到参数 %s 的任何敏感性分析文件，跳过此参数。', param_name);
        continue;
    end
    
    file_list = {BASE_RESULT_FILE};
    for j = 1:length(sensitivity_files)
        file_list{end+1} = fullfile(SENSITIVITY_DIR, sensitivity_files(j).name);
    end
    
    % --- b. 加载数据并运行模拟 ---
    fprintf('  2. 正在为 %d 个模型加载数据并运行模拟...\n', length(file_list));
    all_results = [];
    values = [];
    
    for j = 1:length(file_list)
        file_path = file_list{j};
        
        % 从文件名解析参数值
        if j == 1 % 基准模型
            temp_data = load(file_path, 'cS');
            current_value = getfield(temp_data.cS, param_name);
            label = sprintf('Baseline (%.3g)', current_value);
        else
            [~, fname, ~] = fileparts(file_path);
            token = regexp(fname, [param_name, '_([\d_p]+)'], 'tokens');
            value_str = token{1}{1};
            current_value = str2double(strrep(value_str, 'p', '.'));
            label = sprintf('%.3g', current_value);
        end
        
        fprintf('     - 模拟: %s = %s\n', param_name, label);
        
        % 运行模拟并获取关键指标
        metrics = run_simulation_for_file(file_path);
        
        % 存储结果
        values(j) = current_value;
        all_results(j).label = label;
        all_results(j).metrics = metrics;
        all_results(j).ages = (metrics.cS.tb : metrics.cS.td)';
    end
    
    % --- c. 排序并绘图 ---
    fprintf('  3. 正在生成图表...\n');
    [~, sort_idx] = sort(values);
    all_results = all_results(sort_idx);
    
    % 调用指定的绘图函数
    config.plot_func(all_results, config, FIGURE_WIDTH_CM, OUTPUT_DIR);
    
    fprintf('  ✅ 参数 %s 的图表已生成。\n\n', upper(param_name));
end

fprintf('===== 所有敏感性分析图表已成功生成。 =====\n');


%% ========================================================================
%                     本地辅助函数：模拟与绘图
% =========================================================================

function metrics = run_simulation_for_file(file_path)
    % 描述: 加载指定的.mat文件，严格按照标准流程运行模拟，并计算所有
    %       可能用到的绘图指标，返回一个包含这些指标的结构体。
    
    % --- 加载数据 ---
    data = load(file_path, 'vfi_results', 'cS');
    vfi_results = data.vfi_results;
    cS = data.cS;

    % --- 初始化 ---
    rng(42); 
    nsim = 10000; 
    C_policy = vfi_results.C_policy;
    A_policy = vfi_results.A_policy;
    Q_policy = vfi_results.Q_policy;
    simGPY = ones(cS.tn, nsim);       
    simY_norm = zeros(cS.tn, nsim);    
    simR = zeros(cS.tn, nsim);         
    simW_norm = zeros(cS.tn, nsim);    
    simF_norm = zeros(cS.tn, nsim);    
    simC_norm = zeros(cS.tn, nsim);    
    simQ_norm = zeros(cS.tn, nsim);    
    simA = zeros(cS.tn, nsim);         
    sim_norm_cash = zeros(cS.tn, nsim);

    % --- 模拟外生冲击和收入过程 ---
    work_periods = cS.tr - cS.tb;
    for i1 = 1:floor(nsim/2)
        z_shocks = randn(work_periods, 1) * cS.smav;
        u_shocks_log = randn(work_periods, 1) * cS.smay;
        r_shocks = randn(cS.tn, 1) * cS.sigr;
        simGPY(2:work_periods+1, i1) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(z_shocks);
        simY_norm(1:work_periods, i1) = exp(u_shocks_log);
        simR(:, i1) = cS.rf + cS.mu + r_shocks;
        i2 = nsim/2 + i1;
        simGPY(2:work_periods+1, i2) = cS.f_y(2:work_periods+1) ./ cS.f_y(1:work_periods) .* exp(-z_shocks);
        simY_norm(1:work_periods, i2) = exp(-u_shocks_log);
        simR(:, i2) = cS.rf + cS.mu - r_shocks;
    end
    simY_norm(work_periods+1:cS.tn, :) = cS.ret_fac;

    % --- 迭代模拟生命周期决策 (归一化单位) ---
    for t = 1:cS.tn
        if t <= work_periods 
            norm_cash = simW_norm(t, :) + (1-cS.tau_y) * simY_norm(t, :);
        else 
            norm_pension_payout = (1-cS.tau_q) * simF_norm(t, :) * cS.pension_rate;
            norm_cash = simW_norm(t, :) + simY_norm(t, :) + norm_pension_payout;
        end
        sim_norm_cash(t, :) = norm_cash;
        
        Fc_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, C_policy(:,:,t), 'spline', 'linear');
        Fa_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, A_policy(:,:,t), 'spline', 'linear');
        Fq_interpolant = griddedInterpolant({cS.gcash, cS.gfund}, Q_policy(:,:,t), 'spline', 'linear');

        simC_norm(t, :) = Fc_interpolant(norm_cash, simF_norm(t, :));
        simA(t, :) = Fa_interpolant(norm_cash, simF_norm(t, :));
        simQ_norm(t, :) = Fq_interpolant(norm_cash, simF_norm(t, :));

        simA(t, isnan(simA(t,:))) = 0; % 已修正: 储蓄为0时alpha为0
        simA(t, :) = max(min(simA(t, :), 1), 0);
        simC_norm(t, :) = max(min(simC_norm(t, :), norm_cash), 0);
        simQ_norm(t, :) = max(min(simQ_norm(t, :), cS.Q_max), 0);
        
        if t <= work_periods
            total_outflow = simC_norm(t, :) + (1-cS.tau_y)*simQ_norm(t, :);
            idx_exceed = total_outflow > norm_cash;
            if any(idx_exceed)
                scale_factor = norm_cash(idx_exceed) ./ total_outflow(idx_exceed) * 0.9999;
                simC_norm(t, idx_exceed) = simC_norm(t, idx_exceed) .* scale_factor;
                simQ_norm(t, idx_exceed) = simQ_norm(t, idx_exceed) .* scale_factor;
            end
            liquid_sav_norm = norm_cash - simC_norm(t, :) - (1-cS.tau_y)*simQ_norm(t, :);
        else 
            simQ_norm(t,:) = 0;
            simC_norm(t,:) = min(simC_norm(t,:), norm_cash * 0.9999);
            liquid_sav_norm = norm_cash - simC_norm(t, :);
        end

        if t < cS.tn
            simS_norm = simA(t, :) .* liquid_sav_norm;
            simB_norm = liquid_sav_norm - simS_norm;
            portfolio_return_next = simB_norm * cS.rf + simS_norm .* simR(t, :);
            simW_norm(t+1, :) = portfolio_return_next ./ simGPY(t+1, :);
            if t <= work_periods
                simF_norm(t+1, :) = ((simF_norm(t, :) + simQ_norm(t, :)) * cS.rp) ./ simGPY(t+1, :);
            else
                simF_norm(t+1, :) = simF_norm(t, :);
            end
        end
    end

    % --- 转换为真实人民币单位 ---
    initial_income_yuan = 60000;
    simP_real = zeros(cS.tn, nsim);
    simP_real(1, :) = initial_income_yuan;
    for t = 1:(cS.tn - 1)
        simP_real(t+1, :) = simP_real(t, :) .* simGPY(t+1, :);
    end
    simQ_real = simQ_norm .* simP_real; % 新增

    % --- 计算所有比率和绝对值指标 ---
    sim_c_ratio = simC_norm ./ sim_norm_cash;
    sim_c_ratio(isinf(sim_c_ratio) | isnan(sim_c_ratio)) = NaN;

    % --- 封装结果 ---
    metrics.cS = cS;
    metrics.med_alpha = nanmedian(simA, 2);
    metrics.med_c_ratio = nanmedian(sim_c_ratio, 2);
    metrics.med_Q_real = nanmedian(simQ_real, 2); % 新增
end

% =========================================================================
%                     参数专属绘图函数 (已按要求修改 v4)
% =========================================================================

function plot_gamma_sensitivity(results, config, fig_width_cm, output_dir)
    fig = figure('Name', 'Gamma Sensitivity');
    set(fig, 'Units', 'centimeters', 'Position', [5, 5, fig_width_cm, 5]);
    
    colors = parula(length(results));
    retirement_age = results(1).metrics.cS.tr;
    tb = results(1).metrics.cS.tb;
    yuan_to_wanyuan = 1/10000;

    % 子图 (a): 风险资产配置比例
    ax1 = subplot(1, 3, 1);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_alpha, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    xline(retirement_age, '--', 'Color', [0.5 0.5 0.5], 'HandleVisibility', 'off');
    hold off;
    title('(a) $\alpha_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]); ylim([0, 1]);
    
    % 子图 (b): 个人养老金缴费绝对值
    ax2 = subplot(1, 3, 2);
    work_periods = retirement_age - tb;
    hold on;
    for i = 1:length(results)
        plot(results(i).ages(1:work_periods), results(i).metrics.med_Q_real(1:work_periods) * yuan_to_wanyuan, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    hold off;
    title('(b) $Q_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('CNY (10,000)'); grid on; xlim([tb, retirement_age]);
    
    % 子图 (c): 消费比例
    ax3 = subplot(1, 3, 3);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_c_ratio, 'LineWidth', 1.5, 'Color', colors(i,:), 'DisplayName', ['$\gamma = $ ', results(i).label]);
    end
    xline(retirement_age, '--', 'Color', [0.5 0.5 0.5], 'HandleVisibility', 'off');
    hold off;
    title('(c) $C_t/W_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]);
    
    lgd = legend(ax3, 'show','box','off','Location', 'best', 'FontSize', 7, 'Interpreter', 'latex');
    lgd.ItemTokenSize(1) = 10;
    
    output_filename = fullfile(output_dir, 'sensitivity_gamma.png');
    print(fig, output_filename, '-dpng', '-r400');
end

function plot_smav_sensitivity(results, config, fig_width_cm, output_dir)
    fig = figure('Name', 'Smav Sensitivity');
    set(fig, 'Units', 'centimeters', 'Position', [5, 5, fig_width_cm, 5]);
    
    colors = parula(length(results));
    retirement_age = results(1).metrics.cS.tr;
    tb = results(1).metrics.cS.tb;
    yuan_to_wanyuan = 1/10000;

    % 子图 (a): 风险资产配置比例
    ax1 = subplot(1, 3, 1);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_alpha, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    xline(retirement_age, '--', 'Color', [0.5 0.5 0.5], 'HandleVisibility', 'off');
    hold off;
    title('(a) $\alpha_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]); ylim([0, 1]);

    % 子图 (b): 个人养老金缴费绝对值
    ax2 = subplot(1, 3, 2);
    work_periods = retirement_age - tb;
    hold on;
    for i = 1:length(results)
        plot(results(i).ages(1:work_periods), results(i).metrics.med_Q_real(1:work_periods) * yuan_to_wanyuan, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    hold off;
    title('(b) $Q_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('CNY (10,000)'); grid on; xlim([tb, retirement_age]);

    % 子图 (c): 消费比例
    ax3 = subplot(1, 3, 3);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_c_ratio, 'LineWidth', 1.5, 'Color', colors(i,:), 'DisplayName', ['$\sigma_z = $ ', results(i).label]);
    end
    xline(retirement_age, '--', 'Color', [0.5 0.5 0.5], 'HandleVisibility', 'off');
    hold off;
    title('(c) $C_t/W_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]);

    lgd = legend(ax3, 'show','box','off','Location','best', 'FontSize', 7, 'Interpreter', 'latex');
    lgd.ItemTokenSize(1) = 10;
    
    output_filename = fullfile(output_dir, 'sensitivity_smav.png');
    print(fig, output_filename, '-dpng', '-r400');
end

function plot_sigr_sensitivity(results, config, fig_width_cm, output_dir)
    fig = figure('Name', 'Sigr Sensitivity');
    set(fig, 'Units', 'centimeters', 'Position', [5, 5, fig_width_cm, 5]);
    
    colors = parula(length(results));
    retirement_age = results(1).metrics.cS.tr;
    tb = results(1).metrics.cS.tb;
    yuan_to_wanyuan = 1/10000;
    
    % 子图 (a): 风险资产配置比例
    ax1 = subplot(1, 3, 1);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_alpha, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    xline(retirement_age, '--', 'Color', [0.5 0.5 0.5], 'HandleVisibility', 'off');
    hold off;
    title('(a) $\alpha_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]); ylim([0, 1]);

    % 子图 (b): 个人养老金缴费绝对值
    ax2 = subplot(1, 3, 2);
    work_periods = retirement_age - tb;
    hold on;
    for i = 1:length(results)
        plot(results(i).ages(1:work_periods), results(i).metrics.med_Q_real(1:work_periods) * yuan_to_wanyuan, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    hold off;
    title('(b) $Q_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('CNY (10,000)'); grid on;  xlim([tb, retirement_age]);

    % 子图 (c): 消费比例
    ax3 = subplot(1, 3, 3);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_c_ratio, 'LineWidth', 1.5, 'Color', colors(i,:), 'DisplayName', ['$\sigma_R = $ ', results(i).label]);
    end
    xline(retirement_age, '--', 'Color', [0.5 0.5 0.5], 'HandleVisibility', 'off');
    hold off;
    title('(c) $C_t/W_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]);
    
    lgd = legend(ax3, 'show','box','off','Location','best', 'FontSize', 7, 'Interpreter', 'latex');
    lgd.ItemTokenSize(1) = 10;
    
    output_filename = fullfile(output_dir, 'sensitivity_sigr.png');
    print(fig, output_filename, '-dpng', '-r400');
end

function plot_rp_sensitivity(results, config, fig_width_cm, output_dir)
    fig = figure('Name', 'Rp Sensitivity');
    set(fig, 'Units', 'centimeters', 'Position', [5, 5, fig_width_cm, 5]);
    
    colors = parula(length(results));
    retirement_age = results(1).metrics.cS.tr;
    tb = results(1).metrics.cS.tb;
    yuan_to_wanyuan = 1/10000;
    
    % 子图 (a): 风险资产配置比例
    ax1 = subplot(1, 3, 1);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_alpha, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    xline(retirement_age, '--', 'Color', [0.5 0.5 0.5], 'HandleVisibility', 'off');
    hold off;
    title('(a) $\alpha_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]); ylim([0, 1]);

    % 子图 (b): 个人养老金缴费绝对值
    ax2 = subplot(1, 3, 2);
    work_periods = retirement_age - tb;
    hold on;
    for i = 1:length(results)
        plot(results(i).ages(1:work_periods), results(i).metrics.med_Q_real(1:work_periods) * yuan_to_wanyuan, 'LineWidth', 1.5, 'Color', colors(i,:), 'DisplayName', ['$R_p = $ ', results(i).label]);
    end
    hold off;
    title('(b) $Q_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('CNY (10,000)'); grid on;  xlim([tb, retirement_age]);

    % 子图 (c): 消费比例
    ax3 = subplot(1, 3, 3);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_c_ratio, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    xline(retirement_age, '--', 'Color', [0.5 0.5 0.5], 'HandleVisibility', 'off');
    hold off;
    title('(c) $C_t/W_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]);
    
    lgd = legend(ax2, 'show','box','off','Location','best', 'FontSize', 7, 'Interpreter', 'latex');
    lgd.ItemTokenSize(1) = 10;
    
    output_filename = fullfile(output_dir, 'sensitivity_rp.png');
    print(fig, output_filename, '-dpng', '-r400');
end

function plot_tr_sensitivity(results, config, fig_width_cm, output_dir)
    fig = figure('Name', 'Tr Sensitivity');
    set(fig, 'Units', 'centimeters', 'Position', [5, 5, fig_width_cm, 5]);
    
    colors = parula(length(results));
    tb = results(1).metrics.cS.tb;
    yuan_to_wanyuan = 1/10000;
    
    % 子图 (a): 风险资产配置比例
    ax1 = subplot(1, 3, 1);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_alpha, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    hold off;
    title('(a) $\alpha_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]); ylim([0, 1]);

    % 子图 (b): 个人养老金缴费绝对值 (只绘制工作期)
    ax2 = subplot(1, 3, 2);
    hold on;
    for i = 1:length(results)
        work_periods = results(i).metrics.cS.tr - results(i).metrics.cS.tb;
        plot(results(i).ages(1:work_periods), results(i).metrics.med_Q_real(1:work_periods) * yuan_to_wanyuan, 'LineWidth', 1.5, 'Color', colors(i,:));
    end
    hold off;
    title('(b) $Q_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('CNY (10,000)'); grid on; xlim([tb, 65]); % 设定一个固定的右边界以便比较

    % 子图 (c): 消费比例
    ax3 = subplot(1, 3, 3);
    hold on;
    for i = 1:length(results)
        plot(results(i).ages, results(i).metrics.med_c_ratio, 'LineWidth', 1.5, 'Color', colors(i,:), 'DisplayName', ['$T_R = $ ', results(i).label]);
    end
    hold off;
    title('(c) $C_t/W_t$', 'Interpreter', 'latex'); xlabel('年龄'); ylabel('Ratio'); grid on; xlim([tb, 100]);
    
    lgd = legend(ax3, 'show','box','off','Location','best', 'FontSize', 7, 'Interpreter', 'latex');
    lgd.ItemTokenSize(1) = 10;
    
    output_filename = fullfile(output_dir, 'sensitivity_tr.png');
    print(fig, output_filename, '-dpng', '-r400');
end